In [ ]:
# @title Utils
test = False

from google.colab import drive

drive.mount("/content/drive")

path = "/content/drive/My Drive/Colab Notebooks/nonmyopic/"

botorch_version = "0.4.0"  # @param ['0.4.0', 'latest']

try:
    import botorch
except:
    if botorch_version == "latest":
        !pip install botorch
    else:
        !pip install botorch==0.4.0

import torch
import torch.optim as optim
from torch.optim import SGD
from tqdm import tqdm
import math
from math import exp
import itertools

import numpy as np

np.random.seed(1)

from matplotlib import cm
from matplotlib import pyplot as plt

import botorch
import gpytorch
from gpytorch.constraints import GreaterThan
from gpytorch.mlls import ExactMarginalLogLikelihood

from botorch.sampling.samplers import SobolQMCNormalSampler
from botorch import settings
from botorch.models import SingleTaskGP

import os
from copy import copy
from typing import Callable, Dict, List, Optional

color = {
    "C0": "#1f77b4",
    "C1": "#ff7f0e",
    "C2": "#2ca02c",
    "C3": "#d62728",
    "C4": "#9467bd",
    "C5": "#bcbd22",
    "C6": "#e377c2",
    "C7": "#17becf",
    # 'C8':'#bcbd22',
    # 'C9':'#17becf',
}

figsize = [7, 3]
plt.rcParams["figure.figsize"] = figsize

SMALL_SIZE = 14
MEDIUM_SIZE = 20
BIGGER_SIZE = 22


dpi = 100
imgtype = "png"

plt.rc("axes", titlesize=SMALL_SIZE)  # fontsize of the axes title
plt.rc("axes", labelsize=MEDIUM_SIZE)  # fontsize of the x and y labels
plt.rc("xtick", labelsize=16)  # fontsize of the tick labels
plt.rc("ytick", labelsize=16)  # fontsize of the tick labels
plt.rc("legend", fontsize=SMALL_SIZE)  # legend fontsize


"""
Utilities for Gaussian process (GP) inference.
"""

import numpy as np
from scipy.linalg import solve_triangular
from scipy.spatial.distance import cdist
import itertools


def kern_exp_quad_ard(xmat1, xmat2, ls, alpha):
    """
    Exponentiated quadratic kernel function with
    dimensionwise lengthscales if ls is an ndarray.
    """
    xmat1 = np.expand_dims(xmat1, axis=1)
    xmat2 = np.expand_dims(xmat2, axis=0)
    diff = xmat1 - xmat2
    diff /= ls
    norm = np.sum(diff**2, axis=-1) / 2.0
    kern = alpha**2 * np.exp(-norm)
    return kern


def kern_exp_quad_ard_sklearn(xmat1, xmat2, ls, alpha):
    """
    Exponentiated quadratic kernel function with dimensionwise lengthscales if ls is an
    ndarray, based on scikit-learn implementation.
    """
    dists = cdist(xmat1 / ls, xmat2 / ls, metric="sqeuclidean")
    exp_neg_norm = np.exp(-0.5 * dists)
    return alpha**2 * exp_neg_norm


def kern_exp_quad_ard_per(xmat1, xmat2, ls, alpha, pdims, period=2):
    """
    Exponentiated quadratic kernel function with
    - dimensionwise lengthscales if ls is an ndarray
    - periodic dimensions denoted by pdims. We assume that the period
    is 2.
    """
    xmat1 = np.expand_dims(xmat1, axis=1)
    xmat2 = np.expand_dims(xmat2, axis=0)
    diff = xmat1 - xmat2
    diff[..., pdims] = np.sin((np.pi * diff[..., pdims] / period) % (2 * np.pi))
    # diff[..., pdims] = np.cos( (np.pi/2) + (np.pi * diff[..., pdims] / period) )
    diff /= ls
    norm = np.sum(diff**2, axis=-1) / 2.0
    kern = alpha**2 * np.exp(-norm)

    return kern


def kern_exp_quad_noard(xmat1, xmat2, ls, alpha):
    """
    Exponentiated quadratic kernel function (aka squared exponential kernel aka
    RBF kernel).
    """
    kern = alpha**2 * kern_exp_quad_noard_noscale(xmat1, xmat2, ls)
    return kern


def kern_exp_quad_noard_noscale(xmat1, xmat2, ls):
    """
    Exponentiated quadratic kernel function (aka squared exponential kernel aka
    RBF kernel), without scale parameter.
    """
    distmat = squared_euc_distmat(xmat1, xmat2)
    norm = distmat / (2 * ls**2)
    exp_neg_norm = np.exp(-norm)
    return exp_neg_norm


def squared_euc_distmat(xmat1, xmat2, coef=1.0):
    """
    Distance matrix of squared euclidean distance (multiplied by coef) between
    points in xmat1 and xmat2.
    """
    return coef * cdist(xmat1, xmat2, "sqeuclidean")


def kern_distmat(xmat1, xmat2, ls, alpha, distfn):
    """
    Kernel for a given distmat, via passed in distfn (which is assumed to be fn
    of xmat1 and xmat2 only).
    """
    distmat = distfn(xmat1, xmat2)
    kernmat = alpha**2 * np.exp(-distmat / (2 * ls**2))
    return kernmat


def kern_simple_list(xlist1, xlist2, ls, alpha, base_dist=5.0):
    """
    Kernel for two lists containing elements that can be compared for equality.
    K(a,b) = 1 + base_dist if a and b are equal and K(a,b) = base_dist otherwise.
    """
    distmat = simple_list_distmat(xlist1, xlist2)
    distmat = distmat + base_dist
    kernmat = alpha**2 * np.exp(-distmat / (2 * ls**2))
    return kernmat


def simple_list_distmat(xlist1, xlist2, weight=1.0, additive=False):
    """
    Return distance matrix containing zeros when xlist1[i] == xlist2[j] and 0 otherwise.
    """
    prod_list = list(itertools.product(xlist1, xlist2))
    len1 = len(xlist1)
    len2 = len(xlist2)
    try:
        binary_mat = np.array([x[0] != x[1] for x in prod_list]).astype(int)
    except:
        # For cases where comparison returns iterable of bools
        binary_mat = np.array([all(x[0] != x[1]) for x in prod_list]).astype(int)

    binary_mat = binary_mat.reshape(len1, len2)

    if additive:
        distmat = weight + binary_mat
    else:
        distmat = weight * binary_mat

    return distmat


def get_product_kernel(kernel_list, additive=False):
    """Given a list of kernel functions, return product kernel."""

    def product_kernel(x1, x2, ls, alpha):
        """Kernel returning elementwise-product of kernel matrices from kernel_list."""
        mat_prod = kernel_list[0](x1, x2, ls, 1.0)
        for kernel in kernel_list[1:]:
            if additive:
                mat_prod = mat_prod + kernel(x1, x2, ls, 1.0)
            else:
                mat_prod = mat_prod * kernel(x1, x2, ls, 1.0)
        mat_prod = alpha**2 * mat_prod
        return mat_prod

    return product_kernel


def get_cholesky_decomp(k11_nonoise, sigma, psd_str):
    """Return cholesky decomposition."""
    if psd_str == "try_first":
        k11 = k11_nonoise + sigma**2 * np.eye(k11_nonoise.shape[0])
        try:
            return stable_cholesky(k11, False)
        except np.linalg.linalg.LinAlgError:
            return get_cholesky_decomp(k11_nonoise, sigma, "project_first")
    elif psd_str == "project_first":
        k11_nonoise = project_symmetric_to_psd_cone(k11_nonoise)
        return get_cholesky_decomp(k11_nonoise, sigma, "is_psd")
    elif psd_str == "is_psd":
        k11 = k11_nonoise + sigma**2 * np.eye(k11_nonoise.shape[0])
        return stable_cholesky(k11)


def stable_cholesky(mmat, make_psd=True, verbose=False):
    """Return a 'stable' cholesky decomposition of mmat."""
    if mmat.size == 0:
        return mmat
    try:
        lmat = np.linalg.cholesky(mmat)
    except np.linalg.linalg.LinAlgError as e:
        if not make_psd:
            raise e
        diag_noise_power = -11
        max_mmat = np.diag(mmat).max()
        diag_noise = np.diag(mmat).max() * 1e-11
        break_loop = False
        while not break_loop:
            try:
                lmat = np.linalg.cholesky(
                    mmat + ((10**diag_noise_power) * max_mmat) * np.eye(mmat.shape[0])
                )
                break_loop = True
            except np.linalg.linalg.LinAlgError:
                if diag_noise_power > -9:
                    if verbose:
                        print(
                            "\t*stable_cholesky failed with "
                            "diag_noise_power=%d." % (diag_noise_power)
                        )
                diag_noise_power += 1
            if diag_noise_power >= 5:
                print("\t*stable_cholesky failed: added diag noise = %e" % (diag_noise))
    return lmat


def project_symmetric_to_psd_cone(mmat, is_symmetric=True, epsilon=0):
    """Project symmetric matrix mmat to the PSD cone."""
    if is_symmetric:
        try:
            eigvals, eigvecs = np.linalg.eigh(mmat)
        except np.linalg.LinAlgError:
            print("\tLinAlgError encountered with np.eigh. Defaulting to eig.")
            eigvals, eigvecs = np.linalg.eig(mmat)
            eigvals = np.real(eigvals)
            eigvecs = np.real(eigvecs)
    else:
        eigvals, eigvecs = np.linalg.eig(mmat)
    clipped_eigvals = np.clip(eigvals, epsilon, np.inf)
    return (eigvecs * clipped_eigvals).dot(eigvecs.T)


def solve_lower_triangular(amat, b):
    """Solves amat*x=b when amat is lower triangular."""
    return solve_triangular_base(amat, b, lower=True)


def solve_upper_triangular(amat, b):
    """Solves amat*x=b when amat is upper triangular."""
    return solve_triangular_base(amat, b, lower=False)


def solve_triangular_base(amat, b, lower):
    """Solves amat*x=b when amat is a triangular matrix."""
    if amat.size == 0 and b.shape[0] == 0:
        return np.zeros((b.shape))
    else:
        return solve_triangular(amat, b, lower=lower)


def sample_mvn(mu, covmat, nsamp):
    """
    Sample from multivariate normal distribution with mean mu and covariance
    matrix covmat.
    """
    mu = mu.reshape(-1)
    ndim = len(mu)
    lmat = stable_cholesky(covmat)
    umat = np.random.normal(size=(ndim, nsamp))
    return lmat.dot(umat).T + mu


def gp_post(x_train, y_train, x_pred, ls, alpha, full_cov=True):
    """Compute parameters of GP posterior"""
    k11_nonoise = kern_exp_quad_noard(x_train, x_train, alpha=alpha, ls=ls)
    lmat = get_cholesky_decomp(k11_nonoise, 1e-2, "try_first")
    smat = solve_upper_triangular(lmat.T, solve_lower_triangular(lmat, y_train))
    k21 = kern_exp_quad_noard(x_pred, x_train, alpha=alpha, ls=ls)
    mu2 = k21.dot(smat)
    k22 = kern_exp_quad_noard(x_pred, x_pred, alpha=alpha, ls=ls)
    vmat = solve_lower_triangular(lmat, k21.T)
    k2 = k22 - vmat.T.dot(vmat)
    if full_cov is False:
        k2 = np.sqrt(np.diag(k2))
    return mu2, k2


def ground_truth(
    draw_true_model=False, n_dim=1, draw_3D=False, alpha=1, ls=math.sqrt(0.05)
):
    if n_dim == 1:
        xs = np.linspace(0, 1, 1000)[:, None]
        mean = np.zeros(xs.shape[0])
        cov = kern_exp_quad_noard(xs, xs, alpha=alpha, ls=ls)

        np.random.seed(24)
        ys = sample_mvn(mean, cov, 1).squeeze()
        if draw_true_model:
            plt.plot(xs, ys, color="blue", alpha=0.1)
        return xs, ys

    elif n_dim == 2:
        grid = 20j
        xs = np.mgrid[0:1:grid, 0:1:grid].reshape(2, -1).T
        mean = np.zeros(xs.shape[0])
        cov = kern_exp_quad_noard(xs, xs, alpha=alpha, ls=ls)

        np.random.seed(5)
        ys = sample_mvn(mean, cov, 1).squeeze()

        if draw_true_model:
            xpts = np.linspace(0, 1, int(abs(grid)))
            ypts = np.linspace(0, 1, int(abs(grid)))
            X, Y = np.meshgrid(xpts, ypts)
            resol = int(abs(grid))
            Z = ys.reshape(resol, resol).T

            if draw_3D:
                fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
                cf = ax.plot_surface(
                    X, Y, Z, cmap=cm.coolwarm, linewidth=0, antialiased=False
                )
                cbar = plt.colorbar(cf, fraction=0.046, pad=0.04)

            plt.figure(figsize=figsize)
            plt.title("Ground truth function")
            cf = plt.contourf(
                X,
                Y,
                Z,
                40,
                cmap=cm.coolwarm,
                zorder=0,
                levels=np.linspace(-3, 3, 40),
                extend="both",
            )
            cbar = plt.colorbar(cf, fraction=0.046, pad=0.04, ticks=range(-3, 3, 1))

            plt.show()

        return xs, ys

    else:
        raise


def p_loss(x):
    return torch.maximum(torch.abs(x - 0.5) - 0.5, torch.tensor([0]))


def func(train_x):
    alpha = 1
    ls = math.sqrt(0.05)

    xs, ys = ground_truth(draw_true_model=False, n_dim=n_dim, alpha=alpha, ls=ls)

    train_y, _ = gp_post(x_train=xs, y_train=ys, x_pred=train_x, alpha=alpha, ls=ls)

    train_y = torch.tensor(train_y, dtype=torch.float64)

    return train_y


def init_data(n_init=4):
    if n_dim == 1:
        train_x = torch.linspace(0, 1, n_init).reshape(-1, n_dim)
    elif n_dim == 2:
        n_init = complex(0, n_init)
        train_x = np.mgrid[0:1:n_init, 0:1:n_init].reshape(2, -1).T
        train_x = torch.from_numpy(train_x)
    else:
        raise

    train_x = torch.cat([train_x, prev_x.reshape(-1, n_dim)], axis=0)
    train_y = func(train_x)

    train_x = train_x.double()
    train_y = train_y.double()

    return train_x, train_y


def init_model(n_dim, prev_x, train_x, train_y, train=False):
    train_y = train_y[:, None]
    model = SingleTaskGP(train_X=train_x, train_Y=train_y)
    if train:
        model.likelihood.noise_covar.register_constraint("raw_noise", GreaterThan(1e-6))

    mll = ExactMarginalLogLikelihood(likelihood=model.likelihood, model=model)
    mll = mll.to(train_x)

    if not train:
        alpha = 1
        ls = math.sqrt(0.05)
        model.covar_module.base_kernel.lengthscale = ls
        model.covar_module.outputscale = alpha
        model.likelihood.noise = 1e-10
    else:
        optimizer = SGD([{"params": model.parameters()}], lr=0.05)
        NUM_EPOCHS = 20000

        model.train()

        for epoch in range(NUM_EPOCHS):
            optimizer.zero_grad()
            output = model(train_x)
            loss = -mll(output, model.train_targets)
            loss.backward()
            if (epoch + 1) % 2000 == 0:
                print(
                    f"Epoch {epoch+1:>3}/{NUM_EPOCHS} - Loss: {loss.item():>2.2} ",
                    "lengthscale: ",
                    model.covar_module.base_kernel.lengthscale.data,
                    f"outputscale: {model.covar_module.outputscale.item():>2.2f} ",
                    f"noise: {model.likelihood.noise.item():>2.5f}",
                )

            optimizer.step()

    model.eval()

    return model


def draw_posterior(n_dim, model):
    plt.figure(figsize=figsize)

    if n_dim == 1:
        test_x = torch.linspace(0, 1, 100)
        posterior = model.posterior(test_x)
        test_y = posterior.mean.squeeze()
        lower, upper = posterior.mvn.confidence_region()
        lower = lower.squeeze()
        upper = upper.squeeze()

        if len(test_y.shape) == 2:
            test_y = test_y[0, :]
            lower = lower[0, :]
            upper = upper[0, :]

        plt.plot(
            test_x.cpu().detach().numpy(),
            test_y.cpu().detach().numpy(),
            color["C1"],
            label="Best h-step Posterior mean (before observed data)",
        )
        plt.fill_between(
            test_x.cpu().detach().numpy(),
            lower.cpu().detach().numpy(),
            upper.cpu().detach().numpy(),
            alpha=0.25,
            color=color["C2"],
        )

    elif n_dim == 2:
        grid = 20j
        test_x = np.mgrid[0:1:grid, 0:1:grid].reshape(2, -1).T
        test_x = torch.tensor(test_x)

        posterior = model.posterior(test_x)
        test_y = posterior.mean.detach().numpy().squeeze()

        if len(test_y.shape) == 3:
            test_y = test_y[0, 0, :]
        elif len(test_y.shape) == 2:
            test_y = test_y[0, :]

        xpts = np.linspace(0, 1, int(abs(grid)))
        ypts = np.linspace(0, 1, int(abs(grid)))
        X, Y = np.meshgrid(xpts, ypts)
        resol = int(abs(grid))
        Z = test_y.reshape(resol, resol).T

        # levels = np.linspace(-1.5,2.5,40)

        cf = plt.contourf(
            X,
            Y,
            Z,
            40,
            cmap=cm.coolwarm,
            zorder=0,
            levels=np.linspace(-3, 3, 40),
            extend="both",
        )

        cbar = plt.colorbar(cf, fraction=0.046, pad=0.04, ticks=range(-3, 3, 1))

        plt.scatter(train_x[:, 0], train_x[:, 1], marker="*", color="black")

    else:
        raise


# #!/usr/bin/env python3
# # Copyright (c) Meta Platforms, Inc. and affiliates.
# #
# # This source code is licensed under the MIT license found in the
# # LICENSE file in the root directory of this source tree.

# r"""
# Gaussian Process Regression models based on GPyTorch models.

# These models are often a good starting point and are further documented in the
# tutorials.

# `SingleTaskGP`, `FixedNoiseGP`, and `HeteroskedasticSingleTaskGP` are all
# single-task exact GP models, differing in how they treat noise. They use
# relatively strong priors on the Kernel hyperparameters, which work best when
# covariates are normalized to the unit cube and outcomes are standardized (zero
# mean, unit variance).

# These models all work in batch mode (each batch having its own hyperparameters).
# When the training observations include multiple outputs, these models use
# batching to model outputs independently.

# These models all support multiple outputs. However, as single-task models,
# `SingleTaskGP`, `FixedNoiseGP`, and `HeteroskedasticSingleTaskGP` should be
# used only when the outputs are independent and all use the same training data.
# If outputs are independent and outputs have different training data, use the
# `ModelListGP`. When modeling correlations between outputs, use a multi-task
# model like `MultiTaskGP`.
# """

# from __future__ import annotations

# from typing import Any, List, Optional, Union

# import torch
# from botorch import settings
# from botorch.models.gpytorch import BatchedMultiOutputGPyTorchModel
# from botorch.models.transforms.input import InputTransform
# from botorch.models.transforms.outcome import Log, OutcomeTransform
# from botorch.models.utils import fantasize as fantasize_flag, validate_input_scaling
# from botorch.sampling.samplers import MCSampler
# from gpytorch.constraints.constraints import GreaterThan
# from gpytorch.distributions.multivariate_normal import MultivariateNormal
# from gpytorch.kernels.matern_kernel import MaternKernel
# from gpytorch.kernels.scale_kernel import ScaleKernel
# from gpytorch.kernels import RBFKernel
# from gpytorch.likelihoods.gaussian_likelihood import (
#     _GaussianLikelihoodBase,
#     FixedNoiseGaussianLikelihood,
#     GaussianLikelihood,
# )
# from gpytorch.likelihoods.likelihood import Likelihood
# from gpytorch.likelihoods.noise_models import HeteroskedasticNoise
# from gpytorch.means.constant_mean import ConstantMean
# from gpytorch.means.mean import Mean
# from gpytorch.mlls.noise_model_added_loss_term import NoiseModelAddedLossTerm
# from gpytorch.models.exact_gp import ExactGP
# from gpytorch.module import Module
# from gpytorch.priors.smoothed_box_prior import SmoothedBoxPrior
# from gpytorch.priors.torch_priors import GammaPrior
# from torch import Tensor


# MIN_INFERRED_NOISE_LEVEL = 1e-4


# class SingleTaskGP(BatchedMultiOutputGPyTorchModel, ExactGP):
#     r"""A single-task exact GP model.

#     A single-task exact GP using relatively strong priors on the Kernel
#     hyperparameters, which work best when covariates are normalized to the unit
#     cube and outcomes are standardized (zero mean, unit variance).

#     This model works in batch mode (each batch having its own hyperparameters).
#     When the training observations include multiple outputs, this model will use
#     batching to model outputs independently.

#     Use this model when you have independent output(s) and all outputs use the
#     same training data. If outputs are independent and outputs have different
#     training data, use the ModelListGP. When modeling correlations between
#     outputs, use the MultiTaskGP.

#     Example:
#         >>> train_X = torch.rand(20, 2)
#         >>> train_Y = torch.sin(train_X).sum(dim=1, keepdim=True)
#         >>> model = SingleTaskGP(train_X, train_Y)
#     """

#     def __init__(
#         self,
#         train_X: Tensor,
#         train_Y: Tensor,
#         likelihood: Optional[Likelihood] = None,
#         covar_module: Optional[Module] = None,
#         mean_module: Optional[Mean] = None,
#         outcome_transform: Optional[OutcomeTransform] = None,
#         input_transform: Optional[InputTransform] = None,
#     ) -> None:
#         r"""
#         Args:
#             train_X: A `batch_shape x n x d` tensor of training features.
#             train_Y: A `batch_shape x n x m` tensor of training observations.
#             likelihood: A likelihood. If omitted, use a standard
#                 GaussianLikelihood with inferred noise level.
#             covar_module: The module computing the covariance (Kernel) matrix.
#                 If omitted, use a `MaternKernel`.
#             mean_module: The mean function to be used. If omitted, use a
#                 `ConstantMean`.
#             outcome_transform: An outcome transform that is applied to the
#                 training data during instantiation and to the posterior during
#                 inference (that is, the `Posterior` obtained by calling
#                 `.posterior` on the model will be on the original scale).
#             input_transform: An input transform that is applied in the model's
#                 forward pass.
#         """
#         with torch.no_grad():
#             transformed_X = self.transform_inputs(
#                 X=train_X, input_transform=input_transform
#             )
#         if outcome_transform is not None:
#             train_Y, _ = outcome_transform(train_Y)
#         self._validate_tensor_args(X=transformed_X, Y=train_Y)
#         ignore_X_dims = getattr(self, "_ignore_X_dims_scaling_check", None)
#         validate_input_scaling(
#             train_X=transformed_X, train_Y=train_Y, ignore_X_dims=ignore_X_dims
#         )
#         self._set_dimensions(train_X=train_X, train_Y=train_Y)
#         train_X, train_Y, _ = self._transform_tensor_args(X=train_X, Y=train_Y)
#         if likelihood is None:
#             # noise_prior = GammaPrior(1.1, 0.05)
#             noise_prior = GammaPrior(concentration=0.5, rate=1)
#             noise_prior_mode = (noise_prior.concentration - 1) / noise_prior.rate
#             likelihood = GaussianLikelihood(
#                 noise_prior=noise_prior,
#                 batch_shape=self._aug_batch_shape,
#                 noise_constraint=GreaterThan(
#                     MIN_INFERRED_NOISE_LEVEL,
#                     transform=None,
#                     initial_value=noise_prior_mode,
#                 ),
#             )
#         else:
#             self._is_custom_likelihood = True
#         ExactGP.__init__(self, train_X, train_Y, likelihood)
#         if mean_module is None:
#             mean_module = ConstantMean(batch_shape=self._aug_batch_shape)
#         self.mean_module = mean_module
#         if covar_module is None:
#             covar_module = \
#                 ScaleKernel(
#                     # MaternKernel(
#                     #     nu=2.5,
#                     #     ard_num_dims=transformed_X.shape[-1],
#                     #     batch_shape=self._aug_batch_shape,
#                     #     lengthscale_prior=GammaPrior(3.0, 6.0),),
#                     RBFKernel(
#                         ard_num_dims=transformed_X.shape[-1],
#                         batch_shape=self._aug_batch_shape,
#                         lengthscale_prior=GammaPrior(3.0, 6.0),),
#                     batch_shape=self._aug_batch_shape,
#                     outputscale_prior=GammaPrior(2.0, 0.15),
#                 )
#             self._subset_batch_dict = {
#                 "likelihood.noise_covar.raw_noise": -2,
#                 "mean_module.raw_constant": -1,
#                 "covar_module.raw_outputscale": -1,
#                 "covar_module.base_kernel.raw_lengthscale": -3,
#             }
#         self.covar_module = covar_module
#         # TODO: Allow subsetting of other covar modules
#         if outcome_transform is not None:
#             self.outcome_transform = outcome_transform
#         if input_transform is not None:
#             self.input_transform = input_transform
#         self.to(train_X)

#     def forward(self, x: Tensor) -> MultivariateNormal:
#         if self.training:
#             x = self.transform_inputs(x)
#         mean_x = self.mean_module(x)
#         covar_x = self.covar_module(x)
#         return MultivariateNormal(mean_x, covar_x)

# HES

In [ ]:
# @title EHIG


def sang_sampler(posterior, num_samples=5):
    assert num_samples >= 1
    assert num_samples % 2 == 1

    sample = []
    mean = posterior.mean
    std = torch.sqrt(posterior.variance)
    std_coeff = np.linspace(0, 2, num_samples // 2 + 1)
    for s in std_coeff:
        if s == 0:
            sample.append(mean)
        else:
            sample.append(mean + s * std)
            sample.append(mean - s * std)

    return sample


def compute_ehig(
    xi, horizon, neighbor_size, seed, draw_last_posterior, sampler_name="sang"
):
    with settings.propagate_grads(state=True):
        for h in range(0, horizon + 1):
            pvs_x = xi[h - 1] if h > 0 else prev_x
            xi[h] = torch.sigmoid(xi[h]) * (neighbor_size * 2) + (pvs_x - neighbor_size)

            if torch.any(xi[h] < 0) or torch.any(xi[h] > 1):
                xi[h] = torch.clamp(xi[h], min=0, max=1)

        ehigs = 0
        sample_yi_on_xi_Di = {}
        prev_ind = np.ones(horizon) * -1
        p_f_on_Di = {}
        p_f_on_Di[0] = model

        if sampler_name == "sang":
            sampler = sang_sampler
        elif sampler_name == "sobol":
            sampler = SobolQMCNormalSampler(
                num_samples=4, resample=False, collapse_batch_dims=True, seed=seed
            )
        else:
            raise
        p_y0_on_x0_D0 = model.posterior(xi[0])
        sample_yi_on_xi_Di[0] = sampler(p_y0_on_x0_D0)

        # if horizon >= 3:
        #     iter_range = itertools.product(range(5), range(5), range(5))
        #     norm_const = 5*5*5
        if horizon >= 2:
            iter_range = itertools.product(range(5), range(5))
            norm_const = 5 * 5
        elif horizon == 1:
            iter_range = itertools.product(range(5))
            norm_const = 5
        else:
            print("horizon has to be larger than 0")
            raise

        for ind in iter_range:
            ind = ind + tuple([2] * (horizon - len(ind)))
            equal_bool = np.equal(ind, prev_ind)
            # finding the first false
            comp_ind = np.argmin(equal_bool)
            prev_ind = ind

            for i in range(horizon):
                if i >= comp_ind:
                    p_f_on_Di[i + 1] = p_f_on_Di[i].condition_on_observations(
                        xi[i],
                        # xi[ind[:i]],
                        sample_yi_on_xi_Di[i][ind[i]],
                    )

                    sample_yi_on_xi_Di[i + 1] = sampler(
                        p_f_on_Di[i + 1].posterior(xi[i + 1])
                    )
                    # p_f_on_Di[i+1].posterior(xi[ind[:i+1]]))

            if sampler_name == "sang":
                ehig = sample_yi_on_xi_Di[horizon][0].mean()
            elif sampler_name == "sobol":
                ehig = sample_yi_on_xi_Di[horizon].mean()
            else:
                raise

            ehigs = ehigs + ehig

        # compute distance
        # expand_xi = []
        # for h in range(horizon+1):
        #     total_dim_left = xi[horizon].shape[
        #         0:len(xi[horizon].shape)-len(xi[h].shape)]
        #     total_dim_left = total_dim_left[::-1]

        #     exp_xi = xi[h] # copy.deepcopy(xi[h])
        #     for dim_left in total_dim_left:
        #         exp_xi = torch.repeat_interleave(
        #             exp_xi[None,...], dim_left, dim=0)
        #     expand_xi.append(exp_xi)

        # total_dist = 0
        # for h in range(1, horizon+1):
        #     dist = ((expand_xi[h] - expand_xi[h-1])**2).sum(-1).sqrt()
        #     total_dist = total_dist + dist

        # cost = torch.ones(total_dist.shape)*100
        # cost[total_dist < neighbor_size] = 0

        # ehig = ehig + cost

        ehig = ehigs / norm_const

        if draw_last_posterior:
            draw_posterior(model=p_f_on_Di[horizon], n_dim=len(xi[0].shape))

        return xi, ehig.squeeze()


def argmax_ehig(
    model,
    prev_x,
    seed,
    horizon=10,
    ehig_opt_epoch=100,
    ehig_opt_lr=0.1,
    neighbor_size=0.1,
    use_lr_schedule=False,
):
    xi = []
    for h in range(horizon + 1):
        xi.append((torch.rand(dim_xi)).requires_grad_(True))

        if h == 0:
            dim_xi.insert(0, 5)
        elif h == 1:
            dim_xi.insert(0, 5)
        else:
            dim_xi.insert(0, 1)

    optimizer = optim.Adam(xi, lr=ehig_opt_lr)
    if use_lr_schedule:
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
            optimizer, T_max=10, eta_min=0.0001
        )

    result = []
    metric = []
    for epoch in tqdm(range(ehig_opt_epoch)):
        optimizer.zero_grad()
        xi_in = [element_xi.clone() for element_xi in xi]

        if epoch == ehig_opt_epoch - 1:
            draw_last_posterior = True
        else:
            draw_last_posterior = False
        xi_out, ehig = compute_ehig(
            xi_in,
            horizon=horizon,
            neighbor_size=neighbor_size,
            seed=seed,
            draw_last_posterior=draw_last_posterior,
        )

        ehig.backward(retain_graph=True)
        optimizer.step()
        if use_lr_schedule:
            scheduler.step()

        result.append([xi_out[i].detach() for i in range(horizon + 1)])
        metric.append(ehig.clone().detach().numpy())
        if epoch % 10 == 0:
            print(
                "x0: ",
                result[epoch][0].data.tolist(),
                ", a: ",
                result[epoch][-1].tolist(),
                # 'xi', xi_out,
                ", loss: ",
                metric[epoch].item(),
            )

        del xi_in
        del xi_out
        del ehig

    metric = np.array(metric)
    metric_round = metric.round(decimals=3)

    best = np.random.choice(np.where(metric_round == metric_round.min())[0])

    # best = torch.argmin(metric, dim=0).item()
    best_result = result[best]

    return best_result

#BudgetBO

In [28]:
# Objective and cost functions

from botorch.test_functions.synthetic import Ackley


def get_objective_cost_function(seed: int) -> Callable:
    def objective_function(X: Tensor) -> Tensor:
        X_unnorm = (2.0 * X) - 1.0
        ackley = Ackley(dim=1)
        objective_X = -ackley.evaluate_true(X_unnorm)

        # objective_X = func(X)
        return objective_X

    def cost_function(X: Tensor, prev_X: Tensor) -> Tensor:
        # X_unnorm = (2.0 * X) - 1.0
        # a, b, c = get_cost_function_parameters(seed=seed % 20)
        # ln_cost_X = a * torch.cos(b * (2 * pi) * (X_unnorm + c)).mean(dim=-1)
        # cost_X = torch.exp(ln_cost_X)

        cost_X = torch.abs(X - prev_X).sum(-1)
        return cost_X

    return [objective_function, cost_function]

In [16]:
# @title budgeted_bo_trial.py
from locale import Error
from typing import Callable, Dict, List, Optional

import logging
import numpy as np
import os
import sys
import time
import torch
from botorch.acquisition import ExpectedImprovement
from torch import Tensor

# from budgeted_bo.acquisition_functions.budgeted_multi_step_ei import BudgetedMultiStepExpectedImprovement
# from budgeted_bo.acquisition_functions.ei_puc import ExpectedImprovementPerUnitOfCost
# from budgeted_bo.utils import (
#     evaluate_obj_and_cost_at_X,
#     fit_model,
#     generate_initial_design,
#     get_suggested_budget,
#     optimize_acqf_and_get_suggested_point
# )


def budgeted_bo_trial(
    problem: str,
    algo: str,
    algo_params: Optional[Dict],
    trial: int,
    restart: bool,
    objective_function: Optional[Callable],
    cost_function: Optional[Callable],
    objective_cost_function: Optional[Callable],
    input_dim: int,
    n_init_evals: int,
    budget: float,
    n_max_iter: int = 200,
    ignore_failures: bool = False,
):
    # Make sure that objective and cost functions are passed
    if (objective_cost_function is None) and (
        objective_function is None or cost_function is None
    ):
        raise RuntimeError(
            "Both the objective and cost functions must be passed as inputs."
        )

    # Modify algo's name to account for hyperparameters
    if algo == "B-MS-EI":
        algo_id = algo + "_"

        for n in algo_params.get("lookahead_n_fantasies"):
            algo_id += str(n)

        algo_id += "_"

        if algo_params.get("refill_until_lower_bound_is_reached"):
            algo_id += "1"
        else:
            algo_id += "0"

        algo_id += "_" + str(int(budget))
    elif algo == "EI-PUC-CC":
        algo_id = algo + "_" + str(int(budget))
    else:
        algo_id = algo

    # Get script directory
    script_dir = os.path.dirname(os.path.realpath(sys.argv[0]))
    project_path = script_dir[:-11]
    results_folder = (
        project_path + "/experiments/results/" + problem + "/" + algo_id + "/"
    )

    if restart:
        # Check if training data is already available
        try:
            # Current available evaluations
            X = torch.tensor(np.loadtxt(results_folder + "X/X_" + str(trial) + ".txt"))
            objective_X = torch.tensor(
                np.loadtxt(
                    results_folder + "objective_X/objective_X_" + str(trial) + ".txt"
                )
            )
            cost_X = torch.tensor(
                np.loadtxt(results_folder + "cost_X/cost_X_" + str(trial) + ".txt")
            )

            # Historical best observed objective values and running times
            hist_best_obs_vals = list(
                np.loadtxt(results_folder + "best_obs_vals_" + str(trial) + ".txt")
            )
            running_times = list(
                np.loadtxt(
                    results_folder
                    + "running_times/running_times_"
                    + str(trial)
                    + ".txt"
                )
            )

            # Current best observed objective value
            best_obs_val = torch.tensor(hist_best_obs_vals[-1])

            iteration = len(hist_best_obs_vals) - 1
            print("Restarting experiment from available data.")

        except:
            # Initial evaluations
            X = generate_initial_design(
                num_samples=n_init_evals, input_dim=input_dim, seed=trial
            )
            objective_X, cost_X = evaluate_obj_and_cost_at_X(
                X=X,
                prev_X=prev_X,
                objective_function=objective_function,
                cost_function=cost_function,
                objective_cost_function=objective_cost_function,
            )

            # Current best objective value
            best_obs_val = objective_X.max().item()

            # Historical best observed objective values and running times
            hist_best_obs_vals = [best_obs_val]
            running_times = []

            iteration = 0
    else:
        # Initial evaluations
        X = generate_initial_design(
            num_samples=n_init_evals, input_dim=input_dim, seed=trial
        )

        objective_X, cost_X = evaluate_obj_and_cost_at_X(
            X=X,
            prev_X=torch.cat([X[0][None, :] + 0.01, X[:-1]]),
            objective_function=objective_function,
            cost_function=cost_function,
            objective_cost_function=objective_cost_function,
        )

        # Current best objective value
        best_obs_val = objective_X.max().item()

        # Historical best observed objective values and running times
        hist_best_obs_vals = [best_obs_val]
        running_times = []

        iteration = 0

    cumulative_cost = cost_X.sum().item()
    budget_plus_init_cost = cost_X[:n_init_evals].sum().item() + budget

    algo_params["init_budget"] = budget

    while cumulative_cost <= budget_plus_init_cost and iteration <= n_max_iter:
        iteration += 1
        print("Problem: " + problem)
        print("Sampling policy: " + algo_id)
        print("Trial: " + str(trial))
        print("Iteration: " + str(iteration))

        # New suggested point
        t0 = time.time()

        try:
            new_x = get_new_suggested_point(
                algo=algo,
                X=X,
                objective_X=objective_X,
                cost_X=cost_X,
                budget_left=budget_plus_init_cost - cumulative_cost,
                algo_params=algo_params,
            )
        except:
            if ignore_failures:
                print(
                    "An error ocurred when computing the next point to evaluate. Instead, a point will be chosen uniformly at random."
                )
                new_x = get_new_suggested_point(
                    algo="Random",
                    X=X,
                    objective_X=objective_X,
                    cost_X=cost_X,
                    budget_left=budget_plus_init_cost - cumulative_cost,
                    algo_params=algo_params,
                )
            else:
                raise Error(
                    "An error ocurred when computing the next point to evaluate."
                )

        t1 = time.time()
        running_times.append(t1 - t0)

        # Evaluate objective at new point
        objective_new_x, cost_new_x = evaluate_obj_and_cost_at_X(
            X=new_x,
            prev_X=X[-1],
            objective_function=objective_function,
            cost_function=cost_function,
            objective_cost_function=objective_cost_function,
        )

        # Update training data
        X = torch.cat([X, new_x], 0)
        objective_X = torch.cat([objective_X, objective_new_x], 0)
        cost_X = torch.cat([cost_X, cost_new_x], 0)

        # Update historical best observed objective values and cumulative cost
        cumulative_cost = cost_X.sum().item()
        best_obs_val = objective_X.max().item()
        hist_best_obs_vals.append(best_obs_val)
        print("Best value found so far: " + str(best_obs_val))
        print("Remaining budget: " + str(budget_plus_init_cost - cumulative_cost))

        # Save data
        if not os.path.exists(results_folder):
            os.makedirs(results_folder)
        if not os.path.exists(results_folder + "running_times/"):
            os.makedirs(results_folder + "running_times/")
        if not os.path.exists(results_folder + "X/"):
            os.makedirs(results_folder + "X/")
        if not os.path.exists(results_folder + "objective_X/"):
            os.makedirs(results_folder + "objective_X/")
        if not os.path.exists(results_folder + "cost_X/"):
            os.makedirs(results_folder + "cost_X/")
        np.savetxt(results_folder + "X/X_" + str(trial) + ".txt", X.numpy())
        np.savetxt(
            results_folder + "objective_X/objective_X_" + str(trial) + ".txt",
            objective_X.numpy(),
        )
        np.savetxt(
            results_folder + "cost_X/cost_X_" + str(trial) + ".txt", cost_X.numpy()
        )
        np.savetxt(
            results_folder + "best_obs_vals_" + str(trial) + ".txt",
            np.atleast_1d(hist_best_obs_vals),
        )
        np.savetxt(
            results_folder + "running_times/running_times_" + str(trial) + ".txt",
            np.atleast_1d(running_times),
        )

    return X, objective_X


def get_new_suggested_point(
    algo: str,
    X: Tensor,
    objective_X: Tensor,
    cost_X: Tensor,
    budget_left: float,
    algo_params: Optional[Dict] = None,
) -> Tensor:
    input_dim = X.shape[-1]
    algo_params["budget_left"] = budget_left

    if algo == "Random":
        return torch.rand([1, input_dim])
    elif algo == "B-MS-EI":
        # Model
        model = fit_model(
            X=X,
            objective_X=objective_X,
            cost_X=cost_X,
            training_mode="objective_and_cost",
            noiseless_obs=True,
        )

        # Acquisition function
        suggested_budget, lower_bound = get_suggested_budget(
            strategy="fantasy_costs_from_aux_policy",
            refill_until_lower_bound_is_reached=algo_params.get(
                "refill_until_lower_bound_is_reached"
            ),
            budget_left=budget_left,
            model=model,
            n_lookahead_steps=len(algo_params.get("lookahead_n_fantasies")) + 1,
            X=X,
            objective_X=objective_X,
            cost_X=cost_X,
            init_budget=algo_params.get("init_budget"),
            previous_budget=algo_params.get("current_budget"),
            lower_bound=algo_params.get("lower_bound"),
        )

        algo_params["current_budget"] = suggested_budget
        algo_params["current_budget_plus_cumulative_cost"] = (
            suggested_budget + cost_X.sum().item()
        )
        algo_params["lower_bound"] = lower_bound

        acquisition_function = BudgetedMultiStepExpectedImprovement(
            model=model,
            budget_plus_cumulative_cost=algo_params.get(
                "current_budget_plus_cumulative_cost"
            ),
            batch_size=1,
            lookahead_batch_sizes=[1 for _ in algo_params.get("lookahead_n_fantasies")],
            num_fantasies=algo_params.get("lookahead_n_fantasies"),
        )

    elif algo == "EI":
        # Model
        model = fit_model(
            X=X,
            objective_X=objective_X,
            cost_X=cost_X,
            training_mode="objective",
            noiseless_obs=True,
        )

        acquisition_function = ExpectedImprovement(
            model=model, best_f=objective_X.max().item()
        )

    elif algo == "EI-PUC":
        # Model
        model = fit_model(
            X=X,
            objective_X=objective_X,
            cost_X=cost_X,
            training_mode="objective_and_cost",
            noiseless_obs=True,
        )

        # Acquisition function
        acquisition_function = ExpectedImprovementPerUnitOfCost(
            model=model,
            best_f=objective_X.max().item(),
        )

    elif algo == "EI-PUC-CC":
        # Model
        model = fit_model(
            X=X,
            objective_X=objective_X,
            cost_X=cost_X,
            training_mode="objective_and_cost",
            noiseless_obs=True,
        )

        # Acquisition function
        acquisition_function = ExpectedImprovementPerUnitOfCost(
            model=model,
            best_f=objective_X.max().item(),
            cost_exponent=budget_left / algo_params.get("init_budget"),
        )

    standard_bounds = torch.tensor([[0.0] * input_dim, [1.0] * input_dim])

    new_x = optimize_acqf_and_get_suggested_point(
        acq_func=acquisition_function,
        bounds=standard_bounds,
        batch_size=1,
        algo_params=algo_params,
    )

    return new_x

In [30]:
# @title experiment_manager.py

import os
import sys
from tqdm import tqdm

# from budgeted_bo.budgeted_bo_trial import budgeted_bo_trial


def experiment_manager(
    problem: str,
    algo: str,
    algo_params: Dict,
    restart: bool,
    first_trial: int,
    last_trial: int,
    get_objective_cost_function: Callable,
    input_dim: int,
    n_init_evals: int,
    budget: float,
    n_max_iter: int = 200,
    ignore_failures: bool = False,
):
    Xs = []
    objective_Xs = []
    for trial in tqdm(range(first_trial, last_trial + 1)):
        get_objective_cost_function_output = get_objective_cost_function(seed=trial)

        if len(get_objective_cost_function_output) == 1:
            objective_cost_function = get_objective_cost_function_output[0]
            objective_function = None
            cost_function = None
        elif len(get_objective_cost_function_output) == 2:
            objective_cost_function = None
            objective_function = get_objective_cost_function_output[0]
            cost_function = get_objective_cost_function_output[1]

        X, objective_X = budgeted_bo_trial(
            problem=problem,
            algo=algo,
            algo_params=algo_params,
            trial=trial,
            restart=restart,
            objective_function=objective_function,
            cost_function=cost_function,
            objective_cost_function=objective_cost_function,
            input_dim=input_dim,
            n_init_evals=n_init_evals,
            budget=budget,
            n_max_iter=n_max_iter,
            ignore_failures=ignore_failures,
        )

    return Xs, objective_Xs

In [18]:
# @title Util.py

import logging
from copy import copy, deepcopy
from typing import Callable, Dict, Optional

import torch
from botorch import fit_gpytorch_model
from botorch.acquisition import AcquisitionFunction
from botorch.acquisition.analytic import ExpectedImprovement
from botorch.acquisition.multi_step_lookahead import (
    qMultiStepLookahead,
    warmstart_multistep,
)
from botorch.acquisition.objective import ScalarizedObjective
from botorch.generation.gen import get_best_candidates
from botorch.models import SingleTaskGP
from botorch.models.model import Model
from botorch.models.transforms.outcome import (
    ChainedOutcomeTransform,
    Log,
    Standardize,
)
from botorch.optim.optimize import optimize_acqf
from botorch.sampling.samplers import IIDNormalSampler
from botorch.utils.transforms import normalize
from gpytorch.constraints.constraints import Interval
from gpytorch.likelihoods.gaussian_likelihood import GaussianLikelihood
from gpytorch.mlls.exact_marginal_log_likelihood import ExactMarginalLogLikelihood
from torch import Tensor

# from budgeted_bo.acquisition_functions.ei_puc import ExpectedImprovementPerUnitOfCost
# from budgeted_bo.acquisition_function_optimization.custom_warmstart_multistep import custom_warmstart_multistep


def evaluate_obj_and_cost_at_X(
    X: Tensor,
    prev_X: Tensor,
    objective_function: Optional[Callable],
    cost_function: Optional[Callable],
    objective_cost_function: Optional[Callable],
) -> Tensor:
    if (objective_cost_function is None) and (
        objective_function is None or cost_function is None
    ):
        raise RuntimeError(
            "Both the objective and cost functions must be passed as inputs."
        )

    if objective_cost_function is not None:
        objective_X, cost_X = objective_cost_function(X)
    else:
        objective_X = objective_function(X)
        cost_X = cost_function(X, prev_X)

    return objective_X, cost_X


def fantasize_costs(
    algo: str,
    model: Model,
    n_steps: int,
    budget_left: float,
    init_budget: float,
    input_dim: int,
):
    """
    Fantasizes the observed costs when following a specified sampling
    policy for a given number of steps.
    """
    standard_bounds = torch.tensor([[0.0] * input_dim, [1.0] * input_dim])

    fantasy_costs = []
    fantasy_optimizers = []

    if algo == "EI-PUC_CC":
        for _ in range(n_steps):
            # Acquisition function
            y = torch.transpose(model.train_targets, -2, -1)
            y_original_scale = model.outcome_transform.untransform(y)[0]
            obj_vals = y_original_scale[..., 0]
            best_f = torch.max(obj_vals).item()
            cost_exponent = budget_left / init_budget

            aux_acq_func = ExpectedImprovementPerUnitOfCost(
                model=model,
                best_f=best_f,
                cost_exponent=cost_exponent,
            )

            # Get new point
            new_x, acq_value = optimize_acqf(
                acq_function=aux_acq_func,
                bounds=standard_bounds,
                q=1,
                num_restarts=5 * input_dim,
                raw_samples=50 * input_dim,
                options={
                    "batch_limit": 5,
                    "maxiter": 100,
                    "nonnegative": True,
                    "method": "L-BFGS-B",
                },
                return_best_only=True,
            )

            fantasy_optimizers.append(new_x.clone())

            # Fantasize objective and cost values
            sampler = IIDNormalSampler(
                num_samples=1, resample=True, collapse_batch_dims=True
            )
            posterior_new_x = model.posterior(new_x, observation_noise=True)
            fantasy_obs = sampler(posterior_new_x).squeeze(dim=0).detach()
            fantasy_costs.append(torch.exp(fantasy_obs[0, 1]).item())
            model = model.condition_on_observations(X=new_x, Y=fantasy_obs)

            # Update remaining budget
            budget_left -= fantasy_costs[-1]

    print("Fantasy costs:")
    fantasy_costs = torch.tensor(fantasy_costs)
    print(fantasy_costs)
    return fantasy_costs, fantasy_optimizers


def fit_model(
    X: Tensor,
    objective_X: Tensor,
    cost_X: Tensor,
    training_mode: str,
    noiseless_obs: bool = False,
):
    if training_mode == "objective":
        Y = objective_X
    elif training_mode == "cost":
        Y = cost_X
    elif training_mode == "objective_and_cost":
        Y = torch.cat(
            (objective_X.unsqueeze(dim=-1), torch.log(cost_X).unsqueeze(dim=-1)), dim=-1
        )

    if Y.ndim == 1:
        Y = Y.unsqueeze(-1)

    # Outcome transform
    standardize = Standardize(m=Y.shape[-1], batch_shape=Y.shape[:-2])
    if training_mode == "cost":
        log = Log()
        outcome_transform = ChainedOutcomeTransform(log=log, standardize=standardize)
    else:
        outcome_transform = standardize

    # Likelihood
    if noiseless_obs:
        _, aug_batch_shape = SingleTaskGP.get_batch_dimensions(
            train_X=X,
            train_Y=Y,
        )
        likelihood = GaussianLikelihood(
            batch_shape=aug_batch_shape,
            noise_constraint=Interval(lower_bound=1e-4, upper_bound=1e-3),
        )
    else:
        likelihood = None

    # Define model
    model = SingleTaskGP(
        train_X=X,
        train_Y=Y,
        likelihood=likelihood,
        outcome_transform=outcome_transform,
    )

    model.outcome_transform.eval()
    mll = ExactMarginalLogLikelihood(model.likelihood, model)
    fit_gpytorch_model(mll)

    return model


def generate_initial_design(
    num_samples: int, input_dim: int, sobol: bool = True, seed: int = None
):
    # generate training data
    if sobol:
        soboleng = torch.quasirandom.SobolEngine(
            dimension=input_dim, scramble=True, seed=seed
        )
        X = soboleng.draw(num_samples).to(dtype=torch.double)
    else:
        if seed is not None:
            old_state = torch.random.get_rng_state()
            torch.manual_seed(seed)
            X = torch.rand([num_samples, input_dim])
            torch.random.set_rng_state(old_state)
        else:
            X = torch.rand([num_samples, input_dim])
    return X


def get_suggested_budget(
    strategy: str,
    refill_until_lower_bound_is_reached: bool,
    budget_left: float,
    model: Model,
    n_lookahead_steps: int,
    X: Tensor,
    objective_X: Tensor,
    cost_X: Tensor,
    init_budget: float,
    previous_budget: Optional[float] = None,
    lower_bound: Optional[float] = None,
):
    """
    Computes the suggested budget to be used by the budgeted multi-step
    expected improvement acquisition function.
    """
    if (
        refill_until_lower_bound_is_reached
        and (lower_bound is not None)
        and (previous_budget - cost_X[-1] > lower_bound)
    ):
        suggested_budget = previous_budget - cost_X[-1].item()
        return suggested_budget, lower_bound

    if strategy == "fantasy_costs_from_aux_policy":
        # Fantasize the observed costs following the auxiliary acquisition function
        fantasy_costs, fantasy_optimizers = fantasize_costs(
            algo="EI-PUC_CC",
            model=deepcopy(model),
            n_steps=n_lookahead_steps,
            budget_left=copy(budget_left),
            init_budget=init_budget,
            input_dim=X.shape[-1],
        )

        # Suggested budget is the minimum between the sum of the fantasy costs
        # and the true remaining budget.
        suggested_budget = fantasy_costs.sum().item()
        lower_bound = fantasy_costs.min().item()
    suggested_budget = min(suggested_budget, budget_left)
    return suggested_budget, lower_bound


def optimize_acqf_and_get_suggested_point(
    acq_func: AcquisitionFunction,
    bounds: Tensor,
    batch_size: int,
    algo_params: Dict,
) -> Tensor:
    """Optimizes the acquisition function, and returns the candidate solution."""
    is_ms = isinstance(acq_func, qMultiStepLookahead)
    input_dim = bounds.shape[1]
    q = acq_func.get_augmented_q_batch_size(batch_size) if is_ms else batch_size
    raw_samples = 200 * input_dim * batch_size
    num_restarts = 10 * input_dim * batch_size

    # if is_ms:
    # raw_samples *= (len(algo_params.get("lookahead_n_fantasies")) + 1)
    # num_restarts *=  (len(algo_params.get("lookahead_n_fantasies")) + 1)

    # if algo_params.get("suggested_x_full_tree") is not None:
    #     batch_initial_conditions = custom_warmstart_multistep(
    #         acq_function=acq_func,
    #         bounds=bounds,
    #         num_restarts=num_restarts,
    #         raw_samples=raw_samples,
    #         full_optimizer=algo_params.get("suggested_x_full_tree"),
    #         algo_params=algo_params,
    #     )
    # else:
    batch_initial_conditions = None

    # try:
    candidates, acq_values = optimize_acqf(
        acq_function=acq_func,
        bounds=bounds,
        q=q,
        num_restarts=num_restarts,
        raw_samples=raw_samples,
        options={
            "batch_limit": 2,
            "maxiter": 200,
            "nonnegative": True,
            "method": "L-BFGS-B",
        },
        batch_initial_conditions=batch_initial_conditions,
        return_best_only=False,
        return_full_tree=is_ms,
    )
    # except:
    #     print(acq_func)
    #     print('sang_______________________________ \n')
    #     sang

    candidates = candidates.detach()
    if is_ms:
        # save all tree variables for multi-step initialization
        algo_params["suggested_x_full_tree"] = candidates.clone()
        candidates = acq_func.extract_candidates(candidates)

    acq_values_sorted, indices = torch.sort(acq_values.squeeze(), descending=True)
    print("Acquisition values:")
    print(acq_values_sorted)
    print("Candidates:")
    print(candidates[indices].squeeze())
    print(candidates.squeeze())

    new_x = get_best_candidates(batch_candidates=candidates, batch_values=acq_values)

    return new_x

### acquisition_function_optimization

In [19]:
# @title custom_warmstart_multistep.py

import random
import torch

from botorch import acquisition
from botorch.acquisition.analytic import PosteriorMean
from botorch.acquisition.multi_step_lookahead import (
    qMultiStepLookahead,
    warmstart_multistep,
)
from botorch.optim import optimize_acqf
from botorch.sampling.samplers import IIDNormalSampler
from copy import deepcopy
from torch import Tensor
from typing import List, Dict

# from budgeted_bo.acquisition_functions.budgeted_ei import BudgetedExpectedImprovement
# from budgeted_bo.acquisition_functions.budgeted_multi_step_ei import BudgetedMultiStepExpectedImprovement


def custom_warmstart_multistep(
    acq_function: qMultiStepLookahead,
    bounds: Tensor,
    num_restarts: int,
    raw_samples: int,
    full_optimizer: Tensor,
    algo_params: Dict,
) -> Tensor:
    batch_initial_conditions = warmstart_multistep(
        acq_function=acq_function,
        bounds=bounds,
        num_restarts=num_restarts,
        raw_samples=raw_samples,
        full_optimizer=full_optimizer,
    )

    n_initial_points = batch_initial_conditions.shape[0]
    random_index = random.randrange(n_initial_points)
    print(random_index)
    input_dim = batch_initial_conditions.shape[-1]
    batch_shape, shapes, sizes = acq_function.get_split_shapes(
        X=batch_initial_conditions
    )

    # Safe copy of model
    model = deepcopy(acq_function.model)
    obj_model = model.subset_output(idcs=[0])

    # Define optimization domain
    standard_bounds = torch.tensor([[0.0] * input_dim, [1.0] * input_dim])

    #
    aux_acq_func = PosteriorMean(model=obj_model)

    new_x, acq_value = optimize_acqf(
        acq_function=aux_acq_func,
        bounds=standard_bounds,
        q=1,
        num_restarts=5 * input_dim,
        raw_samples=100 * input_dim,
        options={
            "batch_limit": 5,
            "maxiter": 100,
            "method": "L-BFGS-B",
        },
        return_best_only=True,
    )

    i = 0
    for _ in range(sizes[0]):
        batch_initial_conditions[random_index, i, :] = new_x.clone().squeeze(0)
        i += 1

    # Fantasize objective and cost values
    sampler = IIDNormalSampler(num_samples=1, resample=True, collapse_batch_dims=True)
    posterior_new_x = model.posterior(new_x, observation_noise=True)
    fantasy_obs = sampler(posterior_new_x).squeeze(dim=0).detach()
    fantasy_cost = torch.exp(fantasy_obs[0, 1]).item()
    model = model.condition_on_observations(X=new_x, Y=fantasy_obs)

    n_lookahead_steps = len(algo_params.get("lookahead_n_fantasies")) - 1

    if n_lookahead_steps > 0:
        aux_acq_func = BudgetedMultiStepExpectedImprovement(
            model=model,
            budget_plus_cumulative_cost=algo_params.get(
                "current_budget_plus_cumulative_cost"
            )
            - fantasy_cost,
            batch_size=1,
            lookahead_batch_sizes=[1 for _ in range(n_lookahead_steps)],
            num_fantasies=[1 for _ in range(n_lookahead_steps)],
        )
    else:
        y = torch.transpose(model.train_targets, -2, -1)
        y_original_scale = model.outcome_transform.untransform(y)[0]
        obj_vals = y_original_scale[..., 0]
        best_f = torch.max(obj_vals).item()

        aux_acq_func = BudgetedExpectedImprovement(
            model=model,
            best_f=best_f,
            budget=algo_params.get("current_budget") - fantasy_cost,
        )

    new_x, acq_value = optimize_acqf(
        acq_function=aux_acq_func,
        bounds=standard_bounds,
        q=aux_acq_func.get_augmented_q_batch_size(1) if n_lookahead_steps > 0 else 1,
        num_restarts=5 * input_dim,
        raw_samples=100 * input_dim,
        options={
            "batch_limit": 1,
            "maxiter": 100,
            "method": "L-BFGS-B",
        },
        return_best_only=True,
        return_full_tree=True,
    )

    for j, size in enumerate(sizes[1:]):
        for _ in range(size):
            batch_initial_conditions[random_index, i, :] = new_x[j, :].clone()
            i += 1

    return batch_initial_conditions

### acquisition_functions

In [20]:
# @title budgeted_ei.py

#!/usr/bin/env python3

from __future__ import annotations

from typing import Optional, Union

import torch
from botorch.acquisition.analytic import AnalyticAcquisitionFunction
from botorch.acquisition.monte_carlo import MCAcquisitionFunction
from botorch.acquisition.objective import MCAcquisitionObjective, ScalarizedObjective
from botorch.models.model import Model
from botorch.sampling.samplers import MCSampler
from botorch.utils.objective import soft_eval_constraint
from botorch.utils.transforms import concatenate_pending_points, t_batch_mode_transform
from torch import Tensor
from torch.distributions import Normal


class BudgetedExpectedImprovement(AnalyticAcquisitionFunction):
    r"""Analytic Budgeted Expected Improvement.
    Computes the analytic expected improvement weighted by a probability of
    satisfying a budget constraint. The objective and (log-) cost are assumed
    to be independent and have Gaussian posterior distributions. Only supports
    the case `q=1`. The model should be two-outcome, with the first output
    corresponding to the objective and the second output corresponding to the
    log-cost.
    """

    def __init__(
        self,
        model: Model,
        best_f: Union[float, Tensor],
        budget: Union[float, Tensor],
        maximize: bool = True,
        # objective: Optional[ScalarizedObjective] = None,
        objective: Optional[MCAcquisitionObjective] = None,
    ) -> None:
        r"""Analytic Budgeted Expected Improvement.
        Args:
            model: A fitted two-outcome model, where the first output corresponds
                to the objective and the second one to the log-cost.
            best_f: Either a scalar or a `b`-dim Tensor (batch mode) representing
                the best function value observed so far (assumed noiseless).
            budget: Either a scalar or a `b`-dim Tensor (batch mode) representing
                the budget constraint.
            maximize: If True, consider the problem a maximization problem.
        """
        # use AcquisitionFunction constructor to avoid check for objective
        super(AnalyticAcquisitionFunction, self).__init__(model=model)
        self.objective = None
        self.maximize = maximize
        self.register_buffer("best_f", torch.as_tensor(best_f))
        self.register_buffer("budget", torch.as_tensor(budget))

    @t_batch_mode_transform(expected_q=1)
    def forward(self, X: Tensor) -> Tensor:
        r"""Evaluate Constrained Expected Improvement on the candidate set X.
        Args:
            X: A `(b) x 1 x d`-dim Tensor of `(b)` t-batches of `d`-dim design
                points each.
        Returns:
            A `(b)`-dim Tensor of Expected Improvement values at the given
            design points `X`.
        """
        posterior = self._get_posterior(X=X)
        means = posterior.mean  # (b) x 2
        sigmas = posterior.variance.sqrt().clamp_min(1e-6)  # (b) x 2

        # (b) x 1
        mean_obj = means[..., 0]
        sigma_obj = sigmas[..., 0]
        u = (mean_obj - self.best_f) / sigma_obj

        if not self.maximize:
            u = -u
        standard_normal = Normal(
            torch.zeros(1, device=u.device, dtype=u.dtype),
            torch.ones(1, device=u.device, dtype=u.dtype),
        )
        pdf_u = torch.exp(standard_normal.log_prob(u))
        cdf_u = standard_normal.cdf(u)
        ei = sigma_obj * (pdf_u + u * cdf_u)  # (b) x 1
        # (b) x 1
        prob_feas = self._compute_prob_feas(means=means[..., 1], sigmas=sigmas[..., 1])
        bc_ei = ei.mul(prob_feas)  # (b) x 1
        return bc_ei.squeeze(dim=-1)

    def _compute_prob_feas(self, means: Tensor, sigmas: Tensor) -> Tensor:
        r"""Compute feasibility probability for each batch of X.
        Args:
            X: A `(b) x 1 x d`-dim Tensor of `(b)` t-batches of `d`-dim design
                points each.
            means: A `(b) x 1`-dim Tensor of means.
            sigmas: A `(b) x 1`-dim Tensor of standard deviations.
        Returns:
            A `(b) x 1`-dim tensor of feasibility probabilities.
        """
        standard_normal = Normal(
            torch.zeros(1, device=means.device, dtype=means.dtype),
            torch.ones(1, device=means.device, dtype=means.dtype),
            validate_args=True,
        )
        prob_feas = standard_normal.cdf(
            (torch.log(self.budget.clamp_min(1e-6)) - means) / sigmas
        )
        prob_feas = torch.where(
            self.budget > 1e-6,
            prob_feas,
            torch.zeros(1, device=means.device, dtype=means.dtype),
        )
        return prob_feas


class qBudgetedExpectedImprovement(MCAcquisitionFunction):
    r"""Batch Budget-Constrained Expected Improvement.
    Computes the expected improvement weighted by a probability of satisfying
    a budget constraint. The model should be two-outcome, with the first output
    corresponding to the objective and the second output corresponding to the
    log-cost.
    """

    def __init__(
        self,
        model: Model,
        best_f: Union[float, Tensor],
        budget: Union[float, Tensor],
        sampler: Optional[MCSampler] = None,
        objective: Optional[MCAcquisitionObjective] = None,
        X_pending: Optional[Tensor] = None,
    ) -> None:
        r"""Batch Budgeted Expected Improvement.
        Args:
            model: A fitted two-outcome model, where the first output corresponds
                to the objective and the second one to the log-cost.
            best_f: Either a scalar or a `b`-dim Tensor (batch mode) representing
                the best function value observed so far (assumed noiseless).
            log_budget: Either a scalar or a `b`-dim Tensor (batch mode) representing
                the budget constraint.
            sampler: The sampler used to draw base samples. Defaults to
                `SobolQMCNormalSampler(num_samples=512, collapse_batch_dims=True)`.
            objective: The MCAcquisitionObjective under which the samples are
                evaluated. Defaults to `IdentityMCObjective()`.
            X_pending: A `batch_shape, m x d`-dim Tensor of `m` design points
                that have points that have been submitted for function evaluation
                but have not yet been evaluated.
        """
        # use AcquisitionFunction constructor to avoid check for objective
        super(MCAcquisitionFunction, self).__init__(model=model)
        self.sampler = sampler
        self.objective = None
        self.X_pending = X_pending
        self.register_buffer("best_f", torch.as_tensor(best_f))
        self.register_buffer("budget", torch.as_tensor(budget))

    @concatenate_pending_points
    @t_batch_mode_transform()
    def forward(self, X: Tensor) -> Tensor:
        r"""Evaluate Constrained Expected Improvement on the candidate set X.
        Args:
            X: A `(b) x q x d`-dim Tensor of `(b)` t-batches of `d`-dim design
                points each.
        Returns:
            A `(b)`-dim Tensor of Expected Improvement values at the given
            design points `X`.
        """
        posterior = self.model.posterior(X)
        samples = self.sampler(posterior)
        improvements = (samples[..., 0] - self.best_f).clamp_min(0)
        max_improvement = improvements.max(dim=-1, keepdim=True)[0]
        sum_costs = torch.exp(samples[..., 1]).sum(dim=-1, keepdim=True)
        smooth_feas_ind = soft_eval_constraint(lhs=sum_costs - self.budget)
        bc_ei = torch.mul(max_improvement, smooth_feas_ind).mean(dim=0)
        return bc_ei.squeeze(dim=-1)

In [21]:
# @title budgeted_multi_step_ei.py

#!/usr/bin/env python3

from __future__ import annotations

from typing import Any, Dict, List, Optional, Union

import torch
from botorch.acquisition.multi_step_lookahead import qMultiStepLookahead
from botorch.acquisition.objective import LinearMCObjective, ScalarizedObjective
from botorch.models.model import Model
from botorch.sampling.samplers import MCSampler, SobolQMCNormalSampler

# from budgeted_bo.acquisition_functions.budgeted_ei import (
#     BudgetedExpectedImprovement,
#     qBudgetedExpectedImprovement,
# )
# from budgeted_bo.samplers import PosteriorMeanSampler
from torch import Tensor
from torch.nn import Module


class BudgetedMultiStepExpectedImprovement(qMultiStepLookahead):
    r"""Budget-Constrained Multi-Step Look-Ahead Expected Improvement (one-shot optimization)."""

    def __init__(
        self,
        model: Model,
        budget_plus_cumulative_cost: Union[float, Tensor],
        batch_size: int,
        lookahead_batch_sizes: List[int],
        num_fantasies: Optional[List[int]] = None,
        samplers: Optional[List[MCSampler]] = None,
        X_pending: Optional[Tensor] = None,
        collapse_fantasy_base_samples: bool = True,
    ) -> None:
        r"""Budgeted Multi-Step Expected Improvement.
        Args:
            model: A fitted two-output model, where the first output corresponds to the
                objective, and the second one to the log-cost.
            budget: A value determining the budget constraint.
            batch_size: Batch size of the current step.
            lookahead_batch_sizes: A list `[q_1, ..., q_k]` containing the batch sizes for the
            `k` look-ahead steps.
            num_fantasies: A list `[f_1, ..., f_k]` containing the number of fantasy
                for the `k` look-ahead steps.
            samplers: A list of MCSampler objects to be used for sampling fantasies in
                each stage.
            X_pending: A `m x d`-dim Tensor of `m` design points that have points that
                have been submitted for function evaluation but have not yet been
                evaluated. Concatenated into `X` upon forward call. Copied and set to
                have no gradient.
            collapse_fantasy_base_samples: If True, collapse_batch_dims of the Samplers
                will be applied on fantasy batch dimensions as well, meaning that base
                samples are the same in all subtrees starting from the same level.
        """
        self.budget_plus_cumulative_cost = budget_plus_cumulative_cost
        self.batch_size = batch_size
        batch_sizes = [batch_size] + lookahead_batch_sizes

        # TODO: This objective is never really used.
        weights = torch.zeros(model.num_outputs, dtype=torch.double)
        weights[0] = 1.0

        use_mc_val_funcs = any(bs != 1 for bs in batch_sizes)

        if use_mc_val_funcs:
            objective = LinearMCObjective(weights=weights)

            valfunc_cls = [qBudgetedExpectedImprovement for _ in batch_sizes]

            inner_mc_samples = [128 for bs in batch_sizes]
        else:
            objective = ScalarizedObjective(weights=weights)

            valfunc_cls = [BudgetedExpectedImprovement for _ in batch_sizes]

            inner_mc_samples = None

        valfunc_argfacs = [
            budgeted_ei_argfac(
                budget_plus_cumulative_cost=self.budget_plus_cumulative_cost
            )
            for _ in batch_sizes
        ]

        # Set samplers
        if samplers is None:
            # The batch_range is not set here and left to sampler default of (0, -2),
            # meaning that collapse_batch_dims will be applied on fantasy batch dimensions.
            # If collapse_fantasy_base_samples is False, the batch_range is updated during
            # the forward call.
            samplers: List[MCSampler] = [
                PosteriorMeanSampler(collapse_batch_dims=True)
                if nf == 1
                else SobolQMCNormalSampler(
                    num_samples=nf, resample=False, collapse_batch_dims=True
                )
                for nf in num_fantasies
            ]

        super().__init__(
            model=model,
            batch_sizes=lookahead_batch_sizes,
            samplers=samplers,
            valfunc_cls=valfunc_cls,
            valfunc_argfacs=valfunc_argfacs,
            objective=objective,
            inner_mc_samples=inner_mc_samples,
            X_pending=X_pending,
            collapse_fantasy_base_samples=collapse_fantasy_base_samples,
        )


class budgeted_ei_argfac(Module):
    r"""Extract the best observed value and reamaining budget from the model."""

    def __init__(self, budget_plus_cumulative_cost: Union[float, Tensor]) -> None:
        super().__init__()
        self.budget_plus_cumulative_cost = budget_plus_cumulative_cost

    def forward(self, model: Model, X: Tensor) -> Dict[str, Any]:
        y = torch.transpose(model.train_targets, -2, -1)
        y_original_scale = model.outcome_transform.untransform(y)[0]
        obj_vals = y_original_scale[..., 0]
        log_costs = y_original_scale[..., 1]
        costs = torch.exp(log_costs)
        current_budget = self.budget_plus_cumulative_cost - costs.sum(
            dim=-1, keepdim=True
        )

        params = {
            "best_f": obj_vals.max(dim=-1, keepdim=True).values,
            "budget": current_budget,
        }
        return params

In [22]:
# @title custom_warmstart_multistep_old.py
from botorch import acquisition
import random
import torch

from botorch.acquisition.analytic import ExpectedImprovement, PosteriorMean
from botorch.acquisition.multi_step_lookahead import (
    qMultiStepLookahead,
    warmstart_multistep,
)
from botorch.optim import optimize_acqf
from botorch.sampling.samplers import IIDNormalSampler
from torch import Tensor
from typing import List, Dict


def custom_warmstart_multistep(
    acq_function: qMultiStepLookahead,
    bounds: Tensor,
    num_restarts: int,
    raw_samples: int,
    full_optimizer: Tensor,
    algo_params: Dict,
) -> Tensor:
    batch_initial_conditions = warmstart_multistep(
        acq_function=acq_function,
        bounds=bounds,
        num_restarts=num_restarts,
        raw_samples=raw_samples,
        full_optimizer=full_optimizer,
    )

    n_initial_points = batch_initial_conditions.shape[0]
    random_index = random.randrange(n_initial_points)
    input_dim = batch_initial_conditions.shape[-1]
    batch_shape, shapes, sizes = acq_function.get_split_shapes(
        X=batch_initial_conditions
    )

    # Copy of objective model
    obj_model = acq_function.model.subset_output(idcs=[0])

    # Define optimization domain
    standard_bounds = torch.tensor([[0.0] * input_dim, [1.0] * input_dim])

    i = 0

    for size in sizes:
        # Auxiliar acquisition fucntion
        aux_acq_func_id = "PM"
        if aux_acq_func_id == "PM":
            aux_acq_func = PosteriorMean(model=obj_model)

        elif aux_acq_func_id == "EI":
            standardized_obj_vals = obj_model.train_targets
            obj_vals = obj_model.outcome_transform.untransform(standardized_obj_vals)[0]
            best_f = torch.max(obj_vals).item()

            aux_acq_func = ExpectedImprovement(model=obj_model, best_f=best_f)

        # Get new point
        new_x, acq_value = optimize_acqf(
            acq_function=aux_acq_func,
            bounds=standard_bounds,
            q=1,
            num_restarts=5 * input_dim,
            raw_samples=100 * input_dim,
            options={
                "batch_limit": 5,
                "maxiter": 100,
                "method": "L-BFGS-B",
            },
            return_best_only=True,
        )

        print(new_x)
        for _ in range(size):
            batch_initial_conditions[random_index, i, :] = new_x.clone().squeeze(0)
            i += 1

        # Fantasize objective and cost values
        obj_sampler = IIDNormalSampler(
            num_samples=1, resample=True, collapse_batch_dims=True
        )
        obj_post_X = obj_model.posterior(new_x, observation_noise=True)
        fantasy_obj_val = obj_sampler(obj_post_X).squeeze(dim=0)
        obj_model = obj_model.condition_on_observations(
            X=new_x, Y=fantasy_obj_val.detach()
        )

    print(batch_initial_conditions[random_index])
    return batch_initial_conditions

In [23]:
# @title ei_puc.py

#!/usr/bin/env python3

from __future__ import annotations

from typing import Optional, Union

import torch
from botorch.acquisition.analytic import AnalyticAcquisitionFunction
from botorch.acquisition.objective import ScalarizedObjective
from botorch.models.model import Model
from botorch.utils.transforms import concatenate_pending_points, t_batch_mode_transform
from torch import Tensor
from torch.distributions import Normal


class ExpectedImprovementPerUnitOfCost(AnalyticAcquisitionFunction):
    r"""Expected Improvement Per Unit of Cost (analytic)."""

    def __init__(
        self,
        model: Model,
        best_f: Union[float, Tensor],
        cost_exponent: Union[float, Tensor] = 1.0,
        maximize: bool = True,
        objective: Optional[ScalarizedObjective] = None,
    ) -> None:
        r"""Single-outcome Expected Improvement (analytic).
        Args:
            model: A fitted two-outcome model, where the first output corresponds
                to the objective and the second one to the log-cost.
            best_f: Either a scalar or a `b`-dim Tensor (batch mode) representing
                the best function value observed so far (assumed noiseless).
            maximize: If True, consider the problem a maximization problem.
        """
        # use AcquisitionFunction constructor to avoid check for objective
        super(AnalyticAcquisitionFunction, self).__init__(model=model)
        self.objective = None
        self.maximize = maximize
        self.register_buffer("best_f", torch.as_tensor(best_f))
        self.register_buffer("cost_exponent", torch.as_tensor(cost_exponent))

    @t_batch_mode_transform(expected_q=1, assert_output_shape=False)
    def forward(self, X: Tensor) -> Tensor:
        r"""Evaluate Expected Improvement Per Unit of Cost on the candidate set X.
        Args:
            X: A `b1 x ... bk x 1 x d`-dim batched tensor of `d`-dim design points.
                Expected Improvement is computed for each point individually,
                i.e., what is considered are the marginal posteriors, not the
                joint.
        Returns:
            A `b1 x ... bk`-dim tensor of Expected Improvement Per Unit of Cost values
            at the given design points `X`.
        """
        posterior = self._get_posterior(X=X)
        # posterior = self.model.posterior(X=X)
        means = posterior.mean  # (b) x 2
        vars = posterior.variance.clamp_min(1e-6)  # (b) x 2
        stds = vars.sqrt()

        # (b) x 1
        mean_obj = means[..., 0]
        std_obj = stds[..., 0]
        u = (mean_obj - self.best_f) / std_obj

        if not self.maximize:
            u = -u
        standard_normal = Normal(
            torch.zeros(1, device=u.device, dtype=u.dtype),
            torch.ones(1, device=u.device, dtype=u.dtype),
        )
        pdf_u = torch.exp(standard_normal.log_prob(u))
        cdf_u = standard_normal.cdf(u)
        ei = std_obj * (pdf_u + u * cdf_u)  # (b) x 1
        # (b) x 1
        eic = torch.exp(
            -(self.cost_exponent * means[..., 1])
            + 0.5 * (torch.square(self.cost_exponent) * vars[..., 1])
        )
        ei_puc = ei.mul(eic)  # (b) x 1
        return ei_puc.squeeze(dim=-1)

In [24]:
# @title multi_step_ei.py
#!/usr/bin/env python3

from __future__ import annotations

from typing import Any, Dict, List, Optional

from botorch.acquisition.analytic import ExpectedImprovement
from botorch.acquisition.monte_carlo import qExpectedImprovement
from botorch.acquisition.multi_step_lookahead import qMultiStepLookahead
from botorch.models.model import Model
from botorch.sampling.samplers import MCSampler, SobolQMCNormalSampler
from torch import Tensor


class MultiStepLookaheadEI(qMultiStepLookahead):
    r"""Multi-Step Look-Ahead Expected Improvement (one-shot optimization)."""

    def __init__(
        self,
        model: Model,
        batch_size: int,
        lookahead_batch_sizes: List[int],
        num_fantasies: Optional[List[int]] = None,
        samplers: Optional[List[MCSampler]] = None,
        X_pending: Optional[Tensor] = None,
        collapse_fantasy_base_samples: bool = True,
    ) -> None:
        r"""Multi-Step Look-Ahead Expected Improvement.
        Args:
            model: A single-output Model of appropriate batch size.
            batch_size: Batch size for the current step.
            lookahead_batch_sizes: A list `[q_1, ..., q_k]` containing the batch sizes for the
                `k` look-ahead steps.
            num_fantasies: A list `[f_1, ..., f_k]` containing the number of fantasy
                points to use for the `k` look-ahead steps.
            samplers: A list of MCSampler objects to be used for sampling fantasies in
                each stage.
            X_pending: A `m x d`-dim Tensor of `m` design points that have points that
                have been submitted for function evaluation but have not yet been
                evaluated. Concatenated into `X` upon forward call. Copied and set to
                have no gradient.
            collapse_fantasy_base_samples: If True, collapse_batch_dims of the Samplers
                will be applied on fantasy batch dimensions as well, meaning that base
                samples are the same in all subtrees starting from the same level.
        """
        self.batch_size = batch_size
        batch_sizes = [batch_size] + lookahead_batch_sizes

        if any(bs != 1 for bs in batch_sizes):
            valfunc_cls = [qExpectedImprovement for _ in batch_sizes]
            inner_mc_samples = [512 for bs in batch_sizes]
        else:
            valfunc_cls = [ExpectedImprovement for _ in batch_sizes]
            inner_mc_samples = None

        valfunc_argfacs = [multi_step_ei_argfac for _ in batch_sizes]

        # Set samplers
        if samplers is None:
            # The batch_range is not set here and left to sampler default of (0, -2),
            # meaning that collapse_batch_dims will be applied on fantasy batch dimensions.
            # If collapse_fantasy_base_samples is False, the batch_range is updated during
            # the forward call.
            samplers: List[MCSampler] = [
                PosteriorMeanSampler(collapse_batch_dims=True)
                if nf == 1
                else SobolQMCNormalSampler(
                    num_samples=nf, resample=False, collapse_batch_dims=True
                )
                for nf in num_fantasies
            ]

        super().__init__(
            model=model,
            batch_sizes=lookahead_batch_sizes,
            samplers=samplers,
            valfunc_cls=valfunc_cls,
            valfunc_argfacs=valfunc_argfacs,
            inner_mc_samples=inner_mc_samples,
            X_pending=X_pending,
            collapse_fantasy_base_samples=collapse_fantasy_base_samples,
        )


def multi_step_ei_argfac(model: Model, X: Tensor) -> Dict[str, Any]:
    y = model.train_targets
    y_original_scale = model.outcome_transform.untransform(y)[0]
    obj_vals = y_original_scale
    params = {
        "best_f": obj_vals.max(dim=-1).values,
    }
    return params

### Samplers

In [25]:
# @title posterior_mean_sampler.py
from typing import Optional, Tuple

import torch
from botorch.posteriors import Posterior
from botorch.sampling.samplers import MCSampler, _check_shape_changed
from botorch.utils.sampling import manual_seed
from torch import Tensor


class PosteriorMeanSampler(MCSampler):
    r"""Sampler for MC base samples using iid N(0,1) samples."""

    def __init__(
        self,
        collapse_batch_dims: bool = True,
        batch_range: Tuple[int, int] = (0, -2),
    ) -> None:
        r"""Sampler for MC base samples using iid `N(0,1)` samples.
        Args:
            num_samples: The number of samples to use.
            resample: If `True`, re-draw samples in each `forward` evaluation -
                this results in stochastic acquisition functions (and thus should
                not be used with deterministic optimization algorithms).
            seed: The seed for the RNG. If omitted, use a random seed.
            collapse_batch_dims: If True, collapse the t-batch dimensions to
                size 1. This is useful for preventing sampling variance across
                t-batches.
            batch_range: The range of t-batch dimensions in the `base_sample_shape`
                used by `collapse_batch_dims`. The t-batch dims are
                batch_range[0]:batch_range[1]. By default, this is (0, -2),
                for the case where the non-batch dimensions are -2 (q) and
                -1 (d) and all dims in the front are t-batch dims.
        """
        super().__init__(batch_range=batch_range)
        self._sample_shape = torch.Size([1])
        self.collapse_batch_dims = collapse_batch_dims

    def _construct_base_samples(self, posterior: Posterior, shape: torch.Size) -> None:
        r"""Generate iid `N(0,1)` base samples (if necessary).
        This function will generate a new set of base samples and set the
        `base_samples` buffer if one of the following is true:
        - `resample=True`
        - the MCSampler has no `base_samples` attribute.
        - `shape` is different than `self.base_samples.shape` (if
            `collapse_batch_dims=True`, then batch dimensions of will be
            automatically broadcasted as necessary). This shape is expected to
            be `sample_shape + base_sample_shape`, where `base_sample_shape` has been
            adjusted to account for `collapse_batch_dims` (i.e., the output
            of the function `_get_base_sample_shape`).
        Args:
            posterior: The Posterior for which to generate base samples.
            shape: The shape of the base samples to construct.
        """
        if _check_shape_changed(self.base_samples, self.batch_range, shape) or (
            not self.collapse_batch_dims and shape != self.base_samples.shape
        ):
            base_samples = torch.zeros(
                shape, device=posterior.device, dtype=posterior.dtype
            )
            self.register_buffer("base_samples", base_samples)
        elif self.collapse_batch_dims and shape != self.base_samples.shape:
            self.base_samples = self.base_samples.view(shape)
        if self.base_samples.device != posterior.device:
            self.to(device=posterior.device)  # pragma: nocover
        if self.base_samples.dtype != posterior.dtype:
            self.to(dtype=posterior.dtype)

In [26]:
# @title parametric_cost_function.py
import torch
from botorch.utils.sampling import manual_seed
from math import pi


def get_cost_function_parameters(seed):
    with manual_seed(seed=seed):
        a = torch.rand(1)
        a = 0.75 * a + 0.75

        b = torch.rand(1)
        b = 2.0 * b + 1.0

        c = torch.rand(1)
        c = 2 * pi * c
    return a, b, c

# Cost-constraint BO

In [ ]:
# https://github.com/ericlee0803/lookahead_release
!pip install qmcpy==0.7

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.0 MB 25.4 MB/s 
  Created wheel for qmcpy: filename=qmcpy-0.7-cp37-cp37m-linux_x86_64.whl size=4159290 sha256=f44dc4ace8bdbf4edc75cb9955f9d7e308d0d89469b401dfbdb1e94a0e12cb7e
  Stored in directory: /root/.cache/pip/wheels/67/51/2b/bb041d71a5c4e01ddad2c3a569c74746c3c2adb55aaf1e2b7b
Successfully built qmcpy


In [ ]:
import numpy as np
import qmcpy as qp
import multiprocessing as mp
import copy
import time

from scipy.optimize import minimize
from scipy.stats import norm
from abc import ABCMeta

from multiprocessing import Pool
from scipy.linalg import solve_triangular, cho_factor, cho_solve, cholesky
from scipy.spatial.distance import squareform, pdist, cdist

import scipy as sp
import pprint
import scipy.linalg
import scipy.optimize
import os
import csv

## lookahead/acquisitions/

In [ ]:
# @title acquisition_function_interface.py
# Super simple acquisition function interface to be used


class AcquisitionFunctionInterface(object):
    def __init__(self, gaussian_process, opt_domain, **kwargs):
        self.gaussian_process = gaussian_process
        self.opt_domain = opt_domain
        self.opt_domain_constraints = np.array(
            [
                (interval.min, interval.max)
                for interval in self.opt_domain.get_bounding_box()
            ]
        )
        self.horizon = 1

    # There are some seeds that could be problems for this, I think (if the point [0, 0] is in)
    def low_discrepancy_points(self, num, seed=1234):
        measure = qp.Gaussian(qp.Sobol(dimension=self.horizon, seed=seed))
        points = measure.gen_samples(n=2 ** np.ceil(np.log2(num)))[:num]
        points = np.nan_to_num(points)
        assert not np.any(np.isnan(points))
        return points

    def evaluate_at_point_list(self, points_to_evaluate):
        pass

    def joint_function_gradient_eval(self, points_to_evaluate):
        pass

    def next_point_grid(self, num_grid_points=400):
        grid_points = self.opt_domain.generate_quasi_random_points_in_domain(
            num_grid_points
        )
        vals = self.evaluate_at_point_list(grid_points)
        idx = np.argmax(vals)
        return grid_points[[idx], :]

    def next_point_grad(self, num_restarts=5):
        ymin = np.inf
        num_guess = 10 * self.opt_domain_constraints.shape[0]
        init_guesses = self.opt_domain.generate_quasi_random_points_in_domain(num_guess)
        guesses = self.evaluate_at_point_list(init_guesses)
        idx_guesses = np.argsort(guesses)
        for i in range(num_restarts):
            x0 = init_guesses[[idx_guesses[-i]], :]
            opt_result = minimize(
                fun=self.joint_function_gradient_eval,
                method="L-BFGS-B",
                x0=x0,
                jac=True,
                bounds=self.opt_domain_constraints,
                options={"maxiter": 40},
            )
            if opt_result.fun < ymin:
                x = opt_result.x
                ymin = opt_result.fun
        return x[np.newaxis, :]

    def next_point(self):
        if self.opt_domain_constraints.shape[0] > 2:
            return self.next_point_grad()
        else:
            return self.next_point_grid()

In [ ]:
# @title acquisition_optimizer_bo.py


class BayesOptAcquisitionOptimizer:
    """
    Optimizes acquisition function w/ bayesian optimization
    """

    def __init__(self, gaussian_process, acquisition_handle, opt_domain):
        self.gaussian_process = gaussian_process
        self.acquisition_handle = acquisition_handle
        self.opt_domain = opt_domain

    def get_initial_design(self):
        # maximize the expected improvement first
        ei = ExpectedImprovement(self.gaussian_process, self.opt_domain)
        x_ei_opt = ei.next_point()
        X = self.opt_domain.generate_quasi_random_points_in_domain(5)
        X = np.vstack([X, x_ei_opt])
        return X

    def get_sample_point(self):
        """
        Get next sample point through BO, using 100d BO iterations. We want to maximize, meaning we have to multiple by
        negative when calling the handle
        """
        budget = self.gaussian_process.d * 5
        initial_design = self.get_initial_design()
        X = initial_design
        Y = -1 * self.acquisition_handle(X)

        for i in range(budget):
            # Normalize Y
            Y_train = np.copy(Y)
            Y_train = (Y_train - np.mean(Y_train)) / np.std(Y_train)
            gp = GaussianProcessSimple(X, Y_train)
            gp.train()

            ei = ExpectedImprovement(gp, self.opt_domain)
            xx = ei.next_point()
            yy = -1 * self.acquisition_handle(xx)
            X = np.vstack([X, xx])
            Y = np.concatenate([Y, yy])

        idx = np.argmin(Y)
        return X[[idx], :]

In [ ]:
# @title expected_improvement.py


class ExpectedImprovement(AcquisitionFunctionInterface):
    def __init__(self, gaussian_process, opt_domain):
        super().__init__(gaussian_process, opt_domain)

    def evaluate_at_point_list(self, points_to_evaluate):
        mu = self.gaussian_process.mean(points_to_evaluate)
        sigma2 = self.gaussian_process.variance(points_to_evaluate)
        sigma = np.sqrt(sigma2)
        xhist, yhist = self.gaussian_process.get_historical_data()
        fmin = np.min(yhist)
        z = (fmin - mu) / sigma
        cdf_z = norm.cdf(z)
        pdf_z = norm.pdf(z)
        return sigma * np.fmax(0.0, z * cdf_z + pdf_z)

    def evaluate_grad_at_point_list(self, points_to_evaluate):
        return self._evaluate_grad_at_point_list_core(
            self.gaussian_process.components(points_to_evaluate)
        )

    def _evaluate_at_point_list_core(self, core_components):
        z, sqrt_var, cdf_z, pdf_z, _, _, _ = core_components
        return sqrt_var * np.fmax(0.0, z * cdf_z + pdf_z)

    def _evaluate_grad_at_point_list_core(self, core_components):
        _, _, cdf_z, pdf_z, grad_mean, _, grad_sqrt_var = core_components
        return grad_sqrt_var * pdf_z[:, np.newaxis] - grad_mean * cdf_z[:, np.newaxis]

    def joint_function_gradient_eval(self, points_to_evaluate):
        points_to_evaluate = np.atleast_2d(points_to_evaluate)
        core_components = self.gaussian_process.components(points_to_evaluate)
        ei = self._evaluate_at_point_list_core(core_components)
        ei_grad = self._evaluate_grad_at_point_list_core(core_components)[0, :]
        return -ei, -ei_grad

    def next_point(self):
        return self.next_point_grad()

In [ ]:
# @title knowledge_gradient.py
# -*- coding: utf-8 -*-

MINIMUM_VARIANCE_KG = 2.2250738585072014e-308  # np.finfo(np.float64).tiny

KG_MC_ITERATIONS = 1000
KG_DOMAIN_NUM_SAMPLES = 666


class KnowledgeGradient(AcquisitionFunctionInterface):
    """
    Implementation of Knowledge Gradient (see, e.g., Frazier et al 2008)
    """

    def __init__(self, gaussian_process, opt_domain):
        super().__init__(gaussian_process, opt_domain)
        eval_domain_points = opt_domain.generate_quasi_random_points_in_domain(
            KG_DOMAIN_NUM_SAMPLES
        )
        self.num_mc_iterations = eval_domain_points.shape[0]
        xhist, yhist = self.gaussian_process.get_historical_data()
        self.best_value = np.min(yhist)
        self.eval_domain_points = eval_domain_points
        self.eval_domain_points_mean_estimate = (
            self.gaussian_process.compute_mean_of_points(self.eval_domain_points)
        )

    @property
    def differentiable(self):
        return False

    def evaluate_at_point_list(self, points_to_evaluate):
        num_points = len(points_to_evaluate)
        sigma_star = np.sqrt(
            self.gaussian_process.compute_variance_of_points(points_to_evaluate)
        )
        pred_stdev = self.gaussian_process.cross_correlation_for_samples(
            points_to_evaluate, self.eval_domain_points
        )
        curr_best = self.best_value
        std_norm_rv = np.random.normal(size=(self.num_mc_iterations, num_points))

        scaled_norm_rvs = std_norm_rv[:, np.newaxis] * (pred_stdev / sigma_star)
        post_estimate = (
            self.eval_domain_points_mean_estimate[:, np.newaxis] + scaled_norm_rvs
        )
        # min applied for each mc sample over the eval_domain_points, then mean over all MC samples
        value_kg = curr_best - np.mean(np.min(post_estimate, axis=1), axis=0)
        return value_kg

    def compute_acquisition_function(self, point_to_sample):
        return self.evaluate_at_point_list(np.atleast_2d(point_to_sample))[0]

    def compute_grad_acquisition_function(self, point_to_sample):
        raise NotImplementedError("The gradient of the KG is not implemented yet")

    def next_point(self):
        return self.next_point_grid()

In [ ]:
# @title probability_improvement.py


class ProbabilityImprovement(AcquisitionFunctionInterface):
    def __init__(self, gaussian_process, opt_domain):
        super().__init__(gaussian_process, opt_domain)

    def evaluate_at_point_list(self, points_to_evaluate):
        mu = self.gaussian_process.mean(points_to_evaluate)
        sigma2 = self.gaussian_process.variance(points_to_evaluate)
        sigma = np.sqrt(sigma2)
        xhist, yhist = self.gaussian_process.get_historical_data()
        fmin = np.min(yhist)
        z = (fmin - mu) / sigma
        cdf_z = norm.cdf(z)
        return cdf_z

In [ ]:
# @title rollout_ei.py


class RolloutEI(AcquisitionFunctionInterface):
    """
    A class for calculating the sequential rewards of MDP BayesOpt
    via parallel MC. The code is threaded, and uses the max number of processors available
    :gaussian_process: A GaussianProcess object
    :domain: The domain to optimize the inner acquisition function over
    :horizon: The time horizon
    :mc_iters: Number of Monte Carlo iterations
    :opt_mode: either 'grad' (default) or 'grid' for rollout acquisition optimization. 'grid' is faster for low-d problems.
    """

    def __init__(
        self, gaussian_process, opt_domain, horizon, mc_iters=int(1e2), opt_mode="grad"
    ):
        super().__init__(gaussian_process, opt_domain)
        self.gaussian_process = gaussian_process
        self.opt_domain = opt_domain
        self.horizon = horizon
        self.mc_iters = mc_iters
        self.numthreads = int(mp.cpu_count() / 2)
        self.opt_mode = opt_mode

    def evaluate_at_point_list(self, points_to_evaluate):
        num_points = points_to_evaluate.shape[0]
        rollout_values = np.zeros(num_points)
        for i in range(0, num_points):
            rollout_values[i] = self._evaluate_at_point_list(points_to_evaluate[[i], :])
        return rollout_values

    # Iterate through point list, parallelizing MC
    def _evaluate_at_point_list(self, point_to_evaluate):
        self.point_current = point_to_evaluate

        if self.numthreads > 1:
            serial_mc_iters = [int(self.mc_iters / self.numthreads)] * self.numthreads
            pool = Pool(processes=self.numthreads)
            rewards = pool.map(self._evaluate_point_at_list_serial, serial_mc_iters)
            pool.close()
            pool.join()
        else:
            rewards = self._evaluate_point_at_list_serial(self.mc_iters)

        return np.sum(rewards) / self.numthreads

    def _evaluate_point_at_list_serial(self, mc_iters):
        reward = 0
        for iters in range(0, mc_iters):
            r = self.draw_from_policy(self.point_current)
            reward += r
        return reward / mc_iters

    # Execute policy once using sequential draws from GPs
    def draw_from_policy(self, point_to_evaluate):
        reward = 0
        gp_temp = copy.deepcopy(self.gaussian_process)
        xi = point_to_evaluate
        h = self.horizon
        i = 0

        # Optimize acquisition function and set new sample point
        acquisition = ExpectedImprovement(gp_temp, self.opt_domain)

        while True:
            np.random.seed(int(time.time()))
            fi = gp_temp.sample_single(xi)
            ri = self.reward(gp_temp, fi)
            reward += ri
            i = i + 1
            h = h - 1
            if h <= 0:
                break

            gp_temp.chol_update(xi, fi)
            if self.opt_mode == "grad":
                xi = acquisition.next_point_grad()
            else:
                xi = acquisition.next_point_grid()
        return reward

    def reward(self, gp, fi):
        _, ytrain = gp.get_historical_data()
        ymin = np.min(ytrain)
        r = max(ymin - float(fi), 0)
        return r

    def next_point(self):
        optimizer = BayesOptAcquisitionOptimizer(
            self.gaussian_process, self.evaluate_at_point_list, self.opt_domain
        )
        return optimizer.get_sample_point()

In [ ]:
# @title rollout_ei_vr.py


class RolloutEI_VR(AcquisitionFunctionInterface):
    """
    A class for calculating the sequential rewards of MDP BayesOpt
    via parallel MC. The code is threaded, and uses the max number of processors available
    Variance reduction techniques used:
    QMC, CRN, EI, PI Control Variates
    :gaussian_process: A GaussianProcess object
    :domain: The domain to optimize the inner acquisition function over
    :horizon: The time horizon
    :mc_iters: Number of Monte Carlo iterations
    :opt_mode: either 'grad' (default) or 'grid' for rollout acquisition optimization. 'grid' is faster for low-d problems.
    :grid_size: size of rollout grid, not used if opt_mode is 'grad'
    """

    def __init__(
        self,
        gaussian_process,
        opt_domain,
        horizon,
        mc_iters=int(1e3),
        opt_mode="grad",
        grid_size=400,
        seed=0,
    ):
        super().__init__(gaussian_process, opt_domain)
        self.gaussian_process = gaussian_process
        self.opt_domain = opt_domain
        self.horizon = horizon
        self.mc_iters = mc_iters
        self.numthreads = int(mp.cpu_count() / 2)
        self.opt_mode = opt_mode
        self.grid_size = grid_size
        self.seed = seed

        # Control Variates
        self.ei = ExpectedImprovement(gaussian_process, opt_domain)
        self.pi = ProbabilityImprovement(gaussian_process, opt_domain)

    def evaluate_at_point_list(self, points_to_evaluate):
        num_points = points_to_evaluate.shape[0]
        rollout_values = np.zeros(num_points)
        for i in range(0, num_points):
            rollout_values[i] = self._evaluate_at_point_list(points_to_evaluate[[i], :])
        return rollout_values

    # Iterate through point list, parallelizing MC
    def _evaluate_at_point_list(self, point_to_evaluate):
        self.point_current = point_to_evaluate
        random_number_stream = self.low_discrepancy_points(
            self.mc_iters, seed=self.seed
        )
        pool = Pool(processes=self.numthreads)
        rewards = pool.map(self.draw_from_policy, random_number_stream)
        pool.close()
        pool.join()

        # Sum up samples to get estimate
        value_variates = np.sum(rewards) / self.mc_iters

        # Remove effects of control variates
        ei_value = self.ei.evaluate_at_point_list(point_to_evaluate)
        pi_value = self.pi.evaluate_at_point_list(point_to_evaluate)
        value_corrected = value_variates + ei_value + pi_value
        return value_corrected

    # Execute policy once using sequential draws from GPs
    def draw_from_policy(self, random_number_list):
        """
        Performs a draw from the policy, given a fixed random series of numbers
        :param random_number_list: fixed random series of numbers, of size h, assumed unit normal distributed
        :return:
        """
        assert self.horizon > 1, "Horizon must be greater than 1 to calculate MC"

        gp_temp = copy.deepcopy(self.gaussian_process)
        xi = self.point_current
        h = self.horizon
        rewards = np.zeros(h)

        # Optimize acquisition function and set new sample point
        acquisition = ExpectedImprovement(gp_temp, self.opt_domain)

        for i in range(self.horizon):
            fi = gp_temp.sample_single(xi, random_number_list[i])

            if i < self.horizon - 1:
                rewards[i] = self.reward(gp_temp, fi)
            else:
                rewards[i] = acquisition.evaluate_at_point_list(xi)

            gp_temp.chol_update(xi, fi)
            if self.opt_mode == "grad":
                xi = acquisition.next_point_grad()
            else:
                xi = acquisition.next_point_grid(self.grid_size)

        # Add in control variates (EI, PI)... in this case, an EI control variate
        # removes the first term, and a PI control variate adds 1 if there is
        # improvement in the first sample
        r = np.sum(rewards)
        r -= rewards[0]  # EI Control Variate
        if rewards[0] > 0:
            r -= 1
        return r

    def reward(self, gp, fi):
        _, ytrain = gp.get_historical_data()
        ymin = np.min(ytrain)
        r = max(ymin - float(fi), 0)
        return r

    def next_point(self):
        """
        If self horizon is 1, just return the max of regular EI
        """
        if self.horizon == 1:
            ei = ExpectedImprovement(self.gaussian_process, self.opt_domain)
            return ei.next_point_grid()
        else:
            optimizer = BayesOptAcquisitionOptimizer(
                self.gaussian_process, self.evaluate_at_point_list, self.opt_domain
            )
            return optimizer.get_sample_point()

In [ ]:
# @title rollout_ei_vr_ablation.py


class RolloutEI_VR_AB(AcquisitionFunctionInterface):
    """
    A class for calculating the sequential rewards of MDP BayesOpt
    via parallel MC. The code is threaded, and uses the max number of processors available
    **ONLY TO BE USED FOR ABLATION STUDY**
    Variance reduction techniques used:
    QMC, CRN, Control Variates (optional)
    :gaussian_process: A GaussianProcess object
    :domain: The domain to optimize the inner acquisition function over
    :horizon: The time horizon
    :mc_iters: Number of Monte Carlo iterations
    :opt_mode: either 'grad' (default) or 'grid' for rollout acquisition optimization. 'grid' is faster for low-d problems.
    """

    def __init__(
        self,
        gaussian_process,
        opt_domain,
        horizon,
        mc_iters=int(1e3),
        control_variate=True,
    ):
        super().__init__(gaussian_process, opt_domain)
        self.gaussian_process = gaussian_process
        self.opt_domain = opt_domain
        self.horizon = horizon
        self.mc_iters = mc_iters
        self.numthreads = int(mp.cpu_count() / 2)
        self.random_number_stream = self.low_discrepancy_points(
            self.mc_iters
        )  # For QMC

        # Control Variates
        self.ei = ExpectedImprovement(gaussian_process, opt_domain)
        self.pi = ProbabilityImprovement(gaussian_process, opt_domain)
        self.control_variate = control_variate

    def evaluate_at_point_list(self, points_to_evaluate):
        # Run on each point separately

        num_points = points_to_evaluate.shape[0]
        rollout_values = np.zeros(num_points)
        for i in range(0, num_points):
            rollout_values[i] = self._evaluate_at_point_list(points_to_evaluate[[i], :])
        return rollout_values

    # Iterate through point list, parallelizing MC
    def _evaluate_at_point_list(self, point_to_evaluate):
        """
        Does this with a fixed sequence of random numbers (to decrease variance via common random numbers)
        :param point_to_evaluate:
        :return:
        """
        self.point_current = point_to_evaluate
        pool = Pool(processes=self.numthreads)
        rewards = pool.map(self.draw_from_policy, self.random_number_stream)
        pool.close()
        pool.join()

        # Sum up samples to get estimate
        value_variates = np.sum(rewards) / self.mc_iters

        # Remove effects of control variates
        ei_value = self.ei.evaluate_at_point_list(point_to_evaluate)
        pi_value = self.pi.evaluate_at_point_list(point_to_evaluate)
        if self.control_variate:
            value_corrected = value_variates + ei_value
        else:
            value_corrected = value_variates
        return value_corrected

    # Execute policy once using sequential draws from GPs
    def draw_from_policy(self, random_number_list):
        """
        Performs a draw from the policy, given a fixed random series of numbers
        :param random_number_list: fixed random series of numbers, of size h, assumed unit normal distributed
        :return:
        """
        assert self.horizon > 1, "Horizon must be greater than 1 to calculate MC"

        gp_temp = copy.deepcopy(self.gaussian_process)
        xi = self.point_current
        h = self.horizon
        rewards = np.zeros(h)

        # Optimize acquisition function and set new sample point
        acquisition = ExpectedImprovement(gp_temp, self.opt_domain)

        for i in range(self.horizon):
            fi = gp_temp.sample_single(xi, random_number_list[i])

            if i < self.horizon - 1:
                rewards[i] = self.reward(gp_temp, fi)
            else:
                if self.control_variate:
                    rewards[i] = acquisition.evaluate_at_point_list(xi)
                else:
                    rewards[i] = self.reward(gp_temp, fi)

            gp_temp.chol_update(xi, fi)
            xi = acquisition.next_point_grid(num_grid_points=40)

        # Add in control variates (EI, PI)... in this case, an EI control variate
        # removes the first term, and a PI control variate adds 1 if there is
        # improvement in the first sample
        r = np.sum(rewards)
        if self.control_variate:
            r -= rewards[0]  # EI Control Variate
        # if rewards[0] > 0:
        #     r -= 1
        return r

    def reward(self, gp, fi):
        _, ytrain = gp.get_historical_data()
        ymin = np.min(ytrain)
        r = max(ymin - float(fi), 0)
        return r

    def next_point(self):
        """
        If self horizon is 1, just return the max of regular EI
        """
        if self.horizon == 1:
            ei = ExpectedImprovement(self.gaussian_process, self.opt_domain)
            return ei.next_point_grid()
        else:
            optimizer = BayesOptAcquisitionOptimizer(
                self.gaussian_process, self.evaluate_at_point_list, self.opt_domain
            )
            return optimizer.get_sample_point()

In [ ]:
# @title ucb.py
BASE_KAPPA = 1


class UpperConfidenceBound(AcquisitionFunctionInterface):
    def __init__(self, gaussian_process, opt_domain, **kwargs):
        super().__init__(gaussian_process, opt_domain)
        self.kappa = kwargs.get("base_kappa", BASE_KAPPA)

    def evaluate_at_point_list(self, points_to_evaluate):
        mean, variance = self.gaussian_process.compute_mean_and_variance_of_points(
            points_to_evaluate
        )
        return -mean - self.kappa * np.sqrt(variance)

    def joint_function_gradient_eval(self, points_to_evaluate):
        points_to_evaluate = np.atleast_2d(points_to_evaluate)
        (
            mean,
            sqrt_var,
            _,
            _,
            grad_mean,
            _,
            grad_sqrt_var,
        ) = self.gaussian_process.components(points_to_evaluate)
        ucb = -mean + self.kappa * sqrt_var
        ucb_grad = -grad_mean - self.kappa * grad_sqrt_var
        return -ucb, -ucb_grad

In [ ]:
# @title rollout_portfolio.py
MC_ITERS = 200

# The portfolio of acquisiton functions is a dict mapping acquisitions to a list of kwargs to consider. We consider
# EI and the UCB class of acquisition functions.
PORTFOLIO = {
    ExpectedImprovement: [{}],
    UpperConfidenceBound: [
        {"base_kappa": 0},
        {"base_kappa": 1},
        {"base_kappa": 2},
        {"base_kappa": 4},
        {"base_kappa": 8},
    ],
    KnowledgeGradient: [{}],
}


class RolloutPortfolio(AcquisitionFunctionInterface):
    """
    Choosing an acquisition function with Rollout. Consider: UCB for constants 1 through 5, EI, KG
    """

    def __init__(self, gaussian_process, opt_domain, horizon):
        assert horizon > 0, "Horizon must be greater than 0"
        super().__init__(gaussian_process, opt_domain)
        self.horizon = horizon
        self.numthreads = int(mp.cpu_count() / 2)

    def next_point(self):
        x_next = None
        acquisition_chosen = None
        reward_max = -np.inf
        for portfolio_element, kwargs_list in PORTFOLIO.items():
            for kwargs in kwargs_list:
                acquisition = portfolio_element(
                    self.gaussian_process, self.opt_domain, **kwargs
                )
                x_opt = acquisition.next_point()

                # Roll out EI on KG argmax first to see what happens!
                if portfolio_element is KnowledgeGradient:
                    acquisition_rollout = RolloutPortfolioElement(
                        self.gaussian_process,
                        self.opt_domain,
                        self.horizon,
                        ExpectedImprovement,
                        **kwargs
                    )
                else:
                    acquisition_rollout = RolloutPortfolioElement(
                        self.gaussian_process,
                        self.opt_domain,
                        self.horizon,
                        portfolio_element,
                        **kwargs
                    )

                reward = acquisition_rollout.evaluate_at_point_list(x_opt)
                if reward > reward_max:
                    acquisition_chosen = str(portfolio_element.__name__)
                    if kwargs:
                        acquisition_chosen += str(kwargs["base_kappa"])
                    x_next = x_opt
                    reward_max = reward
        return x_next, acquisition_chosen


class RolloutPortfolioEI(AcquisitionFunctionInterface):
    """
    Chooses an acquisition function with Rollout. Consider: UCB for constants 1 through 5, EI, KG
    """

    def __init__(self, gaussian_process, opt_domain, horizon):
        assert horizon > 0, "Horizon must be greater than 0"
        super().__init__(gaussian_process, opt_domain)
        self.horizon = horizon
        self.numthreads = int(mp.cpu_count() / 2)

    def next_point(self):
        x_next = None
        acquisition_chosen = None
        reward_max = -np.inf
        for portfolio_element, kwargs_list in PORTFOLIO.items():
            for kwargs in kwargs_list:
                acquisition = portfolio_element(
                    self.gaussian_process, self.opt_domain, **kwargs
                )
                x_opt = acquisition.next_point()

                # Roll out EI on KG argmax first to see what happens!
                acquisition_rollout = RolloutPortfolioElement(
                    self.gaussian_process,
                    self.opt_domain,
                    self.horizon,
                    ExpectedImprovement,
                    **kwargs
                )
                reward = acquisition_rollout.evaluate_at_point_list(x_opt)
                if reward > reward_max:
                    acquisition_chosen = str(portfolio_element.__name__)
                    if kwargs:
                        acquisition_chosen += str(kwargs["base_kappa"])
                    x_next = x_opt
                    reward_max = reward
        return x_next, acquisition_chosen


class RolloutPortfolioElement(AcquisitionFunctionInterface):
    """
    Rolls out a specific acquisition function
    """

    def __init__(self, gaussian_process, opt_domain, horizon, acquisition, **kwargs):
        super().__init__(gaussian_process, opt_domain)
        assert horizon > 0, "Horizon must be greater than 0"
        self.horizon = horizon
        self.mc_iters = MC_ITERS
        self.numthreads = 4
        self.acquisition = acquisition

    def evaluate_at_point_list(self, points_to_evaluate):
        num_points = points_to_evaluate.shape[0]
        rollout_values = np.zeros(num_points)
        for i in range(0, num_points):
            rollout_values[i] = self._evaluate_at_point_list(points_to_evaluate[[i], :])
        return rollout_values

    # Iterate through point list, parallelizing MC
    def _evaluate_at_point_list(self, point_to_evaluate):
        self.point_current = point_to_evaluate
        random_number_stream = self.low_discrepancy_points(self.mc_iters)
        pool = Pool(processes=self.numthreads)
        rewards = pool.map(self.draw_from_policy, random_number_stream)
        pool.close()
        pool.join()

        # Sum up samples to get estimate
        value_variates = np.sum(rewards) / self.mc_iters
        return value_variates

    def random_points(self, num):
        """
        Random points distributed w.r.t. unit gaussian, fixed according to a seed.
        """
        np.random.seed(1234)
        return list(np.random.randn(num, self.horizon))

    # Execute policy once using sequential draws from GPs
    def draw_from_policy(self, random_number_list):
        """
        Performs a draw from the policy, given a fixed random series of numbers
        :param random_number_list: fixed random series of numbers, of size h, assumed unit normal distributed
        :return:
        """
        assert self.horizon > 1, "Horizon must be greater than 1 to calculate MC"

        gp_temp = copy.deepcopy(self.gaussian_process)
        xi = self.point_current
        h = self.horizon
        rewards = np.zeros(h)

        # Optimize acquisition function and set new sample point
        acquisition_ei = ExpectedImprovement(gp_temp, self.opt_domain)
        acquisition = self.acquisition(gp_temp, self.opt_domain)

        for i in range(self.horizon):
            fi = gp_temp.sample_single(xi, random_number_list[i])

            if i < self.horizon - 1:
                rewards[i] = self.reward(gp_temp, fi)
            else:
                rewards[i] = acquisition_ei.evaluate_at_point_list(xi)
            gp_temp.chol_update(xi, fi)
            xi = acquisition.next_point()

        r = np.sum(rewards)
        return r

    def reward(self, gp, fi):
        _, ytrain = gp.get_historical_data()
        ymin = np.min(ytrain)
        r = max(ymin - float(fi), 0)
        return r

In [ ]:
# @title utils.py


# Takes min of run information
def process_run(yhist):
    for i in range(1, yhist.shape[0]):
        yhist[i] = np.min(yhist[0 : i + 1])
    return yhist


def filter_cand_points(acq_function, cand_points):
    vals = acq_function.evaluate_at_point_list(cand_points)

    # calculate particular information about vals
    median_vals = np.median(vals)
    return cand_points[vals > median_vals, :]


# creates a reshaped meshgrid matrix of size n^d x d
# where n = size_per_dimension
def unit_grid_points(size_per_dimension, d):
    a = np.linspace(0, 1, size_per_dimension)
    A = (a,) * d
    return np.dstack(np.meshgrid(*A)).ravel("F").reshape(len(A), -1).T


# Generates uniform grid of size_per_dimension x size_per_dimesion,
# reshaped into proper dimensions
def unit_grid_points2(size_per_dimension):
    x = np.linspace(0, 1, size_per_dimension)
    X1, X2 = np.meshgrid(x, x)
    X1vec = np.ndarray.flatten(X1)
    X2vec = np.ndarray.flatten(X2)
    X = np.stack((X1vec, X2vec), axis=1)
    return X, X1, X2


# A check if points are inside unit cube or not
def in_unit_cube(points):
    if points.shape[1] > 2:
        raise Exception("Only used when d < 3")

    if points.shape[1] == 2:
        return (
            all(points[:, 0] >= 0)
            and all(points[:, 0] <= 1)
            and all(points[:, 1] >= 0)
            and all(points[:, 1] <= 1)
        )
    elif points.shape[1] == 1:
        return all(points[:, 0] >= 0) and all(points[:, 0] <= 1)


# A simple normpdf function if
# we don't want to use the scipy one
def normpdf(x, mean, sd):
    var = float(sd) ** 2
    denom = (2 * np.pi * var) ** 0.5
    num = np.exp(-((x - mean) ** 2) / (2 * var))
    return num / denom


def slhd(num_pts, dim, seed=None):
    # Symmetric Latin Hypercube initial design
    # Implementation is found in PySOT:
    #
    # D. Eriksson and D. Bindel, PySOT,
    # https://github.com/dme65/pySOT
    # Fix the seed if necessary
    if seed is not None:
        np.random.seed(seed)

    # Generate a one-dimensional array based on sample number
    points = np.zeros([num_pts, dim])
    points[:, 0] = np.arange(1, num_pts + 1)

    # Get the last index of the row in the top half of the hypercube
    middleind = num_pts // 2

    # special manipulation if odd number of rows
    if num_pts % 2 == 1:
        points[middleind, :] = middleind + 1

    # Generate the top half of the hypercube matrix
    for j in range(1, dim):
        for i in range(middleind):
            if np.random.random() < 0.5:
                points[i, j] = num_pts - i
            else:
                points[i, j] = i + 1
        np.random.shuffle(points[:middleind, j])

    # Generate the bottom half of the hypercube matrix
    for i in range(middleind, num_pts):
        points[i, :] = num_pts + 1 - points[num_pts - 1 - i, :]

    # Get new random seed:
    t = 1000 * time.time()  # current time in milliseconds
    np.random.seed(int(t) % 2**32)

    return (points - 1) / (num_pts - 1)  # Map to [0, 1]^d

## lookahead/model/

In [ ]:
# @title _gaussian_process.py
MINIMUM_STD_DEV_GRAD_CHOLESKY = np.finfo(np.float64).eps
MINIMUM_VARIANCE_GRAD_EI = 150 * MINIMUM_STD_DEV_GRAD_CHOLESKY**2
MINIMUM_KRIGING_VARIANCE = 1e-100  # Just something really small


class GaussianProcessDataInterface(object):
    def __init__(self, covariance, historical_data):
        assert isinstance(covariance, CovarianceInterface)
        assert isinstance(historical_data, HistoricalData)
        assert covariance.dim == historical_data.dim
        self.covariance = copy.deepcopy(covariance)
        self.historical_data = copy.deepcopy(historical_data)
        assert self.num_sampled > 0

    @property
    def dim(self):
        return self.historical_data.dim

    @property
    def num_sampled(self):
        return self.historical_data.num_sampled

    @property
    def differentiable(self):
        return isinstance(self.covariance, DifferentiableCovariance)

    @property
    def points_sampled(self):
        return self.historical_data.points_sampled

    @property
    def points_sampled_value(self):
        return self.historical_data.points_sampled_value

    @property
    def points_sampled_noise_variance(self):
        return self.historical_data.points_sampled_noise_variance


class GaussianProcess(GaussianProcessDataInterface):
    def __init__(self, covariance, historical_data, tikhonov_param=None):
        super().__init__(
            covariance=covariance,
            historical_data=historical_data,
        )
        self.tikhonov_param = tikhonov_param

        self.K_chol = None
        self.K_inv_y = None

        self.build_precomputed_data()

    def build_precomputed_data(self):
        if self.num_sampled == 0:
            self.K_chol = np.array([])
            self.K_inv_y = np.array([])
        else:
            if self.tikhonov_param is not None:
                noise_diag_vector = np.full(self.num_sampled, self.tikhonov_param)
            else:
                noise_diag_vector = self.points_sampled_noise_variance
            kernel_matrix = self.covariance.build_kernel_matrix(
                self.points_sampled,
                noise_variance=noise_diag_vector,
            )
            self.K_chol = cho_factor(kernel_matrix, lower=True, overwrite_a=True)
            self.K_inv_y = cho_solve(self.K_chol, self.points_sampled_value)

    def _compute_core_posterior_components(self, points_to_sample, option):
        K_eval = grad_K_eval = cardinal_functions_at_points_to_sample = None
        if option in ("K_eval", "all"):
            K_eval = self.covariance.build_kernel_matrix(
                self.points_sampled, points_to_sample=points_to_sample
            )
        if option in ("grad_K_eval", "all"):
            grad_K_eval = self.covariance.build_kernel_grad_tensor(
                self.points_sampled, points_to_sample=points_to_sample
            )
        if option == "all":
            cardinal_functions_at_points_to_sample = cho_solve(self.K_chol, K_eval.T).T
        return K_eval, grad_K_eval, cardinal_functions_at_points_to_sample

    def compute_mean_of_points(self, points_to_sample):
        K_eval, _, _ = self._compute_core_posterior_components(
            points_to_sample, "K_eval"
        )
        return self._compute_mean_of_points(points_to_sample, K_eval)

    def _compute_mean_of_points(self, points_to_sample, K_eval):
        return np.dot(K_eval, self.K_inv_y)

    def compute_variance_of_points(self, points_to_sample):
        K_eval, _, _ = self._compute_core_posterior_components(
            points_to_sample, "K_eval"
        )
        return self._compute_variance_of_points(points_to_sample, K_eval)

    def _compute_variance_of_points(
        self, points_to_sample, K_eval, cardinal_functions_at_points_to_sample=None
    ):
        K_x_x_array = self.covariance.covariance(points_to_sample, points_to_sample)
        if cardinal_functions_at_points_to_sample is None:
            V = solve_triangular(
                self.K_chol[0],
                K_eval.T,
                lower=self.K_chol[1],
                overwrite_b=True,
            )
            schur_complement_component = np.sum(V**2, axis=0)
        else:
            schur_complement_component = np.sum(
                K_eval * cardinal_functions_at_points_to_sample, axis=1
            )
        return np.fmax(
            MINIMUM_KRIGING_VARIANCE, K_x_x_array - schur_complement_component
        )

    def compute_mean_and_variance_of_points(self, points_to_sample):
        K_eval, _, _ = self._compute_core_posterior_components(
            points_to_sample, "K_eval"
        )
        mean = self._compute_mean_of_points(points_to_sample, K_eval)
        var = self._compute_variance_of_points(points_to_sample, K_eval)
        return mean, var

    def compute_grad_mean_of_points(self, points_to_sample):
        _, grad_K_eval, _ = self._compute_core_posterior_components(
            points_to_sample, "grad_K_eval"
        )
        return self._compute_grad_mean_of_points(points_to_sample, grad_K_eval)

    def _compute_grad_mean_of_points(self, points_to_sample, grad_K_eval):
        return numpy.einsum("ijk, j", grad_K_eval, self.K_inv_y)

    def compute_grad_variance_of_points(self, points_to_sample):
        _, grad_K_eval, cardinal_functions = self._compute_core_posterior_components(
            points_to_sample, "all"
        )
        return self._compute_grad_variance_of_points(grad_K_eval, cardinal_functions)

    def _compute_grad_variance_of_points(
        self, grad_K_eval, cardinal_functions_at_points_to_sample
    ):
        if not self.covariance.translation_invariant:
            raise NotImplementedError("Not yet ready for general kernels.")

        return -2 * np.sum(
            grad_K_eval * cardinal_functions_at_points_to_sample[:, :, np.newaxis],
            axis=1,
        )

    def compute_mean_variance_grad_of_points(self, points_to_sample):
        (
            K_eval,
            grad_K_eval,
            cardinal_functions,
        ) = self._compute_core_posterior_components(points_to_sample, "all")

        mean = self._compute_mean_of_points(points_to_sample, K_eval)
        var = self._compute_variance_of_points(
            points_to_sample, K_eval, cardinal_functions
        )
        grad_mean = self._compute_grad_mean_of_points(points_to_sample, grad_K_eval)
        grad_var = self._compute_grad_variance_of_points(
            grad_K_eval, cardinal_functions
        )

        return mean, var, grad_mean, grad_var

    def compute_covariance_of_points(self, points_to_sample):
        K_eval_var = self.covariance.build_kernel_matrix(points_to_sample)
        if self.num_sampled == 0:
            return np.diag(np.diag(K_eval_var))

        K_eval = self.covariance.build_kernel_matrix(
            self.points_sampled, points_to_sample=points_to_sample
        )
        V = solve_triangular(
            self.K_chol[0],
            K_eval.T,
            lower=self.K_chol[1],
            overwrite_b=True,
        )

        return K_eval_var - np.dot(V.T, V)

    def cross_correlation_for_samples(self, points_being_sampled, eval_domain_points):
        all_k_x_vectors = self.covariance.build_kernel_matrix(
            self.points_sampled, points_being_sampled
        )
        all_k_xp_vectors = self.covariance.build_kernel_matrix(
            self.points_sampled, eval_domain_points
        )

        V_k = solve_triangular(
            self.K_chol[0],
            all_k_x_vectors.T,
            lower=self.K_chol[1],
            overwrite_b=True,
        )
        V_kprime = solve_triangular(
            self.K_chol[0],
            all_k_xp_vectors.T,
            lower=self.K_chol[1],
            overwrite_b=True,
        )
        K_x_xprime = self.covariance.build_kernel_matrix(
            points_being_sampled, eval_domain_points
        )
        cross_corr = K_x_xprime - np.dot(V_kprime.T, V_k)

        return cross_corr

    def draw_posterior_samples_of_points(self, num_samples, points_to_sample):
        mean = self.compute_mean_of_points(points_to_sample)
        cov = self.compute_covariance_of_points(points_to_sample)
        L = compute_cholesky_for_gp_sampling(cov)

        # z_samples is an array with shape (num_points, num_samples)
        z_samples = np.atleast_2d(np.random.normal(size=(len(mean), num_samples)))
        return mean[np.newaxis, :] + np.transpose(np.dot(L, z_samples))

    def draw_posterior_samples(self, num_samples):
        return self.draw_posterior_samples_of_points(num_samples, self.points_sampled)

In [ ]:
# @title covariance.py


class CovarianceInterface(object, metaclass=ABCMeta):
    process_variance = None

    @property
    def num_hyperparameters(self):
        raise NotImplementedError()

    @property
    def dim(self):
        raise NotImplementedError()

    @property
    def translation_invariant(self):
        return NotImplemented

    def get_hyperparameters(self):
        raise NotImplementedError()

    def set_hyperparameters(self, hyperparameters):
        raise NotImplementedError()

    hyperparameters = property(get_hyperparameters, set_hyperparameters)

    def _covariance(self, x, z):
        raise NotImplementedError

    def covariance(self, x, z):
        assert len(x.shape) == len(z.shape) == 2
        n, d = x.shape
        assert n == z.shape[0]
        assert self.dim == d == z.shape[1]

        covariance_vector = self._covariance(x, z)
        assert len(covariance_vector) == n

        return self.process_variance * covariance_vector

    def _build_kernel_matrix(self, points_sampled, points_to_sample=None):
        raise NotImplementedError

    def build_kernel_matrix(
        self, points_sampled, points_to_sample=None, noise_variance=None
    ):
        kernel_matrix = self.process_variance * self._build_kernel_matrix(
            points_sampled, points_to_sample
        )

        if noise_variance is not None:
            nx, nz = kernel_matrix.shape
            assert (
                nx == nz
            )  # Or else there should be no noise_variance term because it would be meaningless
            kernel_matrix.flat[:: nx + 1] += noise_variance
        return kernel_matrix


class DifferentiableCovariance(CovarianceInterface):
    def _grad_covariance(self, x, z):
        raise NotImplementedError

    def grad_covariance(self, x, z):
        assert len(x.shape) == len(z.shape) == 2
        n, d = x.shape
        assert n == z.shape[0]
        assert self.dim == d == z.shape[1]

        grad_covariance_vector = self._grad_covariance(x, z)
        assert grad_covariance_vector.shape == (n, d)

        return self.process_variance * grad_covariance_vector

    def _hyperparameter_grad_covariance_without_process_variance(self, x, z):
        raise NotImplementedError

    def hyperparameter_grad_covariance(self, x, z):
        assert len(x.shape) == len(z.shape) == 2
        n, d = x.shape
        assert n == z.shape[0]
        assert self.dim == d == z.shape[1]

        hyperparameter_grad_covariance = np.empty((n, self.num_hyperparameters))
        hyperparameter_grad_covariance[:, 0] = self._covariance(x, z)
        hyperparameter_grad_covariance[:, 1:] = (
            self.process_variance
            * self._hyperparameter_grad_covariance_without_process_variance(x, z)
        )

        return hyperparameter_grad_covariance

    def _build_kernel_grad_tensor(self, points_sampled, points_to_sample=None):
        raise NotImplementedError()

    def build_kernel_grad_tensor(self, points_sampled, points_to_sample=None):
        return self.process_variance * self._build_kernel_grad_tensor(
            points_sampled, points_to_sample
        )

    def _build_kernel_hparam_grad_tensor_without_process_variance(
        self, points_sampled, points_to_sample=None
    ):
        raise NotImplementedError()

    def build_kernel_hparam_grad_tensor(self, points_sampled, points_to_sample=None):
        n_cols, _ = points_sampled.shape
        n_rows = n_cols if points_to_sample is None else len(points_to_sample)

        kg_tensor = np.empty((n_rows, n_cols, self.num_hyperparameters))
        kg_tensor[:, :, 0] = self._build_kernel_matrix(points_sampled, points_to_sample)
        kg_tensor[:, :, 1:] = (
            self.process_variance
            * self._build_kernel_hparam_grad_tensor_without_process_variance(
                points_sampled, points_to_sample
            )
        )

        return kg_tensor


class RadialCovariance(CovarianceInterface):
    def __init__(self, hyperparameters):
        self._hyperparameters = None
        self._length_scales = None
        self._length_scales_squared = None
        self._length_scales_cubed = None
        self.process_variance = None
        self.set_hyperparameters(hyperparameters)

    def __str__(self):
        return f"{self.__class__.__name__}_{self.dim}({self.hyperparameters})"

    def check_hyperparameters_are_valid(self, new_hyperparameters):
        new_hyperparameters = np.asarray(new_hyperparameters, dtype=float)
        assert (
            len(new_hyperparameters.shape) == 1
        ), f"Hyperparameters should be in 1D array, not {new_hyperparameters}"
        assert np.all(
            new_hyperparameters > 0
        ), f"For {self.__class__.__name__}, all hyperparameters must be positive"
        return new_hyperparameters

    @property
    def num_hyperparameters(self):
        return self._hyperparameters.size

    @property
    def dim(self):
        return len(self._length_scales)

    @property
    def translation_invariant(self):
        return True

    def get_hyperparameters(self):
        return np.copy(self._hyperparameters)

    def set_hyperparameters(self, hyperparameters):
        self._hyperparameters = self.check_hyperparameters_are_valid(hyperparameters)
        self.process_variance = self._hyperparameters[0]
        self._length_scales = np.copy(self._hyperparameters[1:])
        self._length_scales_squared = self._length_scales**2
        self._length_scales_cubed = self._length_scales**3

    hyperparameters = property(get_hyperparameters, set_hyperparameters)

    def eval_radial_kernel(self, distance_matrix_squared):
        raise NotImplementedError()

    def _distance_between_points(self, data, eval_points):
        data_shape = data.shape
        eval_shape = eval_points.shape
        if len(data_shape) != 2 or len(eval_shape) != 2:
            raise ValueError(
                f"Points must be a 2D array: data.shape = {data_shape}, eval_points.shape = {eval_shape}"
            )
        elif data_shape != eval_shape:
            raise ValueError(f"Data size {data_shape}, Eval size {eval_shape}")
        elif data_shape[1] != self.dim:
            raise ValueError(
                f"Points dimension {data_shape[1]}, Covariance dimension {self.dim}"
            )

        diff_vecs = eval_points - data
        r = np.sqrt(np.sum(np.power(diff_vecs / self._length_scales, 2), axis=1))
        return r, diff_vecs

    def _build_distance_matrix_squared(
        self,
        data,
        eval_points=None,
        build_diff_matrices=False,
    ):
        if eval_points is None:
            return self._build_symmetric_distance_matrix_squared(
                data, build_diff_matrices
            )
        else:
            return self._build_nonsymmetric_distance_matrix_squared(
                data, eval_points, build_diff_matrices
            )

    def _build_symmetric_distance_matrix_squared(self, data, build_diff_matrices):
        diff_mats = None
        dist_mat_sq = squareform(
            pdist(data / self._length_scales[np.newaxis, :], "sqeuclidean")
        )
        if build_diff_matrices:
            diff_mats = data[:, np.newaxis, :] - data[np.newaxis, :, :]
        return dist_mat_sq, diff_mats

    def _build_nonsymmetric_distance_matrix_squared(
        self, data, eval_points, build_diff_matrices
    ):
        diff_mats = None
        x = eval_points / self._length_scales[np.newaxis, :]
        z = data / self._length_scales[np.newaxis, :]
        dist_mat_sq = cdist(x, z) ** 2
        if build_diff_matrices:
            diff_mats = eval_points[:, np.newaxis, :] - data[np.newaxis, :, :]
        return dist_mat_sq, diff_mats

    def _build_kernel_matrix(self, points_sampled, points_to_sample=None):
        return self.eval_radial_kernel(
            self._build_distance_matrix_squared(points_sampled, points_to_sample)[0]
        )


class DifferentiableRadialCovariance(DifferentiableCovariance, RadialCovariance):
    def eval_radial_kernel_grad(self, distance_matrix_squared, difference_matrix):
        raise NotImplementedError()

    def eval_radial_kernel_hparam_grad(
        self, distance_matrix_squared, difference_matrix
    ):
        raise NotImplementedError()

    def _build_kernel_grad_tensor(self, points_sampled, points_to_sample=None):
        dm_sq, diff_mats = self._build_distance_matrix_squared(
            points_sampled, points_to_sample, True
        )
        return self.eval_radial_kernel_grad(dm_sq, diff_mats)

    def _build_kernel_hparam_grad_tensor_without_process_variance(
        self, points_sampled, points_to_sample=None
    ):
        dm_sq, diff_mats = self._build_distance_matrix_squared(
            points_sampled, points_to_sample, True
        )
        return self.eval_radial_kernel_hparam_grad(dm_sq, diff_mats)


def _scale_difference_matrix(scale, difference_matrix):
    return scale[:, :, np.newaxis] * difference_matrix


class C4RadialMatern(DifferentiableRadialCovariance):
    def __init__(self, hyperparameters):
        super().__init__(hyperparameters)

    def eval_radial_kernel(self, distance_matrix_squared):
        r = np.sqrt(distance_matrix_squared)
        return (1 + r + 1.0 / 3.0 * distance_matrix_squared) * np.exp(-r)

    def eval_radial_kernel_grad(self, distance_matrix_squared, difference_matrix):
        r = np.sqrt(distance_matrix_squared)
        return (
            _scale_difference_matrix(
                -(1.0 / 3.0) * (1 + r) * np.exp(-r), difference_matrix
            )
            / self._length_scales_squared
        )

    def eval_radial_kernel_hparam_grad(
        self, distance_matrix_squared, difference_matrix
    ):
        r = np.sqrt(distance_matrix_squared)
        return (
            _scale_difference_matrix(
                (1.0 / 3.0) * (1 + r) * np.exp(-r), (difference_matrix**2)
            )
            / self._length_scales_cubed
        )

    def _covariance(self, x, z):
        r, _ = self._distance_between_points(z, x)
        return (1 + r + 1.0 / 3.0 * r**2) * np.exp(-r)

    def _grad_covariance(self, x, z):
        r, dm = self._distance_between_points(z, x)
        r_2d = r[:, np.newaxis]
        return (
            -(1.0 / 3.0) * (1 + r_2d) * np.exp(-r_2d) * dm / self._length_scales_squared
        )

    def _hyperparameter_grad_covariance_without_process_variance(self, x, z):
        r, dm = self._distance_between_points(z, x)
        r_2d = r[:, np.newaxis]
        return (
            (1.0 / 3.0)
            * (1 + r_2d)
            * np.exp(-r_2d)
            * (dm**2)
            / self._length_scales_cubed
        )

In [ ]:
# @title domain.py
from collections import namedtuple
import copy
import numpy
import qmcpy


def generate_sobol_points(num_points, domain_bounds):
    distribution = qmcpy.Sobol(dimension=len(domain_bounds))
    pts01 = distribution.gen_samples(n=2 ** numpy.ceil(numpy.log2(num_points)))[
        :num_points
    ]
    pts_scale = numpy.array([domain.length for domain in domain_bounds])
    pts_min = numpy.array([domain.min for domain in domain_bounds])
    return pts_min + pts_scale * pts01


class ClosedInterval(namedtuple("BaseInterval", ("min", "max"))):
    __slots__ = ()

    def __repr__(self):
        return f"{self.__class__.__name__}({self.min},{self.max})"

    def __contains__(self, key):
        return self.min <= key <= self.max

    def __nonzero__(self):
        return self.min <= self.max

    def __bool__(self):
        return self.__nonzero__()

    def __eq__(self, other):
        return (
            self.__class__ == other.__class__
            and self.min == other.min
            and self.max == other.max
        )

    def __ne__(self, other):
        return not self.__eq__(other)

    def __hash__(self):
        return hash((self.min, self.max))

    @property
    def length(self):
        return self.max - self.min

    def is_inside(self, value):
        return self.__contains__(value)

    def is_valid(self):
        return self.max >= self.min


class TensorProductDomain(object):
    def __init__(self, domain_bounds):
        """Construct a TensorProductDomain with the specified bounds defined using
        a list of ClosedInterval objects.
        """
        self.domain_bounds = copy.deepcopy(domain_bounds)

        for interval in self.domain_bounds:
            if not interval.is_valid():
                raise ValueError("Tensor product region is EMPTY.")

    def __repr__(self):
        return f"TensorProductDomain({self.domain_bounds})"

    @property
    def dim(self):
        return len(self.domain_bounds)

    def check_point_inside(self, point):
        return all(
            [
                interval.is_inside(point[i])
                for i, interval in enumerate(self.domain_bounds)
            ]
        )

    def check_point_acceptable(self, point):
        assert len(point) == self.dim
        return self.check_point_inside(point)

    def get_bounding_box(self):
        return copy.copy(self.domain_bounds)

    def generate_quasi_random_points_in_domain(self, num_points, log_sample=False):
        r"""Generate quasi-random points in the domain.
        :param num_points: max number of points to generate
        :type num_points: int >= 0
        :param log_sample: sample logarithmically spaced points
        :type log_sample: bool
        :return: uniform random sampling of points from the domain
        :rtype: array of float64 with shape (num_points, dim)
        """
        domain_bounds = self.domain_bounds
        if log_sample:
            domain_bounds = [
                ClosedInterval(numpy.log(a), numpy.log(b)) for (a, b) in domain_bounds
            ]

        points = generate_sobol_points(num_points, domain_bounds)
        return numpy.exp(points) if log_sample else points

In [ ]:
# @title parametrization.py

DEFAULT_TIKHONOV_PARAMETER = 1.0e-10


class GaussianProcessLogMarginalLikelihood(GaussianProcessDataInterface):
    def __init__(
        self,
        covariance,
        historical_data,
        log_domain=False,
        tikhonov_parameter=DEFAULT_TIKHONOV_PARAMETER,
    ):
        super().__init__(covariance, historical_data)
        self.log_domain = log_domain
        self.tikhonov_parameter = tikhonov_parameter
        self.gp = GaussianProcess(
            self.covariance,
            self.historical_data,
            tikhonov_param=self.tikhonov_parameter,
        )

    @property
    def num_hyperparameters(self):
        return self.covariance.num_hyperparameters

    @property
    def tikhonov_param(self):
        return self.gp.tikhonov_param

    def get_hyperparameters(self):
        hyperparameters = self.covariance.hyperparameters
        return np.log(hyperparameters) if self.log_domain else hyperparameters

    def set_hyperparameters(self, hyperparameters):
        hp_linear_domain = (
            np.exp(hyperparameters) if self.log_domain else hyperparameters
        )
        # Don't pass the noise term in covariance.hyperparameters since we pass it as a separate param
        self.covariance.hyperparameters = hp_linear_domain[: self.dim + 1]
        self.gp = GaussianProcess(
            self.covariance, self.historical_data, self.tikhonov_parameter
        )

    hyperparameters = property(get_hyperparameters, set_hyperparameters)
    current_point = hyperparameters

    def compute_objective_function(self):
        y_Pb = self.gp.points_sampled_value
        Kinvy_Pb = self.gp.K_inv_y
        L = self.gp.K_chol[0]
        log_likelihood = np.dot(y_Pb, Kinvy_Pb) + 2 * np.sum(numpy.log(L.diagonal()))
        return -log_likelihood

    def compute_grad_objective_function(self):
        grad_hyperparameter_cov_tensor = (
            self.covariance.build_kernel_hparam_grad_tensor(self.points_sampled)
        )
        grad_log_marginal = np.empty(self.num_hyperparameters)
        Kinvy_Pb = self.gp.K_inv_y
        K_chol = self.gp.K_chol
        for k in range(self.num_hyperparameters):
            dK = grad_hyperparameter_cov_tensor[:, :, k]
            dKKinvy_Pb = np.dot(dK, Kinvy_Pb)
            grad_log_marginal[k] = -np.dot(Kinvy_Pb, dKKinvy_Pb)
            grad_log_marginal[k] += np.trace(
                scipy.linalg.cho_solve(K_chol, dK, overwrite_b=True)
            )

        log_scaling = np.exp(self.hyperparameters) if self.log_domain else 1.0
        return -grad_log_marginal * log_scaling

In [ ]:
# @title scalar_optimization.py

MAXIMUM_REPRESENTABLE_FINITE_FLOAT = np.finfo(np.float64).max


class MultistartMaximizer(object):
    def __init__(self, optimizer, num_multistarts=1, log_sample=False):
        assert not isinstance(optimizer, MultistartMaximizer)
        self.optimizer = optimizer
        assert num_multistarts >= 1
        self.num_multistarts = num_multistarts
        self.log_sample = log_sample

    def optimize(self, **kwargs):
        all_starts = self.optimizer.domain.generate_quasi_random_points_in_domain(
            self.num_multistarts, self.log_sample
        )

        best_point = None
        best_function_value = -np.inf
        for point in all_starts:
            try:
                self.optimizer.objective_function.current_point = point
                self.optimizer.optimize(**kwargs)
            except np.linalg.LinAlgError:
                function_value = float("nan")
                success = False
            else:
                # The negation here is required because the optimizer decorator has already negated the value
                function_value = -self.optimizer.optimization_results.fun
                success = self.optimizer.optimization_results.success

            end_point = self.optimizer.objective_function.current_point
            if not self.optimizer.domain.check_point_acceptable(end_point):
                function_value = float("nan")
                success = False

            if best_point is None or (success and function_value > best_function_value):
                if best_point is None and not success:
                    best_point = point
                    continue
                best_point = end_point
                best_function_value = (
                    function_value
                    if not np.isnan(function_value)
                    else best_function_value
                )

        return best_point


class LBFGSBOptimizer(object):
    def __init__(self, domain, optimizable, approx_grad=False):
        self.domain = domain
        self.objective_function = optimizable
        self.optimization_results = None
        self.approx_grad = approx_grad
        assert self.objective_function.differentiable or self.approx_grad

    @property
    def dim(self):
        return self.domain.dim

    def _domain_as_array(self):
        return np.array(
            [
                (interval.min, interval.max)
                for interval in self.domain.get_bounding_box()
            ]
        )

    def joint_function_gradient_eval(self, **kwargs):
        def decorated(point):
            if np.any(np.isnan(point)):
                return np.inf, np.zeros((self.dim,))

            self.objective_function.current_point = point
            value = -self.objective_function.compute_objective_function(**kwargs)
            gradient = -self.objective_function.compute_grad_objective_function(
                **kwargs
            )
            assert np.isfinite(value) and gradient.shape == (self.dim,)
            return value, gradient

        return decorated

    def _scipy_decorator(self, func, **kwargs):
        def decorated(point):
            self.objective_function.current_point = point
            return -func(**kwargs)

        return decorated

    def optimize(self, **kwargs):
        self.optimization_results = self._optimize_core(**kwargs)
        point = self.optimization_results.x
        self.objective_function.current_point = point

    def _optimize_core(self, **kwargs):
        options = {
            "eps": 1.0e-8,
            "gtol": 1.0e-4,
            "maxcor": 10,
            "maxfun": 15000,
            "ftol": 1e-4,
        }
        if self.approx_grad:
            return scipy.optimize.minimize(
                fun=self._scipy_decorator(
                    self.objective_function.compute_objective_function, **kwargs
                ),
                x0=self.objective_function.current_point.flatten(),
                method="L-BFGS-B",
                bounds=self._domain_as_array(),
                options=options,
            )
        else:
            options.pop("eps")
            return scipy.optimize.minimize(
                fun=self.joint_function_gradient_eval(**kwargs),
                x0=self.objective_function.current_point.flatten(),
                method="L-BFGS-B",
                jac=True,
                bounds=self._domain_as_array(),
                options=options,
            )

In [ ]:
# @title gaussian_process.py
# Separate wrapper for Gaussian process using
# MOE's GaussianProcess class as the base class (for internal use).

# import numpy as np

# from lookahead.model.domain import ClosedInterval
# from lookahead.model.historical_data import HistoricalData
# from lookahead.model.domain import TensorProductDomain
# from lookahead.model.covariance import C4RadialMatern as Matern
# from lookahead.model._gaussian_process import GaussianProcess
# from lookahead.model.parametrization import GaussianProcessLogMarginalLikelihood as GPlml
# from lookahead.model.scalar_optimization import LBFGSBOptimizer as lbfgs_opt
# from lookahead.model.scalar_optimization import MultistartMaximizer as ms_opt

GPlml = GaussianProcessLogMarginalLikelihood
lbfgs_opt = LBFGSBOptimizer
ms_opt = MultistartMaximizer


# Updates Cholesky by adding a
# set of rows/columns, with
# data in c12, c22
def cho_addcol(L1, c12, c22):
    # Assume L1 is in scipy-compliant format
    L1 = L1[0]
    n = L1.shape[0]
    arg_bool = L1[1]
    k = c12.shape[1]
    L = np.zeros((n + k, n + k))
    L12 = solve_triangular(L1, c12, lower=True)
    if k == 1:
        L22 = np.sqrt(c22 - sp.linalg.norm(L12) ** 2)
    else:
        L22 = cholesky(c22 - np.dot(L12.T, L12))
    L[0:n, 0:n] = L1
    if k > 1:
        L[n:, 0:n] = L12.T
        L[n:, n:] = L22.T
    else:
        L[n, 0:n] = L12[:, 0]
        L[n, n] = L22
    return [L, True]


class GaussianProcessSimple(GaussianProcess):
    """
    A less complicated implementation of a Gaussian process with a Matern 5/2 kernel, to be used for BO.
    """

    def __init__(self, xtrain, ytrain):
        self.ymean = np.mean(ytrain)
        self.tikh = 1e-6
        ytrain = np.copy(ytrain) - self.ymean
        self.best_value = np.min(ytrain)  # update best value
        _, self.d = xtrain.shape
        hd = HistoricalData(dim=self.d)
        hd.append_historical_data(
            points_sampled=xtrain,
            points_sampled_value=ytrain,
            points_sampled_noise_variance=self.tikh * np.ones(ytrain.shape),
        )
        params_init = np.ones((self.d + 1))
        cov = C4RadialMatern(params_init)

        super().__init__(
            covariance=cov,
            historical_data=hd,
            tikhonov_param=self.tikh,
        )

    def set_hypers(self, params):
        self.covariance.set_hyperparameters(params)
        super().build_precomputed_data()

    # Train hyperparameters
    def train(self):
        log_marginal_likelihood = GPlml(
            covariance=self.covariance,
            historical_data=self.historical_data,
            log_domain=True,
        )
        hp_domain = TensorProductDomain(
            [ClosedInterval(-7, 3)] + [ClosedInterval(-3, 4)] * self.d
        )
        solver = ms_opt(
            lbfgs_opt(hp_domain, log_marginal_likelihood), num_multistarts=4
        )
        self.covariance.set_hyperparameters(np.exp(solver.optimize()))
        super().build_precomputed_data()

    def mean(self, xx):
        return self.ymean + self.compute_mean_of_points(xx)

    # Only difference between predict and mean is that
    # predict returns an (n,1) array instead of (n) array
    def predict(self, xx):
        temp = self.ymean + self.compute_mean_of_points(xx)
        temp = temp[:, np.newaxis]
        return temp

    def variance(self, xx):
        return self.compute_variance_of_points(xx)

    def add_points(self, XX, YY, retrain=False):
        YY = YY - self.ymean
        self.best_value = min(np.min(YY), self.best_value)  # update best value
        num_points = XX.shape[0]

        for i in range(num_points):
            self.historical_data.append_historical_data(
                points_sampled=XX[[i], :],
                points_sampled_value=YY[[i]],
                points_sampled_noise_variance=np.array([1e-6]),
            )
        if retrain:
            self.train()
        else:
            super().build_precomputed_data()

    # Sample from posterior
    def sample(self, num_samples, points_to_sample):
        return (
            super().draw_posterior_samples_of_points(num_samples, points_to_sample)
            + self.ymean
        )

    # Update Cholesky factorization with a row/column
    def chol_update(self, XX, YY):
        K12 = self.covariance.build_kernel_matrix(XX, self.points_sampled)
        K22 = self.covariance.build_kernel_matrix(XX, XX) + self.tikh * np.identity(
            XX.shape[0]
        )
        YY = YY - self.ymean
        self.best_value = min(np.min(YY), self.best_value)  # update best value
        self.historical_data.append_historical_data(
            points_sampled=XX,
            points_sampled_value=YY,
            points_sampled_noise_variance=1e-6 * np.ones(YY.shape),
        )
        self.K_chol = cho_addcol(self.K_chol, K12, K22)
        self.K_inv_y = cho_solve(self.K_chol, self.points_sampled_value)

    # Return historical data in the form of two arrays xtrain, ytrain
    def get_historical_data(self):
        return (
            self.historical_data.points_sampled,
            self.ymean + self.historical_data.points_sampled_value,
        )

    # Sample single point, should be faster than self.sample()
    def sample_single(self, point_to_sample, random_gaussian_normal_sample=None):
        mu, var = self.mean_variance_single(point_to_sample)
        if random_gaussian_normal_sample is None:
            return np.random.normal(mu, np.sqrt(var), 1)
        else:
            mu, var = self.mean_variance_single(point_to_sample)
            return np.atleast_1d(mu + np.sqrt(var) * random_gaussian_normal_sample)

    # Compute means and variance at single point, should be vaster than self.variance()
    # returns in order mu, sigma (mean and variance respectively)
    def mean_variance_single(self, point_to_sample):
        Kx = self.covariance.build_kernel_matrix(
            self.historical_data.points_sampled, points_to_sample=point_to_sample
        )
        Kxx = self.covariance.build_kernel_matrix(
            point_to_sample, points_to_sample=point_to_sample
        )
        mu = np.dot(Kx, self.K_inv_y) + self.ymean
        Q = solve_triangular(
            self.K_chol[0],
            Kx.T,
            lower=self.K_chol[1],
            overwrite_b=True,
        )
        return mu.item(), Kxx.item() - np.sum(Q**2, axis=0).item()

    def components(self, points_to_evaluate):
        mean, var, grad_mean, grad_var = self.compute_mean_variance_grad_of_points(
            points_to_evaluate
        )
        sqrt_var = np.sqrt(var)
        grad_sqrt_var = 0.5 * grad_var / sqrt_var[:, np.newaxis]
        z = (self.best_value - mean) / sqrt_var
        cdf_z = sp.stats.norm.cdf(z)
        pdf_z = sp.stats.norm.pdf(z)
        return z, sqrt_var, cdf_z, pdf_z, grad_mean, grad_var, grad_sqrt_var

In [ ]:
# @title historical_data.py


class SamplePoint(object):
    def __init__(self, point, value=None, noise_variance=0.0):
        if not (noise_variance >= 0.0 and np.isfinite(noise_variance)):
            raise ValueError(
                f"noise_variance = {noise_variance} must be non-negative and finite!"
            )
        if value is None or np.isinf(value):
            raise ValueError(f"value = {value} must be finite (nan allowed)!")
        if any(~np.isfinite(point)):
            raise ValueError(f"point = {point} must be finite!")

        self.point = np.copy(point)
        self.value = value
        self.noise_variance = noise_variance

    @classmethod
    def from_dict(cls, d):
        return SamplePoint(d["point"], d["value"], d["value_var"])

    @property
    def as_tuple(self):
        return self.point, self.value, self.noise_variance

    def __getitem__(self, item):
        if item == "point":
            return self.point
        elif item == "value":
            return self.value
        elif item == "value_var":
            return self.noise_variance
        else:
            raise ValueError(
                f"key {item} not recognized, must be one of ('point', 'value', 'value_var')"
            )

    def __repr__(self):
        """Pretty print this object as a dict."""
        return pprint.pformat(dict(zip(("point", "value", "value_var"), self.as_tuple)))

    def json_payload(self):
        return {
            "point": list(self.point),  # json needs a list
            "value": self.value,
            "value_var": self.noise_variance,
        }


class HistoricalData(object):
    @staticmethod
    def convert_list_of_sample_points_to_arrays(sample_points):
        if len(sample_points) == 0:
            return np.array([[]]), np.array([]), np.array([])
        return tuple(np.array(v) for v in zip(*[p.as_tuple for p in sample_points]))

    def __init__(self, dim, sample_points=None):
        if sample_points is None:
            sample_points = []

        self.dim = dim
        self.points_sampled = np.empty((0, self.dim))
        self.points_sampled_value = np.empty(0)
        self.points_sampled_noise_variance = np.empty(0)

        self.append_sample_points(sample_points)

    def __str__(self, pretty_print=True):
        """String representation of this HistoricalData object."""
        if pretty_print:
            return pprint.pformat(self.to_list_of_sample_points())
        return "\n".join(
            [
                repr(self.points_sampled),
                repr(self.points_sampled_value),
                repr(self.points_sampled_noise_variance),
            ]
        )

    def append_lies(self, points_being_sampled, lie_value, lie_value_var):
        self.append_sample_points(
            [
                SamplePoint(point, lie_value, lie_value_var)
                for point in points_being_sampled
            ]
        )

    def append_historical_data(
        self, points_sampled, points_sampled_value, points_sampled_noise_variance
    ):
        """Append lists of points_sampled, their values, and their noise variances to the data members of this class."""
        if points_sampled.size == 0:
            return

        assert len(points_sampled.shape) == 2
        assert (
            len(points_sampled_value.shape)
            == len(points_sampled_noise_variance.shape)
            == 1
        )
        assert (
            len(points_sampled)
            == len(points_sampled_value)
            == len(points_sampled_noise_variance)
        )
        assert points_sampled.shape[1] == self.dim

        self.points_sampled = np.append(self.points_sampled, points_sampled, axis=0)
        self.points_sampled_value = np.append(
            self.points_sampled_value, points_sampled_value
        )
        self.points_sampled_noise_variance = np.append(
            self.points_sampled_noise_variance, points_sampled_noise_variance
        )

    def append_sample_points(self, sample_points):
        self.append_historical_data(
            *self.convert_list_of_sample_points_to_arrays(sample_points)
        )

    def to_list_of_sample_points(self):
        return [
            SamplePoint(*a)
            for a in zip(
                self.points_sampled,
                self.points_sampled_value,
                self.points_sampled_noise_variance,
            )
        ]

    @property
    def num_sampled(self):
        """Return the number of sampled points."""
        return self.points_sampled.shape[0]

In [ ]:
# @title utils.py
# -*- coding: utf-8 -*-


def compute_cholesky_for_gp_sampling(covariance_matrix):
    try:
        chol_cov = -scipy.linalg.cholesky(
            covariance_matrix, lower=True, overwrite_a=True, check_finite=False
        )
    except scipy.linalg.LinAlgError:
        U, E, _ = scipy.linalg.svd(
            covariance_matrix, overwrite_a=True, check_finite=False
        )
        chol_cov = U * np.sqrt(E)[np.newaxis, :]
        chol_cov = -scipy.linalg.qr(
            chol_cov.T, mode="r", overwrite_a=True, check_finite=False
        )[0].T
    return chol_cov

## lookahead/runners/

In [ ]:
# @title bayesian_optimization.py
# import numpy as np
# from lookahead.model.gaussian_process import GaussianProcessSimple as GaussianProcess
# import os

GaussianProcess = GaussianProcessSimple


class BayesianOptimization(object):
    def __init__(self, search_space):
        self.opt_name = "abstract"
        self.gaussian_process = None
        self.search_space = search_space

    def run(self, f, seed, budget_minus_initialization, initialization_duration=5):
        # Warm start with 5 points, with fixed random seed
        np.random.seed(seed)
        d = len(self.search_space.domain_bounds)
        xhist = np.random.rand(initialization_duration, d)
        yhist = f(xhist)
        self.gaussian_process = GaussianProcess(xhist, yhist)
        self.gaussian_process.train()

        while budget_minus_initialization > 0:
            # Get next sample point
            xsample = self.get_next_point()
            ysample = f(xsample)
            xhist = np.vstack((xhist, xsample))
            yhist = np.append(yhist, ysample)
            self.gaussian_process = GaussianProcess(xhist, yhist)
            self.gaussian_process.train()
            budget_minus_initialization -= 1

        xhist, yhist = self.gaussian_process.get_historical_data()
        self.save_bo_run(yhist, str(f.__name__), seed)

    def get_next_point(self):
        # To be implemented by each acquisition function
        pass

    def save_bo_run(self, yhist, objective_name, seed):
        seed = str(seed)
        """
        Saves run to the folder ~/Look-Ahead/results/optimizer_name/objective_name/seed.csv
        """
        base_path = os.path.expanduser("~") + "/Look-Ahead/results/"
        # Make paths if necessary
        if not os.path.exists(base_path):
            os.makedirs(base_path)
        path = base_path + self.opt_name + "/"
        if not os.path.exists(path):
            os.makedirs(path)
        path = path + objective_name + "/"
        if not os.path.exists(path):
            os.makedirs(path)
        run_name = path + str(seed) + ".csv"

        # Save data as csv to path
        np.savetxt(run_name, yhist, delimiter=",")

In [ ]:
# @title portfolio_runners.py
# import numpy as np
# from lookahead.runners.bayesian_optimization import BayesianOptimization
# from lookahead.acquisitions.rollout_portfolio import RolloutPortfolio, RolloutPortfolioEI
# from lookahead.model.gaussian_process import GaussianProcessSimple as GaussianProcess
# import os
# import csv

GaussianProcess = GaussianProcessSimple


class PortfolioRunner(BayesianOptimization):
    """
    A  runner class for Portfolio, Diagnostic because it outputs auxillary data.
    """

    def __init__(self, search_space, horizon):
        super().__init__(search_space)
        self.horizon = horizon
        self.opt_name = "portfolio" + str(horizon)

    def run(self, f, seed, budget_minus_initialization, initialization_duration=5):
        acquisition_chosen_all = []

        # Warm start with 5 points, with fixed random seed
        np.random.seed(seed)
        d = len(self.search_space.domain_bounds)
        xhist = np.random.rand(5, d)
        yhist = f(xhist)
        self.gaussian_process = GaussianProcess(xhist, yhist)
        self.gaussian_process.train()

        while budget_minus_initialization > 0:
            # Get next sample point
            xsample, acquisition_chosen = self.get_next_point()
            acquisition_chosen_all.append(acquisition_chosen)
            ysample = f(xsample)
            xhist = np.vstack((xhist, xsample))
            yhist = np.append(yhist, ysample)
            self.gaussian_process = GaussianProcess(xhist, yhist)
            self.gaussian_process.train()
            budget_minus_initialization -= 1

        xhist, yhist = self.gaussian_process.get_historical_data()

        # Save BOTH acquisitions chosen as well as run information
        self.save_auxillary_data(acquisition_chosen_all, str(f.__name__), seed)
        self.save_bo_run(yhist, str(f.__name__), seed)

    def get_next_point(self):
        # To be implemented by each acquisition function
        pr = RolloutPortfolio(self.gaussian_process, self.search_space, self.horizon)
        return pr.next_point()

    def save_auxillary_data(self, acquisition_chosen_all, objective_name, seed):
        seed = str(seed)
        """
        Saves run to the folder ~/Look-Ahead/results/optimizer_name/objective_name/seed.csv
        """
        base_path = os.path.expanduser("~") + "/Look-Ahead/results/"
        # Make paths if necessary
        if not os.path.exists(base_path):
            os.makedirs(base_path)
        path = base_path + self.opt_name + "/"
        if not os.path.exists(path):
            os.makedirs(path)
        path = path + objective_name + "/"
        if not os.path.exists(path):
            os.makedirs(path)
        run_name = path + "portfolio_aux" + str(seed) + ".csv"

        # Save data as csv to path
        with open(run_name, "w") as file:
            writer = csv.writer(file, delimiter="\n")
            writer.writerow(acquisition_chosen_all)


class PortfolioEIRunner(BayesianOptimization):
    """
    A  runner class for Portfolio, Diagnostic because it outputs auxillary data.
    """

    def __init__(self, search_space, horizon):
        super().__init__(search_space)
        self.horizon = horizon
        self.opt_name = "portfolio_ei" + str(horizon)

    def run(self, f, seed, budget_minus_initialization, initialization_duration=5):
        acquisition_chosen_all = []

        # Warm start with 5 points, with fixed random seed
        np.random.seed(seed)
        d = len(self.search_space.domain_bounds)
        xhist = np.random.rand(5, d)
        yhist = f(xhist)
        self.gaussian_process = GaussianProcess(xhist, yhist)
        self.gaussian_process.train()

        while budget_minus_initialization > 0:
            # Get next sample point
            xsample, acquisition_chosen = self.get_next_point()
            acquisition_chosen_all.append(acquisition_chosen)
            ysample = f(xsample)
            xhist = np.vstack((xhist, xsample))
            yhist = np.append(yhist, ysample)
            self.gaussian_process = GaussianProcess(xhist, yhist)
            self.gaussian_process.train()
            budget_minus_initialization -= 1

        # Save BOTH acquisitions chosen as well as run information
        xhist, yhist = self.gaussian_process.get_historical_data()
        self.save_auxillary_data(acquisition_chosen_all, str(f.__name__), seed)
        self.save_bo_run(yhist, str(f.__name__), seed)

    def get_next_point(self):
        # To be implemented by each acquisition function
        pr = RolloutPortfolioEI(self.gaussian_process, self.search_space, self.horizon)
        return pr.next_point()

    def save_auxillary_data(self, acquisition_chosen_all, objective_name, seed):
        seed = str(seed)
        """
        Saves run to the folder ~/Look-Ahead/results/optimizer_name/objective_name/seed.csv
        """
        base_path = os.path.expanduser("~") + "/Look-Ahead/results/"
        # Make paths if necessary
        if not os.path.exists(base_path):
            os.makedirs(base_path)
        path = base_path + self.opt_name + "/"
        if not os.path.exists(path):
            os.makedirs(path)
        path = path + objective_name + "/"
        if not os.path.exists(path):
            os.makedirs(path)
        run_name = path + "portfolio_aux" + str(seed) + ".csv"

        # Save data as csv to path
        with open(run_name, "w") as file:
            writer = csv.writer(file, delimiter="\n")
            writer.writerow(acquisition_chosen_all)

In [ ]:
# @title runners.py
# import numpy as np
# from lookahead.runners.bayesian_optimization import BayesianOptimization
# from lookahead.acquisitions.expected_improvement import ExpectedImprovement
# from lookahead.acquisitions.knowledge_gradient import KnowledgeGradient
# from lookahead.acquisitions.ucb import UpperConfidenceBound
# from lookahead.acquisitions.rollout_ei_vr import RolloutEI_VR


class RandomRunner(BayesianOptimization):
    def __init__(self, search_space):
        super().__init__(search_space)
        self.opt_name = "random"

    def get_next_point(self):
        return self.search_space.generate_quasi_random_points_in_domain(1)


class ExpectedImprovementRunner(BayesianOptimization):
    def __init__(self, search_space):
        super().__init__(search_space)
        self.opt_name = "ei"

    def get_next_point(self):
        ei = ExpectedImprovement(self.gaussian_process, self.search_space)
        return ei.next_point_grad()


class UpperConfidenceBoundRunner(BayesianOptimization):
    def __init__(self, search_space, base_kappa=1):
        super().__init__(search_space)
        self.opt_name = "ucb" + str(base_kappa)
        self.base_kappa = base_kappa

    def get_next_point(self):
        kwargs = {"base_kappa": self.base_kappa}
        ucb = UpperConfidenceBound(self.gaussian_process, self.search_space, **kwargs)
        return ucb.next_point_grad()


class KnowledgeGradientRunner(BayesianOptimization):
    def __init__(self, search_space):
        super().__init__(search_space)
        self.opt_name = "kg"

    def get_next_point(self):
        kg = KnowledgeGradient(self.gaussian_process, self.search_space)
        return kg.next_point_grid()


class RolloutRunner(BayesianOptimization):
    def __init__(self, search_space, horizon):
        super().__init__(search_space)
        self.horizon = horizon
        self.opt_name = "ei" + str(horizon)

    def get_next_point(self):
        eih = RolloutEI_VR(self.gaussian_process, self.search_space, self.horizon)
        return eih.next_point()

## lookahead/test_problems/unconstrained.py

In [ ]:
"""
NOTE 1: all test functions designed to be evaluated on cube [0.1]^d
Make sure domains are slighly uneven (b/c the minimum is often in
the exact middle of the domain for these test functions)
NOTE 2: most test functions' implementations copied directly either from
SFU Virtual Library of Simulation Experiments:
https://www.sfu.ca/~ssurjano/index.html
or from pySOT:
https://github.com/dme65/pySOT/blob/master/pySOT/optimization_problems.py
"""

import numpy as np


def from_unit_box(x, lb, ub):
    return lb + (ub - lb) * x


# Evaluated from [-5, 10]^2
def rosenbrock(x):
    lb = np.full((2,), -5)
    ub = np.full((2,), 10)
    x = from_unit_box(x, lb, ub)
    x1 = x[:, 0]
    x2 = x[:, 1]
    a = 1.0 - x1
    b = x2 - x1 * x1
    return a * a + b * b * 100.0


# Evaluated by default from [-4, 5]^d
def ackley(x, lb=None, ub=None):
    n, d = x.shape
    if lb is None or ub is None:
        lb = np.full((d,), -4)
        ub = np.full((d,), 5)
    x = from_unit_box(x, lb, ub)
    return (
        -20.0 * np.exp(-0.2 * np.sqrt(np.sum(x**2, axis=1) / d))
        - np.exp(np.sum(np.cos(2.0 * np.pi * x), axis=1) / d)
        + 20
        + np.exp(1)
    )


# Evaluated by default from [-2.5, 3]^d
def rastrigin(x, lb=None, ub=None):
    n, d = x.shape
    if lb is None or ub is None:
        lb = np.full((d,), -2.5)
        ub = np.full((d,), 3)
    x = from_unit_box(x, lb, ub)
    return 10 * d + np.sum(x**2 - 10 * np.cos(2 * np.pi * x), axis=1)


# Evaluated by default from [-1.8, 2.2]^2
def sixhump(x, lb=None, ub=None):
    if x.shape[1] != 2:
        raise Exception("Dimension must be 2")
    d = 2
    if lb is None or ub is None:
        lb = np.full((d,), -1.8)
        ub = np.full((d,), 2.2)
    x = from_unit_box(x, lb, ub)
    return (
        (4.0 - 2.1 * x[:, 0] ** 2 + (x[:, 0] ** 4) / 3.0) * x[:, 0] ** 2
        + x[:, 0] * x[:, 1]
        + (-4 + 4 * x[:, 1] ** 2) * x[:, 1] ** 2
    )


# Evaluated from [-5, 10] x [0, 15]
def branin(x, lb=None, ub=None):
    if x.shape[1] != 2:
        raise Exception("Dimension must be 2")
    d = 2
    if lb is None or ub is None:
        lb = np.full((d,), 0)
        ub = np.full((d,), 0)
        lb[0] = -5
        lb[1] = 0
        ub[0] = 10
        ub[1] = 15
    x = from_unit_box(x, lb, ub)
    x1 = x[:, 0]
    x2 = x[:, 1]
    t = 1 / (8 * np.pi)
    s = 10
    r = 6
    c = 5 / np.pi
    b = 5.1 / (4 * np.pi**2)
    a = 1
    term1 = a * (x2 - b * x1**2 + c * x1 - r) ** 2
    term2 = s * (1 - t) * np.cos(x1)
    return term1 + term2 + s


def hartman3(x, lb=None, ub=None):
    if x.shape[1] != 3:
        raise Exception("Dimension must be 3")
    d = 3
    if lb is None or ub is None:
        lb = np.full((d,), 0)
        ub = np.full((d,), 1)
    x = from_unit_box(x, lb, ub)
    alpha = np.array([1, 1.2, 3, 3.2])
    A = np.array(
        [[3.0, 10.0, 30.0], [0.1, 10.0, 35.0], [3.0, 10.0, 30.0], [0.1, 10.0, 35.0]]
    )
    P = np.array(
        [
            [0.3689, 0.1170, 0.2673],
            [0.4699, 0.4387, 0.747],
            [0.1091, 0.8732, 0.5547],
            [0.0381, 0.5743, 0.8828],
        ]
    )
    outer = 0
    for ii in range(4):
        inner = 0
        for jj in range(3):
            xj = x[:, jj]
            Aij = A[ii, jj]
            Pij = P[ii, jj]
            inner += Aij * ((xj - Pij) ** 2)
        outer += alpha[ii] * np.exp(-inner)
    return -outer


def hartman4(x, lb=None, ub=None):
    if x.shape[1] != 4:
        raise Exception("Dimension must be 4")
    d = 4
    if lb is None or ub is None:
        lb = np.full((d,), 0)
        ub = np.full((d,), 1)
    x = from_unit_box(x, lb, ub)
    alpha = np.array([1.0, 1.2, 3.0, 3.2])
    A = np.array(
        [
            [10, 3, 17, 3.5, 1.7, 8],
            [0.05, 10, 17, 0.1, 8, 14],
            [3, 3.5, 1.7, 10, 17, 8],
            [17, 8, 0.05, 10, 0.1, 14],
        ]
    )
    P = 1e-4 * np.array(
        [
            [1312, 1696, 5569, 124, 8283, 5886],
            [2329, 4135, 8307, 3736, 1004, 9991],
            [2348, 1451, 3522, 2883, 3047, 6650],
            [4047, 8828, 8732, 5743, 1091, 381],
        ]
    )
    outer = 0
    for ii in range(4):
        inner = 0
        for jj in range(4):
            xj = x[:, jj]
            Aij = A[ii, jj]
            Pij = P[ii, jj]
            inner = inner + Aij * ((xj - Pij) ** 2)
        outer += alpha[ii] * np.exp(-inner)
    return (1.1 - outer) / 0.839


def levy(x, lb=None, ub=None):
    d = x.shape[1]
    if lb is None or ub is None:
        lb = -5 * np.full((d,), 1)
        ub = 5 * np.full((d,), 1)
    x = from_unit_box(x, lb, ub)
    w = 1 + (x - 1.0) / 4.0
    return (
        np.sin(np.pi * w[:, 0]) ** 2
        + np.sum(
            (w[:, 1 : d - 1] - 1) ** 2
            * (1 + 10 * np.sin(np.pi * w[:, 1 : d - 1] + 1) ** 2),
            axis=1,
        )
        + (w[:, d - 1] - 1) ** 2 * (1 + np.sin(2 * np.pi * w[:, d - 1]) ** 2)
    )


def hartman6(x, lb=None, ub=None):
    if x.shape[1] != 6:
        raise Exception("Dimension must be 6")
    d = 6
    if lb is None or ub is None:
        lb = np.full((d,), 0)
        ub = np.full((d,), 1)
    x = from_unit_box(x, lb, ub)
    alpha = np.array([1.0, 1.2, 3.0, 3.2])
    A = np.array(
        [
            [10.0, 3.0, 17.0, 3.5, 1.7, 8.0],
            [0.05, 10.0, 17.0, 0.1, 8.0, 14.0],
            [3.0, 3.5, 1.7, 10.0, 17.0, 8.0],
            [17.0, 8.0, 0.05, 10.0, 0.1, 14.0],
        ]
    )
    P = 1e-4 * np.array(
        [
            [1312.0, 1696.0, 5569.0, 124.0, 8283.0, 5886.0],
            [2329.0, 4135.0, 8307.0, 3736.0, 1004.0, 9991.0],
            [2348.0, 1451.0, 3522.0, 2883.0, 3047.0, 6650.0],
            [4047.0, 8828.0, 8732.0, 5743.0, 1091.0, 381.0],
        ]
    )
    outer = 0
    for ii in range(4):
        inner = 0
        for jj in range(6):
            xj = x[:, jj]
            Aij = A[ii, jj]
            Pij = P[ii, jj]
            inner += Aij * ((xj - Pij) ** 2)
        outer += alpha[ii] * np.exp(-inner)
    return -outer

## test

In [ ]:
test = False
if test:
    opt_domain = TensorProductDomain([ClosedInterval(0, 1)])

    fig, ax = plt.subplots(1, 1)
    xtrain = np.linspace(0.15, 0.85, 2)[:, None]
    ytrain = np.array([0, 0])
    gp = GaussianProcessSimple(xtrain, ytrain)
    gp.set_hypers([5, 0.25])
    x = np.linspace(0, 1, 40)[:, None]
    y_gp = gp.mean(x)
    y_var = np.sqrt(gp.variance(x))
    _ = ax.plot(x, y_gp, color="r")
    _ = ax.plot(xtrain, ytrain, "k.", markersize=15)
    _ = ax.fill_between(x[:, 0], y_gp - y_var, y_gp + y_var, color="m", alpha=0.25)
    _ = ax.legend(["GP mean", "Observations", "GP uncertainty"])

    (fig, ax) = plt.subplots(1, 2, figsize=(8, 3))
    ei = ExpectedImprovement(gp, opt_domain)
    ei_vals = ei.evaluate_at_point_list(x)

    # We estimate EI2 using 100 MC iterations, using grid search of size 50 to maximize the inner EI
    ei2 = RolloutEI_VR(
        gp, opt_domain, horizon=2, opt_mode="grid", mc_iters=20, grid_size=100
    )
    ei2_vals = ei2.evaluate_at_point_list(x)

    _ = ax[0].plot(x, ei_vals, "--g")
    _ = ax[1].plot(x, ei2_vals, "--g")
    _ = ax[0].set_title("EI Acquisition")
    _ = ax[1].set_title("EI2 Acquisition")

# Exp

In [ ]:
# @title HES
import copy

n_dim = "1"  # @param [1, 2]
n_dim = int(n_dim)
nonmyopic = True  # @param {type:"boolean"}
budget = 20  # @param {type:"integer"}

save = True
draw_legend = False

if n_dim == 1:
    dim_xi = [n_dim]
    prev_x = torch.tensor([0.3])
    prev_x_str = "0.3"
elif n_dim == 2:
    dim_xi = [1, n_dim]
    prev_x = torch.tensor([[0.4, 0.4]])
    prev_x_str = "0.4_0.4"

neighbor_size = 1 / budget  # 0.02
n_init = 0
ehig_opt_epoch = 50
ehig_opt_lr = 0.1
use_lr_schedule = False
train = False

seeds = [0, 1, 2]

optimal_loss_results = []

exp_name = f"{n_dim}D_nonmyopic{nonmyopic}_budget{budget}_\
neighbor_size{neighbor_size}_prev_x{prev_x_str}"

# create folder
if not os.path.exists(f"{path}/{exp_name}"):
    os.makedirs(f"{path}/{exp_name}")
    print(f"Saving to: {path}/{exp_name}")

for seed in seeds:
    if n_dim == 1:
        dim_xi = [n_dim]
        prev_x = torch.tensor([0.3])
    elif n_dim == 2:
        dim_xi = [1, n_dim]
        prev_x = torch.tensor([[0.4, 0.4]])

    if n_dim == 2:
        print("-" * 20, "Ground truth surface", "-" * 20)
        ground_truth(draw_true_model=True, n_dim=n_dim)
        plt.show()

    train_x, train_y = init_data(n_init=n_init)

    model = init_model(
        n_dim=n_dim, prev_x=prev_x, train_x=train_x, train_y=train_y, train=train
    )

    if nonmyopic:
        horizon = budget
    else:
        horizon = 1
    optimal_loss = []
    for step in range(budget):
        best_result = argmax_ehig(
            model,
            prev_x,
            horizon=horizon,
            ehig_opt_epoch=ehig_opt_epoch,
            ehig_opt_lr=ehig_opt_lr,
            neighbor_size=neighbor_size,
            use_lr_schedule=use_lr_schedule,
            seed=seed,
        )

        if nonmyopic:
            horizon = horizon - 1

        with torch.no_grad():
            next_x = best_result[0]
            next_y = func(next_x.reshape(-1, n_dim))

            # draw_posterior(n_dim=n_dim, model=model)
            # if n_dim == 2: plt.show()

            # update data and model
            train_x = torch.cat([train_x, next_x.reshape(-1, n_dim)])
            train_y = torch.cat([train_y, next_y])

            model = init_model(
                n_dim=n_dim,
                prev_x=prev_x,
                train_x=train_x,
                train_y=train_y,
                train=train,
            )

        # eval
        a = (torch.rand([n_dim] if n_dim == 1 else [1, n_dim])).requires_grad_(True)
        optimizer = optim.Adam([a], lr=ehig_opt_lr)

        loss = []
        for epoch in range(10):
            optimizer.zero_grad()

            a = torch.sigmoid(a) * (neighbor_size * 2) + (next_x - neighbor_size)

            p_y_on_a_D = model.posterior(a)
            ell = p_y_on_a_D.mean.mean()
            ell.backward(retain_graph=True)
            optimizer.step()
            loss.append(ell.clone().detach())

        loss = torch.stack(loss).squeeze()
        print(f"Current optimal loss {loss.min().item()};" f"optimal action: {a}")
        optimal_loss.append(loss.min())

        if n_dim == 1:
            dim_xi = [n_dim]
        elif n_dim == 2:
            dim_xi = [1, n_dim]

        # plot & update prev_x
        if n_dim == 1:
            ground_truth(draw_true_model=True, n_dim=n_dim)

        plt.title(f"Step {step} with horizon of {horizon}")

        # draw_posterior(n_dim=n_dim, model=model)

        if n_dim == 1:
            dmin = -4
            dmax = 4
            plt.vlines(prev_x, dmin, dmax, color=color["C5"], label="Current location")
            plt.vlines(
                prev_x - neighbor_size, dmin, dmax, color="black", linestyle="--"
            )
            plt.vlines(
                prev_x + neighbor_size, dmin, dmax, color="black", linestyle="--"
            )
            prev_x = next_x
            plt.plot(
                train_x.squeeze().cpu().numpy(), train_y.squeeze().cpu().numpy(), "k*"
            )
            plt.vlines(
                best_result[0], dmin, dmax, color=color["C3"], label="Optimal query"
            )
            plt.vlines(
                best_result[-1], dmin, dmax, color=color["C4"], label="Optimal action"
            )
            plt.ylim(dmin, dmax)

        elif n_dim == 2:
            plt.hlines(prev_x[0, 1] - neighbor_size, 0, 1, linestyle="--")
            plt.hlines(prev_x[0, 1] + neighbor_size, 0, 1, linestyle="--")
            plt.vlines(prev_x[0, 0] - neighbor_size, 0, 1, linestyle="--")
            plt.vlines(prev_x[0, 0] + neighbor_size, 0, 1, linestyle="--")
            plt.scatter(
                prev_x[0, 0], prev_x[0, 1], color=color["C5"], label="Current location"
            )
            prev_x = next_x
            plt.scatter(train_x[:, 0], train_x[:, 1], marker="*", color="black")
            plt.scatter(
                best_result[0][0, 0],
                best_result[0][0, 1],
                color=color["C3"],
                label="Optimal query",
            )
            plt.scatter(
                best_result[-1][..., 0],
                best_result[-1][..., 1],
                color=color["C4"],
                label="Optimal action",
            )
            plt.ylim(0, 1)

        plt.tight_layout()
        if draw_legend:
            plt.legend()

        if save:
            plt.savefig(
                f"{path}/{exp_name}/seed{seed}_step{step}.{imgtype}",
                dpi=dpi,
                bbox_inches="tight",
            )

        plt.show()

    # plot optimal loss over query
    optimal_loss = torch.stack(optimal_loss).numpy()

    optimal_loss_results.append(optimal_loss)

optimal_loss_results = np.array(optimal_loss_results)
mean = optimal_loss_results.mean(0)
std = optimal_loss_results.std(0)

plt.plot(-mean, label=color["C1"])
plt.fill_between(
    np.arange(len(mean)),
    -(mean - 2 * std),
    -(mean + 2 * std),
    color=color["C2"],
    alpha=0.1,
)

plt.show()

np.save(f"{path}/{exp_name}/data.npy", optimal_loss_results)

In [ ]:
# @title Random
import copy

n_dim = "2"  # @param [1, 2]
n_dim = int(n_dim)
nonmyopic = True  # @param {type:"boolean"}
budget = 20  # @param {type:"integer"}

save = True
draw_legend = False

if n_dim == 1:
    dim_xi = [n_dim]
    prev_x = torch.tensor([0.3])
    prev_x_str = "0.3"
elif n_dim == 2:
    dim_xi = [1, n_dim]
    prev_x = torch.tensor([[0.4, 0.4]])
    prev_x_str = "0.4_0.4"

neighbor_size = 1 / budget  # 0.02
n_init = 0
ehig_opt_epoch = 50
ehig_opt_lr = 0.1
use_lr_schedule = False
train = False

seeds = [0, 1, 2]

optimal_loss_results = []

exp_name = f"{n_dim}D_nonmyopic{nonmyopic}_budget{budget}_\
neighbor_size{neighbor_size}_prev_x{prev_x_str}"

# create folder
if not os.path.exists(f"{path}/{exp_name}"):
    os.makedirs(f"{path}/{exp_name}")
    print(f"Saving to: {path}/{exp_name}")

for seed in seeds:
    if n_dim == 1:
        dim_xi = [n_dim]
        prev_x = torch.tensor([0.3])
    elif n_dim == 2:
        dim_xi = [1, n_dim]
        prev_x = torch.tensor([[0.4, 0.4]])

    if n_dim == 2:
        print("-" * 20, "Ground truth surface", "-" * 20)
        ground_truth(draw_true_model=True, n_dim=n_dim)
        plt.show()

    train_x, train_y = init_data(n_init=n_init)

    model = init_model(
        n_dim=n_dim, prev_x=prev_x, train_x=train_x, train_y=train_y, train=train
    )

    if nonmyopic:
        horizon = budget
    else:
        horizon = 1
    optimal_loss = []
    for step in range(budget):
        with torch.no_grad():
            if n_dim == 1:
                low = (prev_x - neighbor_size).item()
                high = (prev_x + neighbor_size).item()
                next_x = torch.FloatTensor(1).uniform_(low, high)

                print(next_x)
            elif n_dim == 2:
                low0 = (prev_x[0, 0] - neighbor_size).item()
                high0 = (prev_x[0, 0] + neighbor_size).item()
                next_x0 = torch.FloatTensor(1, 1).uniform_(low0, high0)
                low1 = (prev_x[0, 1] - neighbor_size).item()
                high1 = (prev_x[0, 1] + neighbor_size).item()
                next_x1 = torch.FloatTensor(1, 1).uniform_(low1, high1)
                next_x = torch.cat([next_x0, next_x1], dim=1)

            next_y = func(next_x.reshape(-1, n_dim))

            # draw_posterior(n_dim=n_dim, model=model)
            # if n_dim == 2: plt.show()

            # update data and model
            train_x = torch.cat([train_x, next_x.reshape(-1, n_dim)])
            train_y = torch.cat([train_y, next_y])

            model = init_model(
                n_dim=n_dim,
                prev_x=prev_x,
                train_x=train_x,
                train_y=train_y,
                train=train,
            )

        # eval
        a = (torch.rand([n_dim] if n_dim == 1 else [1, n_dim])).requires_grad_(True)
        optimizer = optim.Adam([a], lr=ehig_opt_lr)

        loss = []
        for epoch in range(10):
            optimizer.zero_grad()

            a = torch.sigmoid(a) * (neighbor_size * 2) + (next_x - neighbor_size)

            p_y_on_a_D = model.posterior(a)
            ell = p_y_on_a_D.mean.mean()
            ell.backward(retain_graph=True)
            optimizer.step()
            loss.append(ell.clone().detach())

        loss = torch.stack(loss).squeeze()
        print(f"Current optimal loss {loss.min().item()};" f"optimal action: {a}")
        optimal_loss.append(loss.min())

        if n_dim == 1:
            dim_xi = [n_dim]
        elif n_dim == 2:
            dim_xi = [1, n_dim]

        # plot & update prev_x
        if n_dim == 1:
            ground_truth(draw_true_model=True, n_dim=n_dim)

        plt.title(f"Step {step} with horizon of {horizon}")

        # draw_posterior(n_dim=n_dim, model=model)

        if n_dim == 1:
            dmin = -4
            dmax = 4
            plt.vlines(prev_x, dmin, dmax, color=color["C5"], label="Current location")
            plt.vlines(
                prev_x - neighbor_size, dmin, dmax, color="black", linestyle="--"
            )
            plt.vlines(
                prev_x + neighbor_size, dmin, dmax, color="black", linestyle="--"
            )
            prev_x = next_x
            plt.plot(
                train_x.squeeze().cpu().numpy(), train_y.squeeze().cpu().numpy(), "k*"
            )
            plt.vlines(
                best_result[0], dmin, dmax, color=color["C3"], label="Optimal query"
            )
            plt.vlines(
                best_result[-1], dmin, dmax, color=color["C4"], label="Optimal action"
            )
            plt.ylim(dmin, dmax)

        elif n_dim == 2:
            plt.hlines(prev_x[0, 1] - neighbor_size, 0, 1, linestyle="--")
            plt.hlines(prev_x[0, 1] + neighbor_size, 0, 1, linestyle="--")
            plt.vlines(prev_x[0, 0] - neighbor_size, 0, 1, linestyle="--")
            plt.vlines(prev_x[0, 0] + neighbor_size, 0, 1, linestyle="--")
            plt.scatter(
                prev_x[0, 0], prev_x[0, 1], color=color["C5"], label="Current location"
            )
            prev_x = next_x
            plt.scatter(train_x[:, 0], train_x[:, 1], marker="*", color="black")
            plt.scatter(
                best_result[0][0, 0],
                best_result[0][0, 1],
                color=color["C3"],
                label="Optimal query",
            )
            plt.scatter(
                best_result[-1][..., 0],
                best_result[-1][..., 1],
                color=color["C4"],
                label="Optimal action",
            )
            plt.ylim(0, 1)

        plt.tight_layout()
        if draw_legend:
            plt.legend()

        if save:
            plt.savefig(
                f"{path}/{exp_name}/seed{seed}_step{step}.{imgtype}",
                dpi=dpi,
                bbox_inches="tight",
            )

        plt.show()

    # plot optimal loss over query
    optimal_loss = torch.stack(optimal_loss).numpy()

    optimal_loss_results.append(optimal_loss)

optimal_loss_results = np.array(optimal_loss_results)
mean = optimal_loss_results.mean(0)
std = optimal_loss_results.std(0)

plt.plot(-mean, label=color["C1"])
plt.fill_between(
    np.arange(len(mean)),
    -(mean - 2 * std),
    -(mean + 2 * std),
    color=color["C2"],
    alpha=0.1,
)

plt.show()

np.save(f"{path}/{exp_name}/data.npy", optimal_loss_results)

In [ ]:
# @title DDPG

#!/usr/bin/env python
"""
Reinformcement learning baseline for non-myopic
Bayesian optimization
"""

# from synthfunc import SynthFunc
from collections import namedtuple
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import pyplot as plt
from argparse import Namespace, ArgumentParser

__author__ = "Sang T. Truong, Willie Neiswanger, Shengjia Zhao, Stefano Ermon"
__copyright__ = "Copyright 2022, Stanford University"

import gym
import random
import torch
import torch.nn as nn
import torch.nn.functional as F

float_formatter = "{:.2f}".format
np.set_printoptions(formatter={"float_kind": float_formatter})

n_dim = 2


class SynthFuncEnv(gym.Env):
    def __init__(self, n_dim):
        self.T = 100
        self.state_dim = 1
        self.n_dim = n_dim
        hypers = {"ls": 1.0, "alpha": 2.0, "sigma": 1e-2, "n_dimx": self.n_dim}
        self.sf_bounds = [0, 10]
        sf_domain = [self.sf_bounds] * self.n_dim
        self.lower_bounds = [self.sf_bounds[0]] * self.n_dim
        self.upper_bounds = [self.sf_bounds[1]] * self.n_dim
        self.action_space = gym.spaces.Box(
            low=np.array(self.lower_bounds, dtype=np.float32),
            high=np.array(self.upper_bounds, dtype=np.float32),
        )
        self.state = torch.tensor([])
        self.sf = func

    def step(self, action):
        act_input = action.reshape(-1, n_dim)
        reward = func(act_input)

        self.state = torch.cat([self.state, action, torch.tensor([reward])])
        self.state_dim = self.state_dim + self.n_dim + 1 if self.state_dim > 1 else 3
        self.T = self.T - 1
        done = False if self.T > 0 else True
        return self.state, reward, done

    def reset(self):
        """
        Return the initial state
        """
        return torch.tensor(0)

    def update_action_bound(self, action, r):
        self.lower_bounds = []
        self.upper_bounds = []
        for i in range(self.n_dim):
            lower_bound = action[i].item() - r
            lower_bound = (
                lower_bound if lower_bound >= self.sf_bounds[0] else self.sf_bounds[0]
            )
            self.lower_bounds.append(lower_bound)

            upper_bound = action[i].item() + r
            upper_bound = (
                upper_bound if upper_bound <= self.sf_bounds[1] else self.sf_bounds[1]
            )
            self.upper_bounds.append(upper_bound)

        self.action_space = gym.spaces.Box(
            low=np.array(self.lower_bounds, dtype=np.float32),
            high=np.array(self.upper_bounds, dtype=np.float32),
        )

        self.lower_bounds = torch.tensor(self.lower_bounds)
        self.upper_bounds = torch.tensor(self.upper_bounds)

        return self.lower_bounds, self.upper_bounds


class ReplayBuffer:
    def __init__(self, capacity):
        self.capacity = capacity
        self.buffer = []
        self.position = 0
        self.transition = namedtuple(
            "Transition", ("state", "action", "next_state", "reward", "done")
        )

    def push(self, *args):
        if len(self.buffer) < self.capacity:
            self.buffer.append(None)
        self.buffer[self.position] = self.transition(*args)
        self.position = (self.position + 1) % self.capacity

    def sample(self, batch_size):
        return random.sample(self.buffer, batch_size)

    def __len__(self):
        return len(self.buffer)


class Noise:
    def __init__(self, params):
        self.mu = params.mu
        self.theta = params.theta
        self.sigma = params.sigma
        self.reset()

    def reset(self):
        self.state = np.full(params.action_dim, self.mu)

    def make_noise(self):
        state = self.state
        delta = self.theta * (self.mu - state) + self.sigma * np.random.randn(
            len(state)
        )
        self.state = state + delta
        return self.state


class Actor(nn.Module):
    def __init__(self, params):
        super(Actor, self).__init__()
        self.fc1 = nn.Linear(params.state_dim, params.act_hid_1)
        self.fc2 = nn.Linear(params.act_hid_1, params.act_hid_2)
        self.fc3 = nn.Linear(params.act_hid_2, params.action_dim)

    def forward(self, x):
        self.__init__(params)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return torch.tanh(self.fc3(x))


class Critic(nn.Module):
    def __init__(self, params):
        super(Critic, self).__init__()

        self.fc1 = nn.Linear(params.state_dim, params.crit_hid_1)
        self.fc2 = nn.Linear(params.crit_hid_1 + params.action_dim, params.crit_hid_2)
        self.fc3 = nn.Linear(params.crit_hid_2, 1)

    def forward(self, x, action):
        x = F.relu(self.fc1(x))
        x = torch.cat((x, action), dim=1)
        x = F.relu(self.fc2(x))
        return self.fc3(x)


class Model:
    def __init__(self, params):
        self.device = params.device

        self.actor = Actor(params).to(self.device)
        self.actor_target = Actor(params).to(self.device)
        self.critic = Critic(params).to(self.device)
        self.critic_target = Critic(params).to(self.device)

        self.actor_optimizer = torch.optim.Adam(
            self.actor.parameters(), lr=params.lr_actor
        )
        self.critic_optimizer = torch.optim.Adam(
            self.critic.parameters(), lr=params.lr_critic
        )

        self.tau_actor = params.tau_actor
        self.tau_critic = params.tau_critic

        self.__update(self.actor_target, self.actor)
        self.__update(self.critic_target, self.critic)

    def __update(self, target, local):
        target.load_state_dict(local.state_dict())

    def __soft_update(self, target, local, tau):
        for target_param, param in zip(target.parameters(), local.parameters()):
            target_param.data.copy_(target_param.data * (1.0 - tau) + param.data * tau)

    def update_target_nn(self):
        self.__soft_update(self.actor_target, self.actor, self.tau_actor)
        self.__soft_update(self.critic_target, self.critic, self.tau_critic)


class DDPG:
    def __init__(self, params):
        self.device = params.device
        self.gamma = params.gamma
        self.batch_size = params.batch_size
        self.act_up, self.act_down = params.act_up, params.act_down

        self.explor_noise = Noise(params)
        self.buffer = ReplayBuffer(params.buffer_size)
        self.model = Model(params)

    def update(self):
        if len(self.buffer) <= self.batch_size:
            return

        transitions = self.buffer.sample(self.batch_size)
        batch = self.buffer.transition(*zip(*transitions))

        state_batch = self.tensor(batch.state).float()
        action_batch = self.tensor(batch.action).float()
        reward_batch = self.tensor(batch.reward).float()
        next_state_batch = self.tensor(batch.next_state).float()

        with torch.no_grad():
            next_actions = self.model.actor_target(next_state_batch)
        Q_next = self.model.critic_target(next_state_batch, next_actions).detach()

        reward_batch = reward_batch.unsqueeze(1)
        not_terminate_batch = ~torch.tensor(batch.done).to(self.device).unsqueeze(1)

        Q = self.model.critic(state_batch, action_batch)
        Q_expected = reward_batch + self.gamma * Q_next * not_terminate_batch

        L = F.mse_loss(Q, Q_expected)
        self.model.critic_optimizer.zero_grad()
        L.backward()
        for param in self.model.critic.parameters():
            param.grad.data.clamp_(-1, 1)
        self.model.critic_optimizer.step()

        a = self.model.actor(state_batch)
        L_policy = -self.model.critic(state_batch, a).mean()
        self.model.actor_optimizer.zero_grad()
        L_policy.backward()
        for param in self.model.actor.parameters():
            param.grad.data.clamp_(-1, 1)
        self.model.actor_optimizer.step()

        self.model.update_target_nn()

    def act(self, state, eps=0):
        state = state.float().unsqueeze(0)
        with torch.no_grad():
            action = self.model.actor(state)
        action = action + eps * self.explor_noise.make_noise()
        return np.clip(action, self.act_down, self.act_up)

    def reset(self):
        self.explor_noise.reset()

    def update_action_bound(self):
        self.act_up, self.act_down = params.act_up, params.act_down


class Paramaters:
    def __init__(self, env):
        self.env = env
        self.state_dim = env.state_dim  # env.observation_space.shape[0]
        self.action_dim = env.action_space.shape[0]
        self.act_up = env.upper_bounds  # env.action_space.high[0]
        self.act_down = env.lower_bounds  # env.action_space.low[0]

        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.seed = 1
        self.gamma = 0.9
        self.episodes = 1  # there is a single episode
        self.max_steps = 20
        self.batch_size = 256
        self.buffer_size = int(1e4)

        self.eps = 1.0
        self.eps_decay = 0.95
        self.eps_min = 0.01

        self.tau_actor = 0.1
        self.tau_critic = 0.3
        self.lr_actor = 1e-4
        self.lr_critic = 1e-3

        self.mu = 0
        self.theta = 0.15
        self.sigma = 0.2

        self.act_hid_1, self.act_hid_2 = 128, 128
        self.crit_hid_1, self.crit_hid_2 = 128, 128

        self.reward_coef = 20

        self.froze_seed()

        self.r = 0.01  # spotlight radius

    def froze_seed(self):
        self.env.reset()
        torch.manual_seed(self.seed)
        np.random.seed(self.seed)
        random.seed(self.seed)

    def update_eps(self):
        self.eps = max(self.eps_min, self.eps * self.eps_decay)

    def update_state_dim(self):
        self.state_dim = self.env.state_dim

    def update_action_bound(self, action):
        self.env.update_action_bound(action, self.r)
        self.act_down = env.lower_bounds
        self.act_up = env.upper_bounds


def run(params, agent, plot_reward=True, plot_action=True):
    rewards = []
    actions = []
    for i in range(1, params.episodes + 1):
        state = env.reset()
        agent.reset()
        params.update_eps()
        total_reward, steps = 0, 0
        done = False
        t = 1
        while not done:
            action = agent.act(state, params.eps).squeeze()
            if len(actions) > 0:
                d = ((action - actions[-1]) ** 2).sum().sqrt()
                # print(d)
            actions.append(action)
            params.update_action_bound(action)
            agent.update_action_bound()
            next_state, reward, done = env.step(action)
            params.update_state_dim()
            agent.buffer.push(state, action, next_state, reward, done)
            state = next_state
            agent.update()
            # total_reward += reward
            rewards.append(reward)
            steps += 1

            # print("Time step {}, Action: {}, Reward {:.2f}".format(
            #     t, action.numpy(), reward))
            t = t + 1
        # rewards.append(total_reward)
        # print(f"Episode {i}, reward: {total_reward}")

    if plot_action:
        actions = torch.stack(actions)
        # print(actions.shape)
        plt.scatter(actions[:, 0], actions[:, 1], c=torch.arange(0, actions.shape[0]))
        plt.title("Action")
        plt.show()

    rewards = torch.cat(rewards).numpy()

    # print(rewards)
    # sang

    if plot_reward:
        plt.plot(rewards)
        plt.ylabel("Reward")
        plt.xlabel("Episodes")
        plt.title("Training scores")
        plt.show()
    return agent


if __name__ == "__main__":
    env = SynthFuncEnv(n_dim=2)
    params = Paramaters(env)
    agent = DDPG(params)
    run(params=params, agent=agent)

In [ ]:
# @title BudgetBO
# from botorch.settings import debug
# from math import pi
# from typing import Callable
# torch.set_default_dtype(torch.float64)
# debug._set_state(True)

# script_dir = os.path.dirname(os.path.realpath(sys.argv[0]))
# sys.path.append(script_dir[:-12])
# sys.path.append(script_dir[:-11] + "budgeted_bo")
# sys.path.append(script_dir[:-11] + "budgeted_bo/acquisition_functions")


# Algos
algo = "B-MS-EI"

if algo == "B-MS-EI":
    algo_params = {
        "lookahead_n_fantasies": [1, 1, 1],
        "refill_until_lower_bound_is_reached": True,
        "soft_plus_transform_budget": False,
    }
else:
    algo_params = {}

n_init_evals = 8
n_dim = "1"  # @param [1, 2]
n_dim = int(n_dim)
nonmyopic = True  # @param {type:"boolean"}
budget = 20  # @param {type:"integer"}
save = True
draw_legend = False

if n_dim == 1:
    dim_xi = [n_dim]
    prev_x = torch.tensor([0.3])
    prev_x_str = "0.3"
elif n_dim == 2:
    dim_xi = [1, n_dim]
    prev_x = torch.tensor([[0.4, 0.4]])
    prev_x_str = "0.4_0.4"

neighbor_size = 1 / budget  # 0.02
n_init = 0
ehig_opt_epoch = 50
ehig_opt_lr = 0.1
use_lr_schedule = False
train = False

seeds = [0, 1, 2]


# Run experiment
first_trial = 0
last_trial = 2
# if len(sys.argv) == 3:
#     first_trial = int(sys.argv[1])
#     last_trial =  int(sys.argv[2])
# elif len(sys.argv) == 2:
#     first_trial = int(sys.argv[1])
#     last_trial =  int(sys.argv[1])

optimal_loss_results = []

exp_name = f"{n_dim}D_nonmyopic{nonmyopic}_budget{budget}_\
neighbor_size{neighbor_size}_prev_x{prev_x_str}"

# create folder
if not os.path.exists(f"{path}/{exp_name}"):
    os.makedirs(f"{path}/{exp_name}")
    print(f"Saving to: {path}/{exp_name}")


Xs, objective_Xs = experiment_manager(
    problem="ackley",
    algo=algo,
    algo_params=algo_params,
    restart=False,
    first_trial=first_trial,
    last_trial=last_trial,
    get_objective_cost_function=get_objective_cost_function,
    input_dim=n_dim,
    n_init_evals=n_init_evals,
    budget=60.0,
    n_max_iter=20,
)


for seed in seeds:
    X = Xs[seed]
    objective_X = objective_Xs[seed]

    train_x = X[:n_init_evals]
    train_y = objective_X[:n_init_evals]

    model = init_model(
        n_dim=n_dim, prev_x=prev_x, train_x=train_x, train_y=train_y, train=train
    )

    if nonmyopic:
        horizon = budget
    else:
        horizon = 1
    optimal_loss = []
    for step in range(budget):
        with torch.no_grad():
            next_x = X[n_init_evals + step]
            next_y = objective_X[n_init_evals + step]

            # draw_posterior(n_dim=n_dim, model=model)
            # if n_dim == 2: plt.show()

            # update data and model
            train_x = torch.cat([train_x, next_x.reshape(-1, n_dim)])
            train_y = torch.cat([train_y, next_y])

            model = init_model(
                n_dim=n_dim,
                prev_x=prev_x,
                train_x=train_x,
                train_y=train_y,
                train=train,
            )

        # eval
        a = (torch.rand([n_dim] if n_dim == 1 else [1, n_dim])).requires_grad_(True)
        optimizer = optim.Adam([a], lr=ehig_opt_lr)

        loss = []
        for epoch in range(10):
            optimizer.zero_grad()

            a = torch.sigmoid(a) * (neighbor_size * 2) + (next_x - neighbor_size)

            p_y_on_a_D = model.posterior(a)
            ell = p_y_on_a_D.mean.mean()
            ell.backward(retain_graph=True)
            optimizer.step()
            loss.append(ell.clone().detach())

        loss = torch.stack(loss).squeeze()
        print(f"Current optimal loss {loss.min().item()};" f"optimal action: {a}")
        optimal_loss.append(loss.min())

        if n_dim == 1:
            dim_xi = [n_dim]
        elif n_dim == 2:
            dim_xi = [1, n_dim]

        # plot & update prev_x
        if n_dim == 1:
            ground_truth(draw_true_model=True, n_dim=n_dim)

        plt.title(f"Step {step} with horizon of {horizon}")

        # draw_posterior(n_dim=n_dim, model=model)

        if n_dim == 1:
            dmin = -4
            dmax = 4
            plt.vlines(prev_x, dmin, dmax, color=color["C5"], label="Current location")
            plt.vlines(
                prev_x - neighbor_size, dmin, dmax, color="black", linestyle="--"
            )
            plt.vlines(
                prev_x + neighbor_size, dmin, dmax, color="black", linestyle="--"
            )
            prev_x = next_x
            plt.plot(
                train_x.squeeze().cpu().numpy(), train_y.squeeze().cpu().numpy(), "k*"
            )
            plt.vlines(
                best_result[0], dmin, dmax, color=color["C3"], label="Optimal query"
            )
            plt.vlines(
                best_result[-1], dmin, dmax, color=color["C4"], label="Optimal action"
            )
            plt.ylim(dmin, dmax)

        elif n_dim == 2:
            plt.hlines(prev_x[0, 1] - neighbor_size, 0, 1, linestyle="--")
            plt.hlines(prev_x[0, 1] + neighbor_size, 0, 1, linestyle="--")
            plt.vlines(prev_x[0, 0] - neighbor_size, 0, 1, linestyle="--")
            plt.vlines(prev_x[0, 0] + neighbor_size, 0, 1, linestyle="--")
            plt.scatter(
                prev_x[0, 0], prev_x[0, 1], color=color["C5"], label="Current location"
            )
            prev_x = next_x
            plt.scatter(train_x[:, 0], train_x[:, 1], marker="*", color="black")
            plt.scatter(
                best_result[0][0, 0],
                best_result[0][0, 1],
                color=color["C3"],
                label="Optimal query",
            )
            plt.scatter(
                best_result[-1][..., 0],
                best_result[-1][..., 1],
                color=color["C4"],
                label="Optimal action",
            )
            plt.ylim(0, 1)

        plt.tight_layout()
        if draw_legend:
            plt.legend()

        if save:
            plt.savefig(
                f"{path}/{exp_name}/seed{seed}_step{step}.{imgtype}",
                dpi=dpi,
                bbox_inches="tight",
            )

        plt.show()

    # plot optimal loss over query
    optimal_loss = torch.stack(optimal_loss).numpy()

    optimal_loss_results.append(optimal_loss)

optimal_loss_results = np.array(optimal_loss_results)
mean = optimal_loss_results.mean(0)
std = optimal_loss_results.std(0)

plt.plot(-mean, label=color["C1"])
plt.fill_between(
    np.arange(len(mean)),
    -(mean - 2 * std),
    -(mean + 2 * std),
    color=color["C2"],
    alpha=0.1,
)

plt.show()

np.save(f"{path}/{exp_name}/data.npy", optimal_loss_results)

  0%|          | 0/3 [00:00<?, ?it/s]

Problem: ackley
Sampling policy: B-MS-EI_111_1_60
Trial: 0
Iteration: 1
Fantasy costs:
tensor([0.0087, 0.0094, 0.0097, 0.0095])
Acquisition values:
tensor([4.9270e-02, 3.3761e-02, 3.3761e-02, 3.3761e-02, 3.3761e-02, 3.3761e-02,
        3.3761e-02, 6.0077e-03, 6.0077e-03, 2.3409e-13], dtype=torch.float64)
Candidates:
tensor([0.4953, 0.4925, 0.4925, 0.4925, 0.4925, 0.4925, 0.4925, 0.4586, 0.4586,
        0.6423])
tensor([0.4586, 0.4925, 0.4953, 0.6423, 0.4925, 0.4925, 0.4925, 0.4586, 0.4925,
        0.4925])
Best value found so far: -0.02620410919189453
Remaining budget: 59.80655935406685
Problem: ackley
Sampling policy: B-MS-EI_111_1_60
Trial: 0
Iteration: 2
Fantasy costs:
tensor([0.3751, 0.1961, 0.3115, 0.2473])
Acquisition values:
tensor([1.6072e-02, 1.6072e-02, 1.5697e-02, 1.5323e-02, 1.5038e-02, 1.5037e-02,
        1.5036e-02, 1.4937e-02, 1.4936e-02, 5.0273e-06], dtype=torch.float64)
Candidates:
tensor([0.5027, 0.5027, 0.8986, 0.4741, 0.8901, 0.8913, 0.1326, 0.8972, 0.3652,
        

 33%|███▎      | 1/3 [09:29<18:58, 569.11s/it]

Acquisition values:
tensor([9.8761e-04, 8.5988e-04, 8.5988e-04, 8.5988e-04, 8.5952e-04, 8.5886e-04,
        8.5881e-04, 8.5870e-04, 8.5845e-04, 5.5764e-07], dtype=torch.float64)
Candidates:
tensor([0.3139, 0.5024, 0.5024, 0.5024, 0.7462, 0.0893, 0.8337, 0.8830, 0.7652,
        0.5209])
tensor([0.5024, 0.5024, 0.5024, 0.8337, 0.7652, 0.3139, 0.0893, 0.8830, 0.5209,
        0.7462])
Best value found so far: -0.012444496154785156
Remaining budget: 56.45750242471695
Problem: ackley
Sampling policy: B-MS-EI_111_1_60
Trial: 1
Iteration: 1
Acquisition values:
tensor([0.2861, 0.2710, 0.2709, 0.2707, 0.2679, 0.2667, 0.2584, 0.2507, 0.2467,
        0.0010], dtype=torch.float64)
Candidates:
tensor([0.5683, 0.4951, 0.2677, 0.1956, 0.4259, 1.0000, 1.0000, 0.3142, 1.0000,
        1.0000])
tensor([1.0000, 0.1956, 1.0000, 0.2677, 1.0000, 0.4259, 0.4951, 1.0000, 0.3142,
        0.5683])
Best value found so far: -0.8836782581414124
Remaining budget: 59.873373568058014
Problem: ackley
Sampling policy: B-